# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-d1ad8ea442-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Ancient Civilization explorative search") 

Consider the following exploratory information need:

> Investigate the Ancient Egypt and the related archeological explorations and findings.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q13444093`| Meidum Pyramid    | node |
| `wd:Q41127`| Poseidon   | node |





Also consider

```
wd:Q13444093 ?p ?obj .
```

is the BGP to retrieve all **properties of Meidum Pyramid**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.



The workload should:


1. Identify the BGP for Ancient Egypt

2. Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

3. Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

4. Identify the BGP for archeologist.

5. For each ancient civilization find the number of archeologists who worked on it (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

6. Find the BGP for deity.

7. List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

8. Compare ancient Greece with ancient Egypt and find how many deities each one has.


## Task 1
Identify the BGP for Ancient Egypt

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q13444093 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1481'), ('name', 'vici.org ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1566'), ('name', 'GeoNames ID')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P186'), ('name', 'made from material')]
[('p', 'http://www.wikidata.org/prop/direct/P2044'), ('name', 'elevation above sea level')]
[('p', 'http://www.wikidata.org/prop/direct/P2048'), ('name', 'height')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2326'), ('name', 'GNS Unique Feature ID')]
[('p', 'http://www.wi

In [3]:
# query example
# Exploring wd:Q13444093 Meidum Pyramid
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q13444093 ?p ?o.
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {?p sc:name ?pName}.
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q30683'), ('oName', 'Beni Suef Governorate')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after'), ('o', 'http://www.wikidata.org/entity/Q1066424'), ('oName', 'Meidum')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q79'), ('oName', 'Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P186'), ('pName', 'made from material'), ('o', 'http://www.wikidata.org/entity/Q23757'), ('oName', 'limestone')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('o', 'http://www.wikidata.org/entity/Q1066424'), ('oName', 'Meidum')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1456099'), ('oName', 'step pyramid')]
[('p', 'http://www.wikidata.org/prop/direct/P366'

In [4]:
# query example
# Exploring from wd:Q13444093 Meidum Pyramid

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q1456099 ?p ?o.
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {?p sc:name ?pName}.
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q12516'), ('oName', 'pyramid')]
[('p', 'http://www.wikidata.org/prop/direct/P461'), ('pName', 'opposite of'), ('o', 'http://www.wikidata.org/entity/Q4363262'), ('oName', 'smooth-sided pyramid')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image'), ('o', 'http://commons.wikimedia.org/wiki/Special:FilePath/Small%20pyramid.jpg')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pName', 'Commons category'), ('o', 'Step pyramids')]
[('p', 'http://www.wikidata.org/prop/direct/P4212'), ('pName', 'PACTOLS thesaurus ID'), ('o', 'pcrtGxsF5GefJE')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID'), ('o', '/m/05yb77')]
6


In [5]:
# TASK 1: Identify the BGP for Ancient Egypt
# Exploring from wd:Q13444093 Meidum Pyramid

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q12516 ?p ?o.
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {?p sc:name ?pName}.
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q3181656'), ('oName', 'The Nuttall Encyclopædia')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q867541'), ('oName', 'Encyclopædia Britannica 11th edition')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q106727050'), ('oName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P1419'), ('pName', 'shape'), ('o', 'http://www.wikidata.org/entity/Q3358290'), ('oName', 'pyramid')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q13361861'), ('oName', 'Template:Infobox pyramid')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pName', 'has quality'), ('o', 'http://www.wikidata.

In [6]:
# TASK 1: Identify the BGP for Ancient Egypt
# Exploring from wd:Q13444093 Meidum Pyramid

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')

# BACKWARD
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   ?o ?p wd:Q12516.
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {?p sc:name ?pName}.
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q13361861'), ('oName', 'Template:Infobox pyramid')]
[('p', 'http://www.wikidata.org/prop/direct/P123'), ('pName', 'publisher'), ('o', 'http://www.wikidata.org/entity/Q7769556'), ('oName', 'The Tomorrow People')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after'), ('o', 'http://www.wikidata.org/entity/Q2281305'), ('oName', 'The Pyramids')]
[('p', 'http://www.wikidata.org/prop/direct/P1419'), ('pName', 'shape'), ('o', 'http://www.wikidata.org/entity/Q97689488'), ('oName', 'La Pernoderie')]
[('p', 'http://www.wikidata.org/prop/direct/P1419'), ('pName', 'shape'), ('o', 'http://www.wikidata.org/entity/Q2226897'), ('oName', 'Pyramid Arena')]
[('p', 'http://www.wikidata.org/prop/direct/P180'), ('pName', 'depicts'), ('o', 'http://www.wikidata.org/entity/Q15155983'), ('oName', 'Memorial to the Soldiers and Sailors who died during the Siege

In [7]:
# TASK 1: Identify the BGP for Ancient Egypt
# Exploring from wd:Q13444093 Meidum Pyramid


# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')

# wd:Q175582 ('Egyptian pyramids') wdt:P279 ('subclass of') wd:Q12516 

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q175582 ?p ?o.
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   ?o ?p wd:Q12516.
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {?p sc:name ?pName}.
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19180675'), ('oName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q4114391'), ('oName', 'Sytin Military Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q570116'), ('oName', 'tourist attraction')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q602358'), ('oName', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q79'), ('oName', 'Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q125

In [8]:
# TASK 1: Identify the BGP for Ancient Egypt
# Exploring from wd:Q13444093 Meidum Pyramid

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')

# wd:Q175582 ('Egyptian pyramids') wdt:P279 ('subclass of') wd:Q12516 

# wd:Q175582 wdt:P17 ( 'country') wd:Q79 ('Egypt')

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q79 ?p ?o.
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2596'), ('pName', 'culture'), ('o', 'http://www.wikidata.org/entity/Q2655498'), ('oName', 'culture of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('pName', 'contains administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q29937'), ('oName', 'Aswan Governorate')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('pName', 'office held by head of government'), ('o', 'http://www.wikidata.org/entity/Q1571396'), ('oName', 'Prime Minister of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pName', 'language used'), ('o', 'http://www.wikidata.org/entity/Q29919'), ('oName', 'Egyptian Arabic')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1323642'), ('oName', 'transcontinental country')]
[('p', 'http://www.wikidata.org/prop/direct/P38'), ('pName', 'currency'), ('o', 'http://www.wikidata.org/entity/Q199462'), ('oName', 

In [9]:
# TASK 1: Identify the BGP for Ancient Egypt
# Exploring from wd:Q13444093 Meidum Pyramid

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')

# wd:Q175582 ('Egyptian pyramids') wdt:P279 ('subclass of') wd:Q12516 

# wd:Q175582 wdt:P17 ( 'country') wd:Q79 ('Egypt')

# FILTERING OUT wdt:P530 ('diplomatic relation')

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q79 ?p ?o.
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {?o sc:name ?oName}.
   FILTER (?p not in (wdt:P530))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2596'), ('pName', 'culture'), ('o', 'http://www.wikidata.org/entity/Q2655498'), ('oName', 'culture of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('pName', 'contains administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q29937'), ('oName', 'Aswan Governorate')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('pName', 'office held by head of government'), ('o', 'http://www.wikidata.org/entity/Q1571396'), ('oName', 'Prime Minister of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P2936'), ('pName', 'language used'), ('o', 'http://www.wikidata.org/entity/Q29919'), ('oName', 'Egyptian Arabic')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1323642'), ('oName', 'transcontinental country')]
[('p', 'http://www.wikidata.org/prop/direct/P38'), ('pName', 'currency'), ('o', 'http://www.wikidata.org/entity/Q199462'), ('oName', 

In [10]:
# TASK 1: Identify the BGP for Ancient Egypt
# Exploring from wd:Q13444093 Meidum Pyramid

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')

# wd:Q175582 ('Egyptian pyramids') wdt:P279 ('subclass of') wd:Q12516 
# wd:Q20504694 ('Modern Egyptian') wdt:P180 'depicts' wd:Q12516

# wd:Q175582 wdt:P17 ( 'country') wd:Q79 ('Egypt')

# BACK TO wd:Q20504694 ('Modern Egyptian')
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q20504694 ?p ?o.
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {?o sc:name ?oName}.
   #FILTER (?p not in (wdt:P530))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P180'), ('pName', 'depicts'), ('o', 'http://www.wikidata.org/entity/Q107425'), ('oName', 'landscape')]
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('pName', 'collection'), ('o', 'http://www.wikidata.org/entity/Q1192305'), ('oName', 'Smithsonian American Art Museum')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('o', 'http://www.wikidata.org/entity/Q1192305'), ('oName', 'Smithsonian American Art Museum')]
[('p', 'http://www.wikidata.org/prop/direct/P180'), ('pName', 'depicts'), ('o', 'http://www.wikidata.org/entity/Q134307'), ('oName', 'portrait')]
[('p', 'http://www.wikidata.org/prop/direct/P6216'), ('pName', 'copyright status'), ('o', 'http://www.wikidata.org/entity/Q19652'), ('oName', 'public domain')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pName', 'country of origin'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]
[('p', 'http://www.wikidata

In [11]:
# Identify the BGP for Ancient Egypt

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')

# wd:Q175582 ('Egyptian pyramids') wdt:P279 ('subclass of') wd:Q12516 
# wd:Q20504694 ('Modern Egyptian') wdt:P180 'depicts' wd:Q12516

# wd:Q175582 wdt:P17 ( 'country') wd:Q79 ('Egypt')

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   ?o ?p wd:Q12516.
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {?o sc:name ?oName}.
   FILTER (?p not in (wdt:P180))   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q13361861'), ('oName', 'Template:Infobox pyramid')]
[('p', 'http://www.wikidata.org/prop/direct/P123'), ('pName', 'publisher'), ('o', 'http://www.wikidata.org/entity/Q7769556'), ('oName', 'The Tomorrow People')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after'), ('o', 'http://www.wikidata.org/entity/Q2281305'), ('oName', 'The Pyramids')]
[('p', 'http://www.wikidata.org/prop/direct/P1419'), ('pName', 'shape'), ('o', 'http://www.wikidata.org/entity/Q97689488'), ('oName', 'La Pernoderie')]
[('p', 'http://www.wikidata.org/prop/direct/P1419'), ('pName', 'shape'), ('o', 'http://www.wikidata.org/entity/Q2226897'), ('oName', 'Pyramid Arena')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('o', 'http://www.wikidata.org/entity/Q1996868'), ('oName', 'Pyramids')]
[('p', 'http://www.wikidata.org/prop/direct

In [12]:
# Identify the BGP for Ancient Egypt

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371' ('Sneferu');
#                                          wd :Q220240' ('Huni');

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')

# wd:Q175582 ('Egyptian pyramids') wdt:P279 ('subclass of') wd:Q12516 
# wd:Q20504694 ('Modern Egyptian') wdt:P180 'depicts' wd:Q12516

# wd:Q175582 wdt:P17 ( 'country') wd:Q79 ('Egypt')

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q189371 ?p ?o.
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {?o sc:name ?oName}.
   FILTER (?p not in (wdt:P180))   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('pName', 'child'), ('o', 'http://www.wikidata.org/entity/Q122426'), ('oName', 'Henutsen')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('pName', 'child'), ('o', 'http://www.wikidata.org/entity/Q161904'), ('oName', 'Khufu')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('o', 'http://www.wikidata.org/entity/Q177819'), ('oName', 'Old Kingdom of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P53'), ('pName', 'family'), ('o', 'http://www.wikidata.org/entity/Q211001'), ('oName', 'Fourth Dynasty of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('pName', 'spouse'), ('o', 'http://www.wikidata.org/entity/Q256515'), ('oName', 'Hetepheres I')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('pName', 'sibling'), ('o', 'http://www.wikidata.org/entity/Q256515'), ('oName', 'Hetepheres I')]
[('p', 'http://www.wikidata.org/prop/direct/P40'), ('pName', 'child'), ('o', 'http://www

In [14]:
# Identify the BGP for Ancient Egypt

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')

# wd:Q175582 ('Egyptian pyramids') wdt:P279 ('subclass of') wd:Q12516 
# wd:Q20504694 ('Modern Egyptian') wdt:P180 'depicts' wd:Q12516

# wd:Q175582 wdt:P17 ( 'country') wd:Q79 ('Egypt')

queryString = """
SELECT DISTINCT ?timePeriod ?timePeriodName
WHERE {
   # bind something
   wd:Q189371 wdt:P2348 ?timePeriod.
   # get the label
   ?timePeriod sc:name ?timePeriodName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('timePeriod', 'http://www.wikidata.org/entity/Q177819'), ('timePeriodName', 'Old Kingdom of Egypt')]
[('timePeriod', 'http://www.wikidata.org/entity/Q11768'), ('timePeriodName', 'Ancient Egypt')]
2


In [16]:
# Identify the BGP for Ancient Egypt

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')

# wd:Q175582 ('Egyptian pyramids') wdt:P279 ('subclass of') wd:Q12516 
# wd:Q20504694 ('Modern Egyptian') wdt:P180 'depicts' wd:Q12516

# wd:Q175582 wdt:P17 ( 'country') wd:Q79 ('Egypt')

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q189371 wdt:P27 ?o.
   # get the label
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q11768'), ('oName', 'Ancient Egypt')]
1


### Final query for this task

In [2]:
# Identify the BGP for Ancient Egypt

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')

# wd:Q175582 ('Egyptian pyramids') wdt:P279 ('subclass of') wd:Q12516 
# wd:Q20504694 ('Modern Egyptian') wdt:P180 'depicts' wd:Q12516

# wd:Q175582 wdt:P17 ( 'country') wd:Q79 ('Egypt')

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q189371 wdt:P27 ?o. # Sneferu (wd:Q189371) --country of citizenship (wdt:P27)--> 'Ancient Egypt' (wd:Q11768)
   # get the label
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q11768'), ('oName', 'Ancient Egypt')]
1


## Task 2
Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

In [18]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child 
#    (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')

# wd:Q175582 ('Egyptian pyramids') wdt:P279 ('subclass of') wd:Q12516 
# wd:Q20504694 ('Modern Egyptian') wdt:P180 'depicts' wd:Q12516

# wd:Q175582 wdt:P17 ( 'country') wd:Q79 ('Egypt')

# wd:Q11768 ('Ancient Egypt')

# EXPLORING wd:Q11768 ('Ancient Egypt')
queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q11768 ?p ?o.
   # get the label
   OPTIONAL {?p sc:name ?pName}.
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal"), ('o', 'http://www.wikidata.org/entity/Q8440998'), ('oName', 'Portal:Ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q1138524'), ('oName', 'Paulys Realenzyklopädie der klassischen Altertumswissenschaft')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q4173137'), ('oName', 'Jewish Encyclopedia of Brockhaus and Efron')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q17378135'), ('oName', 'Great Soviet Encyclopedia (1969–1978)')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q714601'), ('oName', 'Predynastic Period of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P155

In [19]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child 
#    (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')

# wd:Q175582 ('Egyptian pyramids') wdt:P279 ('subclass of') wd:Q12516 
# wd:Q20504694 ('Modern Egyptian') wdt:P180 'depicts' wd:Q12516

# wd:Q175582 wdt:P17 ( 'country') wd:Q79 ('Egypt')

# EXPLORING wd:Q189371 ('Sneferu') to get a list of children, pharaoh's BGP and DATES

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q189371 ?p ?o.
   # get the label
   OPTIONAL {?p sc:name ?pName}.
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q372436'), ('oName', 'statesperson')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('pName', 'place of burial'), ('o', 'http://www.wikidata.org/entity/Q135332'), ('oName', 'Red Pyramid')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q602358'), ('oName', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q46002746'), ('oName', 'Dictionary of African Biography')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pName', 'place of birth'), ('o', 'http://www.wikidata.org/entity/Q79'), ('oName', 'Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P20'), ('pName', 'place of death'), ('o', 'http://www.wikidata.org/entity/Q79'), ('oName', 'Egypt')]
[('p', 'http:/

In [20]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child 
#    (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')

# wd:Q175582 ('Egyptian pyramids') wdt:P279 ('subclass of') wd:Q12516 
# wd:Q20504694 ('Modern Egyptian') wdt:P180 'depicts' wd:Q12516

# wd:Q175582 wdt:P17 ( 'country') wd:Q79 ('Egypt')

# EXPLORING Q220240 ('Huni') to get a list of children, pharaoh's BGP and DATES

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q220240 ?p ?o.
   # get the label
   OPTIONAL {?p sc:name ?pName}.
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q372436'), ('oName', 'statesperson')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('pName', 'place of burial'), ('o', 'http://www.wikidata.org/entity/Q1066424'), ('oName', 'Meidum')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('pName', 'replaces'), ('o', 'http://www.wikidata.org/entity/Q220320'), ('oName', 'Khaba')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pName', 'replaced by'), ('o', 'http://www.wikidata.org/entity/Q189371'), ('oName', 'Sneferu')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pName', 'sex or gender'), ('o', 'http://www.wikidata.org/entity/Q6581097'), ('oName', 'male')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('o', 'http://www.wikidata.org/entity/Q11768'), ('oName', 'Ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P26'), ('pName', 'spouse'), ('o', 'http://ww

In [21]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child 
#    (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')

# EXPLORING to get a list of children, NOW GOT BGP of wd:Q37110 ('pharaoh')

queryString = """
SELECT DISTINCT ?s ?sName
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q151828'), ('sName', 'Djet')]
[('s', 'http://www.wikidata.org/entity/Q152375'), ('sName', 'Djer')]
[('s', 'http://www.wikidata.org/entity/Q19248'), ('sName', 'Senusret III')]
[('s', 'http://www.wikidata.org/entity/Q230774'), ('sName', 'Nitocris')]
[('s', 'http://www.wikidata.org/entity/Q263218'), ('sName', 'Djedkhonsuefankh')]
[('s', 'http://www.wikidata.org/entity/Q269265'), ('sName', 'Pepi I Meryre')]
[('s', 'http://www.wikidata.org/entity/Q320908'), ('sName', 'Merenre Nemtyemsaf II')]
[('s', 'http://www.wikidata.org/entity/Q336395'), ('sName', 'Smendes')]
[('s', 'http://www.wikidata.org/entity/Q380676'), ('sName', 'Psusennes I')]
[('s', 'http://www.wikidata.org/entity/Q428633'), ('sName', 'Piankh')]
[('s', 'http://www.wikidata.org/entity/Q434877'), ('sName', 'Amenemnisu')]
[('s', 'http://www.wikidata.org/entity/Q440496'), ('sName', 'Khyan')]
[('s', 'http://www.wikidata.org/entity/Q6198'), ('sName', 'Pepi II')]
[('s', 'http://www.wikidat

In [22]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

# TRYING to filter the dates, NOW GOT BGP of wd:Q37110 ('pharaoh')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   FILTER (xsd:date(?dob) > '-1500-01-01')
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [23]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

# TRYING to filter the dates
queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   FILTER (xsd:date(?dob) < '-1500-01-01')
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


#### TRYING cope with dates

In [24]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   #FILTER (xsd:date(?dob) < '-1500-01-01')
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q151828'), ('sName', 'Djet'), ('dob', '-3000-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q152375'), ('sName', 'Djer'), ('dob', '-3001-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q19248'), ('sName', 'Senusret III'), ('dob', '-1950-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q230774'), ('sName', 'Nitocris'), ('dob', '-2250-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q263218'), ('sName', 'Djedkhonsuefankh'), ('dob', '-1100-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q269265'), ('sName', 'Pepi I Meryre'), ('dob', '-2400-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q320908'), ('sName', 'Merenre Nemtyemsaf II'), ('dob', '-2300-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q336395'), ('sName', 'Smendes'), ('dob', '-1200-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q380676'), ('sName', 'Psusennes I'), ('dob', '-1100-01-01T00:00:00Z')]
[('s',

In [25]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   FILTER (xsd:datetime(?dob) > '-1500-01-01T00:00:00Z')
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [26]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob ?year
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   BIND (xsd:gYear(?dob) as ?year)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q151828'), ('sName', 'Djet'), ('dob', '-3000-01-01T00:00:00Z'), ('year', '-3000-01-01Z')]
[('s', 'http://www.wikidata.org/entity/Q152375'), ('sName', 'Djer'), ('dob', '-3001-01-01T00:00:00Z'), ('year', '-3001-01-01Z')]
[('s', 'http://www.wikidata.org/entity/Q19248'), ('sName', 'Senusret III'), ('dob', '-1950-01-01T00:00:00Z'), ('year', '-1950-01-01Z')]
[('s', 'http://www.wikidata.org/entity/Q230774'), ('sName', 'Nitocris'), ('dob', '-2250-01-01T00:00:00Z'), ('year', '-2250-01-01Z')]
[('s', 'http://www.wikidata.org/entity/Q263218'), ('sName', 'Djedkhonsuefankh'), ('dob', '-1100-01-01T00:00:00Z'), ('year', '-1100-01-01Z')]
[('s', 'http://www.wikidata.org/entity/Q269265'), ('sName', 'Pepi I Meryre'), ('dob', '-2400-01-01T00:00:00Z'), ('year', '-2400-01-01Z')]
[('s', 'http://www.wikidata.org/entity/Q320908'), ('sName', 'Merenre Nemtyemsaf II'), ('dob', '-2300-01-01T00:00:00Z'), ('year', '-2300-01-01Z')]
[('s', 'http://www.wikidata.org/entity/Q

In [27]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob ?year
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   BIND (xsd:gYear(?dob) as ?year)
   FILTER (xsd:datetime(?dob) > '-1500-01-01T00:00:00Z'^^xsd:datetime)
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q151828'), ('sName', 'Djet'), ('dob', '-3000-01-01T00:00:00Z'), ('year', '-3000-01-01Z')]
[('s', 'http://www.wikidata.org/entity/Q152375'), ('sName', 'Djer'), ('dob', '-3001-01-01T00:00:00Z'), ('year', '-3001-01-01Z')]
[('s', 'http://www.wikidata.org/entity/Q19248'), ('sName', 'Senusret III'), ('dob', '-1950-01-01T00:00:00Z'), ('year', '-1950-01-01Z')]
[('s', 'http://www.wikidata.org/entity/Q230774'), ('sName', 'Nitocris'), ('dob', '-2250-01-01T00:00:00Z'), ('year', '-2250-01-01Z')]
[('s', 'http://www.wikidata.org/entity/Q263218'), ('sName', 'Djedkhonsuefankh'), ('dob', '-1100-01-01T00:00:00Z'), ('year', '-1100-01-01Z')]
[('s', 'http://www.wikidata.org/entity/Q269265'), ('sName', 'Pepi I Meryre'), ('dob', '-2400-01-01T00:00:00Z'), ('year', '-2400-01-01Z')]
[('s', 'http://www.wikidata.org/entity/Q320908'), ('sName', 'Merenre Nemtyemsaf II'), ('dob', '-2300-01-01T00:00:00Z'), ('year', '-2300-01-01Z')]
[('s', 'http://www.wikidata.org/entity/Q

In [29]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   FILTER (xsd:gYear(?dob) >= '-1500-01-01T00:00:00Z'^^xsd:gYear && xsd:gYear(?dob) <= '-1000-12-31T00:00:00Z'^^xsd:gYear)
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [30]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   FILTER (xsd:gYear(?dob) >= '-1500-01-01T00:00:00Z'^^xsd:gYear)
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [31]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   FILTER (xsd:datetime(?dob) >= '-1500-01-01T00:00:00Z'^^xsd:datetime)
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q151828'), ('sName', 'Djet'), ('dob', '-3000-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q152375'), ('sName', 'Djer'), ('dob', '-3001-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q19248'), ('sName', 'Senusret III'), ('dob', '-1950-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q230774'), ('sName', 'Nitocris'), ('dob', '-2250-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q263218'), ('sName', 'Djedkhonsuefankh'), ('dob', '-1100-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q269265'), ('sName', 'Pepi I Meryre'), ('dob', '-2400-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q320908'), ('sName', 'Merenre Nemtyemsaf II'), ('dob', '-2300-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q336395'), ('sName', 'Smendes'), ('dob', '-1200-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q380676'), ('sName', 'Psusennes I'), ('dob', '-1100-01-01T00:00:00Z')]
[('s',

In [32]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   FILTER (xsd:datetime(?dob) >= '-1500-01-01T00:00:00Z'^^xsd:datetime && xsd:datetime(?dob) <= '-1000-12-31T00:00:00Z'^^xsd:datetime)
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [33]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   FILTER (xsd:datetime(?dob) <= '-1500-01-01T00:00:00Z'^^xsd:datetime)
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [42]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   FILTER (xsd:datetime(?dob) > '1500-01-01T00:00:00Z'^^xsd:datetime)
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q151828'), ('sName', 'Djet'), ('dob', '-3000-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q152375'), ('sName', 'Djer'), ('dob', '-3001-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q19248'), ('sName', 'Senusret III'), ('dob', '-1950-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q230774'), ('sName', 'Nitocris'), ('dob', '-2250-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q263218'), ('sName', 'Djedkhonsuefankh'), ('dob', '-1100-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q269265'), ('sName', 'Pepi I Meryre'), ('dob', '-2400-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q320908'), ('sName', 'Merenre Nemtyemsaf II'), ('dob', '-2300-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q336395'), ('sName', 'Smendes'), ('dob', '-1200-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q380676'), ('sName', 'Psusennes I'), ('dob', '-1100-01-01T00:00:00Z')]
[('s',

In [49]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob ?min
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   BIND('-1500-01-01T00:00:00Z'^^xsd:dateTime AS ?min)
   #FILTER ( xsd:dateTime(?dob) > ?min)
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q151828'), ('sName', 'Djet'), ('dob', '-3000-01-01T00:00:00Z'), ('min', '-1500-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q152375'), ('sName', 'Djer'), ('dob', '-3001-01-01T00:00:00Z'), ('min', '-1500-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q19248'), ('sName', 'Senusret III'), ('dob', '-1950-01-01T00:00:00Z'), ('min', '-1500-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q230774'), ('sName', 'Nitocris'), ('dob', '-2250-01-01T00:00:00Z'), ('min', '-1500-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q263218'), ('sName', 'Djedkhonsuefankh'), ('dob', '-1100-01-01T00:00:00Z'), ('min', '-1500-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q269265'), ('sName', 'Pepi I Meryre'), ('dob', '-2400-01-01T00:00:00Z'), ('min', '-1500-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q320908'), ('sName', 'Merenre Nemtyemsaf II'), ('dob', '-2300-01-01T00:00:00Z'), ('min', '-1500-01-

In [57]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob ?min ?year
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   BIND('-1500-01-01T00:00:00Z'^^xsd:dateTime AS ?min).
   BIND(substr(str(?dob),1) as ?year).
   #FILTER ( xsd:dateTime(?dob) > ?min)
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q151828'), ('sName', 'Djet'), ('dob', '-3000-01-01T00:00:00Z'), ('min', '-1500-01-01T00:00:00Z'), ('year', '-3000-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q152375'), ('sName', 'Djer'), ('dob', '-3001-01-01T00:00:00Z'), ('min', '-1500-01-01T00:00:00Z'), ('year', '-3001-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q19248'), ('sName', 'Senusret III'), ('dob', '-1950-01-01T00:00:00Z'), ('min', '-1500-01-01T00:00:00Z'), ('year', '-1950-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q230774'), ('sName', 'Nitocris'), ('dob', '-2250-01-01T00:00:00Z'), ('min', '-1500-01-01T00:00:00Z'), ('year', '-2250-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q263218'), ('sName', 'Djedkhonsuefankh'), ('dob', '-1100-01-01T00:00:00Z'), ('min', '-1500-01-01T00:00:00Z'), ('year', '-1100-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q269265'), ('sName', 'Pepi I Meryre'), ('dob', '-2400-01-01T00:00:00Z'

In [58]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob year(?min) ?year
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   BIND('-1500-01-01T00:00:00Z'^^xsd:dateTime AS ?min).
   BIND(substr(str(?dob),1) as ?year).
   #FILTER ( xsd:dateTime(?dob) > ?min)
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q151828'), ('sName', 'Djet'), ('dob', '-3000-01-01T00:00:00Z'), ('callret-3', '-1500'), ('year', '-3000-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q152375'), ('sName', 'Djer'), ('dob', '-3001-01-01T00:00:00Z'), ('callret-3', '-1500'), ('year', '-3001-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q19248'), ('sName', 'Senusret III'), ('dob', '-1950-01-01T00:00:00Z'), ('callret-3', '-1500'), ('year', '-1950-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q230774'), ('sName', 'Nitocris'), ('dob', '-2250-01-01T00:00:00Z'), ('callret-3', '-1500'), ('year', '-2250-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q263218'), ('sName', 'Djedkhonsuefankh'), ('dob', '-1100-01-01T00:00:00Z'), ('callret-3', '-1500'), ('year', '-1100-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q269265'), ('sName', 'Pepi I Meryre'), ('dob', '-2400-01-01T00:00:00Z'), ('callret-3', '-1500'), ('year', '-2400-01-01T0

In [81]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob  ?year  #(year(?min) as ?minYear) (year(?max) as ?maxYear)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   
   #BIND('-1500-01-01T00:00:00Z'^^xsd:dateTime AS ?min).
   #BIND('-1000-01-01T00:00:00Z'^^xsd:dateTime AS ?max).
   #BIND(xsd:dateTime(?dob) as ?year). #Virtuoso 22007 Error DT006: Cannot convert nodeID://b374576 to datetime : Incorrect year delimiter\
   #BIND(strdt(concat(str(year(?start)),"-",str(month(?start)),"-",str(day(?start))), xsd:date)as ?s)
   
   BIND((substr(str(?dob), 2,4)) as ?year)
   
   #FILTER ( xsd:dateTime(?dob) > ?min)
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q151828'), ('sName', 'Djet'), ('dob', '-3000-01-01T00:00:00Z'), ('year', '3000')]
[('s', 'http://www.wikidata.org/entity/Q152375'), ('sName', 'Djer'), ('dob', '-3001-01-01T00:00:00Z'), ('year', '3001')]
[('s', 'http://www.wikidata.org/entity/Q19248'), ('sName', 'Senusret III'), ('dob', '-1950-01-01T00:00:00Z'), ('year', '1950')]
[('s', 'http://www.wikidata.org/entity/Q230774'), ('sName', 'Nitocris'), ('dob', '-2250-01-01T00:00:00Z'), ('year', '2250')]
[('s', 'http://www.wikidata.org/entity/Q263218'), ('sName', 'Djedkhonsuefankh'), ('dob', '-1100-01-01T00:00:00Z'), ('year', '1100')]
5


In [82]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT (count(*) as ?tot) #?s ?sName ?dob  ?year  #(year(?min) as ?minYear) (year(?max) as ?maxYear)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   
   #BIND('-1500-01-01T00:00:00Z'^^xsd:dateTime AS ?min).
   #BIND('-1000-01-01T00:00:00Z'^^xsd:dateTime AS ?max).
   #BIND(xsd:dateTime(?dob) as ?year). #Virtuoso 22007 Error DT006: Cannot convert nodeID://b374576 to datetime : Incorrect year delimiter\
   #BIND(strdt(concat(str(year(?start)),"-",str(month(?start)),"-",str(day(?start))), xsd:date)as ?s)
   
   BIND((substr(str(?dob), 2,4)) as ?year)
   
   #FILTER ( xsd:dateTime(?dob) > ?min)
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('tot', '396')]
1


In [83]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT (count(*) as ?tot) #?s ?sName ?dob  ?year  #(year(?min) as ?minYear) (year(?max) as ?maxYear)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110.
   #;       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   
   #BIND('-1500-01-01T00:00:00Z'^^xsd:dateTime AS ?min).
   #BIND('-1000-01-01T00:00:00Z'^^xsd:dateTime AS ?max).
   #BIND(xsd:dateTime(?dob) as ?year). #Virtuoso 22007 Error DT006: Cannot convert nodeID://b374576 to datetime : Incorrect year delimiter\
   #BIND(strdt(concat(str(year(?start)),"-",str(month(?start)),"-",str(day(?start))), xsd:date)as ?s)
   
   #BIND((substr(str(?dob), 2,4)) as ?year)
   
   #FILTER ( xsd:dateTime(?dob) > ?min)
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('tot', '460')]
1


#### prob 1 missing many years --> KB problem
#### prob 2 conversion failure to xsd:dateTime | xsd:datetime 

In [88]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob (?year)  #(year(?min) as ?minYear) (year(?max) as ?maxYear)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   #BIND((substr(str(?dob), 2,4)) as ?year)
   BIND(REPLACE(STR(?dob), 'T.*', '') AS ?year)
   
   #FILTER ( xsd:gYear(?year) > "1000" )
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q151828'), ('sName', 'Djet'), ('dob', '-3000-01-01T00:00:00Z'), ('year', '-3000-01-01')]
[('s', 'http://www.wikidata.org/entity/Q152375'), ('sName', 'Djer'), ('dob', '-3001-01-01T00:00:00Z'), ('year', '-3001-01-01')]
[('s', 'http://www.wikidata.org/entity/Q19248'), ('sName', 'Senusret III'), ('dob', '-1950-01-01T00:00:00Z'), ('year', '-1950-01-01')]
[('s', 'http://www.wikidata.org/entity/Q230774'), ('sName', 'Nitocris'), ('dob', '-2250-01-01T00:00:00Z'), ('year', '-2250-01-01')]
[('s', 'http://www.wikidata.org/entity/Q263218'), ('sName', 'Djedkhonsuefankh'), ('dob', '-1100-01-01T00:00:00Z'), ('year', '-1100-01-01')]
5


In [90]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob (?year)  #(year(?min) as ?minYear) (year(?max) as ?maxYear)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   BIND((substr(str(?dob), 1,5)) as ?year)
   
   FILTER ( xsd:int(?year) < 1000 )
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q151828'), ('sName', 'Djet'), ('dob', '-3000-01-01T00:00:00Z'), ('year', '-3000')]
[('s', 'http://www.wikidata.org/entity/Q152375'), ('sName', 'Djer'), ('dob', '-3001-01-01T00:00:00Z'), ('year', '-3001')]
[('s', 'http://www.wikidata.org/entity/Q19248'), ('sName', 'Senusret III'), ('dob', '-1950-01-01T00:00:00Z'), ('year', '-1950')]
[('s', 'http://www.wikidata.org/entity/Q230774'), ('sName', 'Nitocris'), ('dob', '-2250-01-01T00:00:00Z'), ('year', '-2250')]
[('s', 'http://www.wikidata.org/entity/Q263218'), ('sName', 'Djedkhonsuefankh'), ('dob', '-1100-01-01T00:00:00Z'), ('year', '-1100')]
5


In [91]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob (?year)  #(year(?min) as ?minYear) (year(?max) as ?maxYear)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   BIND(strdt(substr(str(?dob), 1,5),xsd:int) as ?year)
   
   FILTER ( ?year < -1000) #1500 BCE and 1000 BCE
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q151828'), ('sName', 'Djet'), ('dob', '-3000-01-01T00:00:00Z'), ('year', '-3000')]
[('s', 'http://www.wikidata.org/entity/Q152375'), ('sName', 'Djer'), ('dob', '-3001-01-01T00:00:00Z'), ('year', '-3001')]
[('s', 'http://www.wikidata.org/entity/Q19248'), ('sName', 'Senusret III'), ('dob', '-1950-01-01T00:00:00Z'), ('year', '-1950')]
[('s', 'http://www.wikidata.org/entity/Q230774'), ('sName', 'Nitocris'), ('dob', '-2250-01-01T00:00:00Z'), ('year', '-2250')]
[('s', 'http://www.wikidata.org/entity/Q263218'), ('sName', 'Djedkhonsuefankh'), ('dob', '-1100-01-01T00:00:00Z'), ('year', '-1100')]
5


#### Found a Solution to the prob 2

In [92]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob (?year)  #(year(?min) as ?minYear) (year(?max) as ?maxYear)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   BIND(strdt(substr(str(?dob), 1,5),xsd:int) as ?year)
   
   FILTER ( ?year <= -1000 && ?year >=-1500) #1500 BCE and 1000 BCE
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q263218'), ('sName', 'Djedkhonsuefankh'), ('dob', '-1100-01-01T00:00:00Z'), ('year', '-1100')]
[('s', 'http://www.wikidata.org/entity/Q336395'), ('sName', 'Smendes'), ('dob', '-1200-01-01T00:00:00Z'), ('year', '-1200')]
[('s', 'http://www.wikidata.org/entity/Q380676'), ('sName', 'Psusennes I'), ('dob', '-1100-01-01T00:00:00Z'), ('year', '-1100')]
[('s', 'http://www.wikidata.org/entity/Q428633'), ('sName', 'Piankh'), ('dob', '-1200-01-01T00:00:00Z'), ('year', '-1200')]
[('s', 'http://www.wikidata.org/entity/Q434877'), ('sName', 'Amenemnisu'), ('dob', '-1100-01-01T00:00:00Z'), ('year', '-1100')]
5


In [93]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob (?year)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   BIND(strdt(substr(str(?dob), 1,5),xsd:int) as ?year)
   
   FILTER ( ?year <= -1000 && ?year >=-1500) #1500 BCE and 1000 BCE
}
ORDER BY DESC(?year)
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q459153'), ('sName', 'Osorkon II'), ('dob', '-1000-01-01T00:00:00Z'), ('year', '-1000')]
[('s', 'http://www.wikidata.org/entity/Q1056348'), ('sName', 'Nimlot'), ('dob', '-1000-01-01T00:00:00Z'), ('year', '-1000')]
[('s', 'http://www.wikidata.org/entity/Q311811'), ('sName', 'Shoshenq I'), ('dob', '-1000-01-01T00:00:00Z'), ('year', '-1000')]
[('s', 'http://www.wikidata.org/entity/Q515574'), ('sName', 'Osorkon I'), ('dob', '-1000-01-01T00:00:00Z'), ('year', '-1000')]
[('s', 'http://www.wikidata.org/entity/Q708340'), ('sName', 'Sheshonk II'), ('dob', '-1000-01-01T00:00:00Z'), ('year', '-1000')]
5


In [94]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') ('o', '-2650-01-01T00:00:00Z')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?dob (?year)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   BIND(strdt(substr(str(?dob), 1,5),xsd:int) as ?year)
   
   FILTER ( ?year <= -1000 && ?year >=-1500) #1500 BCE and 1000 BCE
}
ORDER BY ASC(?year)
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q517662'), ('sName', 'Yuya'), ('dob', '-1500-01-01T00:00:00Z'), ('year', '-1500')]
[('s', 'http://www.wikidata.org/entity/Q157985'), ('sName', 'Thutmose IV'), ('dob', '-1500-01-01T00:00:00Z'), ('year', '-1500')]
[('s', 'http://www.wikidata.org/entity/Q1462824'), ('sName', 'Senneferi'), ('dob', '-1500-01-01T00:00:00Z'), ('year', '-1500')]
[('s', 'http://www.wikidata.org/entity/Q650643'), ('sName', 'Rekhmire'), ('dob', '-1500-01-01T00:00:00Z'), ('year', '-1500')]
[('s', 'http://www.wikidata.org/entity/Q653168'), ('sName', 'Snaaib'), ('dob', '-1500-01-01T00:00:00Z'), ('year', '-1500')]
5


#### Getting children

In [97]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') '-2650-01-01T00:00:00Z'
#            wdt:P40 ('child') wd:Q146029 ('Ay')

# NOTE: forgot to remove year from ordering

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob;
       ?p ?o.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   OPTIONAL {?p sc:name ?pName}.
   OPTIONAL {?o sc:name ?oName}.
   BIND(strdt(substr(str(?dob), 1,5),xsd:int) as ?year)
   
   FILTER ( ?year <= -1000 && ?year >=-1500) #1500 BCE and 1000 BCE
}
ORDER BY ASC(?year)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q517662'), ('sName', 'Yuya'), ('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q8125919'), ('oName', 'political adviser')]
[('s', 'http://www.wikidata.org/entity/Q517662'), ('sName', 'Yuya'), ('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q732358'), ('oName', 'nomarch')]
[('s', 'http://www.wikidata.org/entity/Q517662'), ('sName', 'Yuya'), ('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q42603'), ('oName', 'priest')]
[('s', 'http://www.wikidata.org/entity/Q517662'), ('sName', 'Yuya'), ('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q192322'), ('oName', 'Min')]
[('s', 'http://www.wikidata.org/entity/Q517662'), ('sName', 'Yuya'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('

In [98]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') '-2650-01-01T00:00:00Z'
#            wdt:P40 ('child') wd:Q146029 ('Ay')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?p ?pName ?o ?oName ?child ?childName
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob;
       wdt:P40 ?child.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   OPTIONAL {?child sc:name ?childName}.
   
   BIND(strdt(substr(str(?dob), 1,5),xsd:int) as ?year)
   FILTER ( ?year <= -1000 && ?year >=-1500) #1500 BCE and 1000 BCE
}
ORDER BY ASC(?year)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q517662'), ('sName', 'Yuya'), ('child', 'http://www.wikidata.org/entity/Q146029'), ('childName', 'Ay')]
[('s', 'http://www.wikidata.org/entity/Q517662'), ('sName', 'Yuya'), ('child', 'http://www.wikidata.org/entity/Q152230'), ('childName', 'Tiye')]
[('s', 'http://www.wikidata.org/entity/Q517662'), ('sName', 'Yuya'), ('child', 'http://www.wikidata.org/entity/Q301004'), ('childName', 'Anen')]
[('s', 'http://www.wikidata.org/entity/Q157985'), ('sName', 'Thutmose IV'), ('child', 'http://www.wikidata.org/entity/Q42606'), ('childName', 'Amenhotep III')]
[('s', 'http://www.wikidata.org/entity/Q157985'), ('sName', 'Thutmose IV'), ('child', 'http://www.wikidata.org/entity/Q780634'), ('childName', 'Tiaa')]
[('s', 'http://www.wikidata.org/entity/Q157985'), ('sName', 'Thutmose IV'), ('child', 'http://www.wikidata.org/entity/Q689614'), ('childName', 'Amenemopet')]
[('s', 'http://www.wikidata.org/entity/Q157985'), ('sName', 'Thutmose IV'), ('child', 'ht

In [99]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') '-2650-01-01T00:00:00Z'
#            wdt:P40 ('child') wd:Q146029 ('Ay')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?child ?childName
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob;
       wdt:P40 ?child.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   OPTIONAL {?child sc:name ?childName}.
   
   BIND(strdt(substr(str(?dob), 1,5),xsd:int) as ?year)
   FILTER ( ?year <= -1000 && ?year >=-1500) #1500 BCE and 1000 BCE
}
ORDER BY ASC(?year)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q517662'), ('sName', 'Yuya'), ('child', 'http://www.wikidata.org/entity/Q146029'), ('childName', 'Ay')]
[('s', 'http://www.wikidata.org/entity/Q517662'), ('sName', 'Yuya'), ('child', 'http://www.wikidata.org/entity/Q152230'), ('childName', 'Tiye')]
[('s', 'http://www.wikidata.org/entity/Q517662'), ('sName', 'Yuya'), ('child', 'http://www.wikidata.org/entity/Q301004'), ('childName', 'Anen')]
[('s', 'http://www.wikidata.org/entity/Q157985'), ('sName', 'Thutmose IV'), ('child', 'http://www.wikidata.org/entity/Q42606'), ('childName', 'Amenhotep III')]
[('s', 'http://www.wikidata.org/entity/Q157985'), ('sName', 'Thutmose IV'), ('child', 'http://www.wikidata.org/entity/Q780634'), ('childName', 'Tiaa')]
[('s', 'http://www.wikidata.org/entity/Q157985'), ('sName', 'Thutmose IV'), ('child', 'http://www.wikidata.org/entity/Q689614'), ('childName', 'Amenemopet')]
[('s', 'http://www.wikidata.org/entity/Q157985'), ('sName', 'Thutmose IV'), ('child', 'ht

In [102]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') '-2650-01-01T00:00:00Z'
#            wdt:P40 ('child') wd:Q146029 ('Ay')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?year (COUNT(*) as ?children)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob;
       wdt:P40 ?child.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   OPTIONAL {?child sc:name ?childName}.
   
   BIND(strdt(substr(str(?dob), 1,5),xsd:int) as ?year)
   FILTER ( ?year <= -1000 && ?year >=-1500) #1500 BCE and 1000 BCE
}
GROUP BY ?s ?sName ?year
ORDER BY ASC(?year)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q517662'), ('sName', 'Yuya'), ('year', '-1500'), ('children', '3')]
[('s', 'http://www.wikidata.org/entity/Q157985'), ('sName', 'Thutmose IV'), ('year', '-1500'), ('children', '6')]
[('s', 'http://www.wikidata.org/entity/Q157899'), ('sName', 'Thutmose III'), ('year', '-1480'), ('children', '8')]
[('s', 'http://www.wikidata.org/entity/Q42606'), ('sName', 'Amenhotep III'), ('year', '-1403'), ('children', '10')]
[('s', 'http://www.wikidata.org/entity/Q158052'), ('sName', 'Amenhotep II'), ('year', '-1401'), ('children', '8')]
[('s', 'http://www.wikidata.org/entity/Q81794'), ('sName', 'Akhenaten'), ('year', '-1400'), ('children', '9')]
[('s', 'http://www.wikidata.org/entity/Q158003'), ('sName', 'Smenkhkare'), ('year', '-1400'), ('children', '1')]
[('s', 'http://www.wikidata.org/entity/Q146029'), ('sName', 'Ay'), ('year', '-1400'), ('children', '2')]
[('s', 'http://www.wikidata.org/entity/Q157918'), ('sName', 'Seti I'), ('year', '-1322'), ('chil

In [103]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') '-2650-01-01T00:00:00Z'
#            wdt:P40 ('child') wd:Q146029 ('Ay')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?year (COUNT(*) as ?children)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob;
       wdt:P40 ?child.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   OPTIONAL {?child sc:name ?childName}.
   
   BIND(strdt(substr(str(?dob), 1,5),xsd:int) as ?year)
   FILTER ( ?year <= -1000 && ?year >=-1500) #1500 BCE and 1000 BCE
}
GROUP BY ?s ?sName ?year
HAVING (COUNT(*) >= 8)
ORDER BY ASC(?year)
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q157899'), ('sName', 'Thutmose III'), ('year', '-1480'), ('children', '8')]
[('s', 'http://www.wikidata.org/entity/Q42606'), ('sName', 'Amenhotep III'), ('year', '-1403'), ('children', '10')]
[('s', 'http://www.wikidata.org/entity/Q158052'), ('sName', 'Amenhotep II'), ('year', '-1401'), ('children', '8')]
[('s', 'http://www.wikidata.org/entity/Q81794'), ('sName', 'Akhenaten'), ('year', '-1400'), ('children', '9')]
[('s', 'http://www.wikidata.org/entity/Q1523'), ('sName', 'Ramesses II'), ('year', '-1302'), ('children', '18')]
[('s', 'http://www.wikidata.org/entity/Q1528'), ('sName', 'Ramesses III'), ('year', '-1216'), ('children', '10')]
[('s', 'http://www.wikidata.org/entity/Q444965'), ('sName', 'Pinedjem I'), ('year', '-1200'), ('children', '8')]
7


#### Got a complete sol, with BIND & substr to solve prob 2

In [104]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') '-2650-01-01T00:00:00Z'
#            wdt:P40 ('child') wd:Q146029 ('Ay')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName (COUNT(*) as ?children)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob;
       wdt:P40 ?child.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   OPTIONAL {?child sc:name ?childName}.
   
   BIND(strdt(substr(str(?dob), 1,5),xsd:int) as ?year)
   FILTER ( ?year <= -1000 && ?year >=-1500) #1500 BCE and 1000 BCE
}
GROUP BY ?s ?sName
HAVING (COUNT(*) > 8) #more than, not at least
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1528'), ('sName', 'Ramesses III'), ('children', '10')]
[('s', 'http://www.wikidata.org/entity/Q1523'), ('sName', 'Ramesses II'), ('children', '18')]
[('s', 'http://www.wikidata.org/entity/Q42606'), ('sName', 'Amenhotep III'), ('children', '10')]
[('s', 'http://www.wikidata.org/entity/Q81794'), ('sName', 'Akhenaten'), ('children', '9')]
4


#### Another way to solve, found another solution for prob 2

In [31]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') '-2650-01-01T00:00:00Z'
#            wdt:P40 ('child') wd:Q146029 ('Ay')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName (COUNT(*) as ?children)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?birthDate;
       wdt:P40 ?child.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   OPTIONAL {?child sc:name ?childName}.
   
   BIND(IF(datatype(?birthDate) = xsd:dateTime,year(xsd:dateTime(?birthDate)),-1000) as ?year)
   FILTER ( ?year <= -1000 && ?year >=-1500) #1500 BCE and 1000 BCE
}
GROUP BY ?s ?sName
HAVING (COUNT(*) > 8) #more than, not at least
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1528'), ('sName', 'Ramesses III'), ('children', '10')]
[('s', 'http://www.wikidata.org/entity/Q1523'), ('sName', 'Ramesses II'), ('children', '18')]
[('s', 'http://www.wikidata.org/entity/Q42606'), ('sName', 'Amenhotep III'), ('children', '10')]
[('s', 'http://www.wikidata.org/entity/Q81794'), ('sName', 'Akhenaten'), ('children', '9')]
4


In [33]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') '-2650-01-01T00:00:00Z'
#            wdt:P40 ('child') wd:Q146029 ('Ay')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT * #?s ?sName (COUNT(*) as ?children)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?birthDate;
       wdt:P40 ?child.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   OPTIONAL {?child sc:name ?childName}.
   
   BIND(IF(datatype(?birthDate) = xsd:dateTime,year(xsd:dateTime(?birthDate)),1000) as ?year)
   #FILTER ( ?year <= -1000 && ?year >=-1500) #1500 BCE and 1000 BCE
   FILTER ( ?year >= 1000) #1500 BCE and 1000 BCE
}
#GROUP BY ?s ?sName
#HAVING (COUNT(*) > 8) #more than, not at least
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q18771'), ('birthDate', 'nodeID://b374576'), ('child', 'http://www.wikidata.org/entity/Q18763'), ('sName', 'Amenemhat I'), ('childName', 'Senusret I'), ('year', '1000')]
[('s', 'http://www.wikidata.org/entity/Q18771'), ('birthDate', 'nodeID://b374576'), ('child', 'http://www.wikidata.org/entity/Q461071'), ('sName', 'Amenemhat I'), ('childName', 'Neferu III'), ('year', '1000')]
[('s', 'http://www.wikidata.org/entity/Q111079'), ('birthDate', 'nodeID://b374635'), ('child', 'http://www.wikidata.org/entity/Q151805'), ('sName', 'Anedjib'), ('childName', 'Semerkhet'), ('year', '1000')]
[('s', 'http://www.wikidata.org/entity/Q450994'), ('birthDate', 'nodeID://b374929'), ('child', 'http://www.wikidata.org/entity/Q360012'), ('sName', 'Nepherites I'), ('childName', 'Hakor'), ('year', '1000')]
[('s', 'http://www.wikidata.org/entity/Q242267'), ('birthDate', 'nodeID://b167167'), ('child', 'http://www.wikidata.org/entity/Q264576'), ('sName', 'Bardiya'), 

In [35]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') '-2650-01-01T00:00:00Z'
#            wdt:P40 ('child') wd:Q146029 ('Ay')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName (COUNT(*) as ?children)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?birthDate;
       wdt:P40 ?child.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   OPTIONAL {?child sc:name ?childName}.
   
   BIND(IF(datatype(?birthDate) = xsd:dateTime,year(xsd:dateTime(?birthDate)),1000) as ?year) 
   #bug found:its not a date, sol: if its not a date then use 1000 which will be filtered out
   
   FILTER ( ?year <= -1000 && ?year >=-1500) #1500 BCE and 1000 BCE
}
GROUP BY ?s ?sName
HAVING (COUNT(*) > 8) #more than, not at least
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1528'), ('sName', 'Ramesses III'), ('children', '10')]
[('s', 'http://www.wikidata.org/entity/Q1523'), ('sName', 'Ramesses II'), ('children', '18')]
[('s', 'http://www.wikidata.org/entity/Q42606'), ('sName', 'Amenhotep III'), ('children', '10')]
[('s', 'http://www.wikidata.org/entity/Q81794'), ('sName', 'Akhenaten'), ('children', '9')]
4


### Final query for this task

In [3]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') '-2650-01-01T00:00:00Z'
#            wdt:P40 ('child') wd:Q146029 ('Ay')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName (COUNT(*) as ?children)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?birthDate;
       wdt:P40 ?child.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   OPTIONAL {?child sc:name ?childName}.
   
   BIND(IF(datatype(?birthDate) = xsd:dateTime,year(xsd:dateTime(?birthDate)),1000) as ?year) 
   #bug found:its not a date, sol: if its not a date then use 1000 which will be filtered out
   
   FILTER ( ?year <= -1000 && ?year >=-1500) #1500 BCE and 1000 BCE
}
GROUP BY ?s ?sName
HAVING (COUNT(*) > 8) #more than, not at least
LIMIT 20

"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1528'), ('sName', 'Ramesses III'), ('children', '10')]
[('s', 'http://www.wikidata.org/entity/Q1523'), ('sName', 'Ramesses II'), ('children', '18')]
[('s', 'http://www.wikidata.org/entity/Q42606'), ('sName', 'Amenhotep III'), ('children', '10')]
[('s', 'http://www.wikidata.org/entity/Q81794'), ('sName', 'Akhenaten'), ('children', '9')]
4


#### OLD SOL: binding with substr

In [4]:
# Find the Ancient Egypt pharaohs born between 1500 BCE and 1000 BCE who had more than 8 child (the result set must be a list of triples pharaoh IRI,label and #children).

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P53 ('family') wd:Q220276 ('Third Dynasty of Egypt')
#            wdt:P97 ('noble title') wd:Q37110 ('pharaoh')
#            wdt:P569 ('date of birth') '-2650-01-01T00:00:00Z'
#            wdt:P40 ('child') wd:Q146029 ('Ay')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName (COUNT(*) as ?children)
WHERE {
   # bind something
   ?s wdt:P97 wd:Q37110;
       wdt:P569 ?dob;
       wdt:P40 ?child.
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   OPTIONAL {?child sc:name ?childName}.
   
   BIND(strdt(substr(str(?dob), 1,5),xsd:int) as ?year)
   FILTER ( ?year <= -1000 && ?year >=-1500) #1500 BCE and 1000 BCE
}
GROUP BY ?s ?sName
HAVING (COUNT(*) > 8) #more than, not at least
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1528'), ('sName', 'Ramesses III'), ('children', '10')]
[('s', 'http://www.wikidata.org/entity/Q1523'), ('sName', 'Ramesses II'), ('children', '18')]
[('s', 'http://www.wikidata.org/entity/Q42606'), ('sName', 'Amenhotep III'), ('children', '10')]
[('s', 'http://www.wikidata.org/entity/Q81794'), ('sName', 'Akhenaten'), ('children', '9')]
4


## Task 3
Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

In [105]:
# Identify the BGP for Ancient Egypt

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')

# wd:Q175582 ('Egyptian pyramids') wdt:P279 ('subclass of') wd:Q12516 
# wd:Q20504694 ('Modern Egyptian') wdt:P180 'depicts' wd:Q12516

# wd:Q175582 wdt:P17 ( 'country') wd:Q79 ('Egypt')

# Q11768 ('Ancient Egypt') 
# RESUMING FROM TASK 1 to get the BGP for civilization

queryString = """
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q11768 ?p ?o. 
   # get the label
   OPTIONAL {?p sc:name ?pName}.
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q8440998'), ('oName', 'Portal:Ancient Egypt')]
[('o', 'http://www.wikidata.org/entity/Q1138524'), ('oName', 'Paulys Realenzyklopädie der klassischen Altertumswissenschaft')]
[('o', 'http://www.wikidata.org/entity/Q4173137'), ('oName', 'Jewish Encyclopedia of Brockhaus and Efron')]
[('o', 'http://www.wikidata.org/entity/Q17378135'), ('oName', 'Great Soviet Encyclopedia (1969–1978)')]
[('o', 'http://www.wikidata.org/entity/Q714601'), ('oName', 'Predynastic Period of Egypt')]
[('o', 'http://www.wikidata.org/entity/Q208520'), ('oName', 'Prehistoric Egypt')]
[('o', 'http://www.wikidata.org/entity/Q766876'), ('oName', 'history of ancient Egypt')]
[('o', 'http://www.wikidata.org/entity/Q4752891'), ('oName', 'ancient Near Eastern studies')]
[('o', 'http://www.wikidata.org/entity/Q2487803'), ('oName', 'geography of Ancient Egypt')]
[('o', 'http://www.wikidata.org/entity/Q15'), ('oName', 'Africa')]
[('o', 'http://www.wikidata.org/entity/Q3502482'), 

In [106]:
# Identify the BGP for Ancient Egypt

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

# wd:Q13444093 wdt:P31 ('instance of') wd:Q1456099' ('step pyramid')
# wd:Q1456099  wdt:P279 ('subclass of') wd:Q12516 (pyramid')

# wd:Q175582 ('Egyptian pyramids') wdt:P279 ('subclass of') wd:Q12516 
# wd:Q20504694 ('Modern Egyptian') wdt:P180 'depicts' wd:Q12516

# wd:Q175582 wdt:P17 ( 'country') wd:Q79 ('Egypt')

# Q11768 ('Ancient Egypt') 
# RESUMING FROM TASK 1 to get the BGP for civilization

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q11768 ?p ?o. 
   # get the label
   OPTIONAL {?p sc:name ?pName}.
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal"), ('o', 'http://www.wikidata.org/entity/Q8440998'), ('oName', 'Portal:Ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q1138524'), ('oName', 'Paulys Realenzyklopädie der klassischen Altertumswissenschaft')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q4173137'), ('oName', 'Jewish Encyclopedia of Brockhaus and Efron')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q17378135'), ('oName', 'Great Soviet Encyclopedia (1969–1978)')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q714601'), ('oName', 'Predynastic Period of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P155

In [108]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')

# retrieving all ancient civilizations and durations

queryString = """
SELECT DISTINCT ?s ?sName
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280. 
   # get the label
   OPTIONAL {?s sc:name ?sName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q35355'), ('sName', 'Sumer')]
[('s', 'http://www.wikidata.org/entity/Q25558247'), ('sName', 'Chinese civilization')]
[('s', 'http://www.wikidata.org/entity/Q38060'), ('sName', 'Gaul')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization')]
[('s', 'http://www.wikidata.org/entity/Q173082'), ('sName', 'Dacia')]
[('s', 'http://www.wikidata.org/entity/Q2429397'), ('sName', 'Ancient Carthage')]
[('s', 'http://www.wikidata.org/entity/Q3327860'), ('sName', 'Celtic kingdom of Ireland')]
[('s', 'http://www.wikidata.org/entity/Q193748'), ('sName', 'history of ancient Israel and Judah')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('sName', 'Ancient Egypt')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('sName', 'Ancient Greece')]
[('s', 'http://www.wikidata.org/entity/Q41741'), ('sName', 'Thrace')]
[('s', 'http://www.wikidata.org/entity/Q128904'), ('sName', 'Elam')]
[('s', 'http://www.wikidata.org/entity/Q185068'

In [109]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')

# retrieving all ancient civilizations and durations

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName
WHERE {
   # bind something
   ?s ?p wd:Q28171280. 
   # get the label
   OPTIONAL {?s sc:name ?sName}.
   OPTIONAL {?p sc:name ?pName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q35355'), ('sName', 'Sumer'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q25558247'), ('sName', 'Chinese civilization'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q38060'), ('sName', 'Gaul'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q173082'), ('sName', 'Dacia'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q2429397'), ('sName', 'Ancient Carthage'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q3327860'), ('sName', 'Celtic kingdom

In [110]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')

# retrieving all ancient civilizations and durations

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q28171280 ?p ?o.
   # get the label
   OPTIONAL {?o sc:name ?oName}.
   OPTIONAL {?p sc:name ?pName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q8432'), ('oName', 'civilization')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('pName', 'PSH ID'), ('o', '5071')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('pName', 'Quora topic ID'), ('o', 'Ancient-Civilizations')]
[('p', 'http://www.wikidata.org/prop/direct/P3916'), ('pName', 'UNESCO Thesaurus ID'), ('o', 'concept2326')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('pName', 'KBpedia ID'), ('o', 'AncientCivilization')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P3916'), ('o', 'http://vocabularies.unesco.org/thesaurus/concept2326')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P8408'), ('o', 'http://kbpedia.org/kko/rc/AncientCivilization')]
[('p', 'http://schema.org/name'), ('o', 'ancient civilization')]
9


In [111]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')

# loking for duration of wd:Q11768 ('Ancient Egypt')

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q11768 ?p ?o.
   # get the label
   OPTIONAL {?o sc:name ?oName}.
   OPTIONAL {?p sc:name ?pName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q28171280'), ('oName', 'ancient civilization')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('pName', 'history of topic'), ('o', 'http://www.wikidata.org/entity/Q766876'), ('oName', 'history of ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q714601'), ('oName', 'Predynastic Period of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q1138524'), ('oName', 'Paulys Realenzyklopädie der klassischen Altertumswissenschaft')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('pName', 'continent'), ('o', 'http://www.wikidata.org/entity/Q15'), ('oName', 'Africa')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q17378135'), 

In [112]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName ?startTime ?endTime
WHERE {
   # bind something
   ?s ?p wd:Q28171280.
   OPTIONAL {?s wdt:P580 ?startTime}.
   OPTIONAL {?s wdt:P582 ?endTime}.
   
   # get the label
   OPTIONAL {?o sc:name ?oName}.
   OPTIONAL {?p sc:name ?pName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/P348'), ('oName', 'software version identifier'), ('startTime', '-1200-01-01T00:00:00Z'), ('endTime', '0600-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/P4669'), ('oName', 'tabular software version'), ('startTime', '-1200-01-01T00:00:00Z'), ('endTime', '0600-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/P747'), ('oName', 'has edition or translation'), ('startTime', '-1200-01-01T00:00:00Z'), ('endTime', '0600-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1002728'), ('oName', 'Fonsi Nieto'), ('startTime', '-1200-01-01T00:00:00Z'), ('endTime', '0600-01-01T00:00:00Z')]
[('p', 'http://www.wikidata.org/prop/di

In [113]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'

queryString = """
SELECT DISTINCT ?s ?sName ?startTime ?endTime
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280.
   OPTIONAL {?s wdt:P580 ?startTime}.
   OPTIONAL {?s wdt:P582 ?endTime}.
   OPTIONAL {?s sc:name ?sName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('startTime', '-3299-01-01T00:00:00Z'), ('endTime', '-1099-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('startTime', '-2000-01-01T00:00:00Z'), ('endTime', '-1099-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('sName', 'Ancient Egypt'), ('startTime', '-4000-01-01T00:00:00Z'), ('endTime', '0395-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('sName', 'Ancient Greece'), ('startTime', '-1200-01-01T00:00:00Z'), ('endTime', '0600-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('sName', 'Indus Valley Civilization'), ('startTime', '-2999-01-01T00:00:00Z'), ('endTime', '-1799-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('sName', 'Israelite period'), ('startTime', '-1199-01-01T00:00:00Z'), ('endTime', '-585-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/e

##### Attention duplicate Minoan civilization

#### GETTING DURATION for each ancient civilization

In [6]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'

queryString = """
SELECT DISTINCT ?s ?sName ?startTime ?endTime ?totYears
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280.
   OPTIONAL {?s sc:name ?sName}.
   
   OPTIONAL {
       ?s wdt:P580 ?startTime.
       ?s wdt:P582 ?endTime
       BIND(strdt(substr(str(?startTime), 1,5),xsd:int) as ?startYear)
       BIND(strdt(substr(str(?endTime), 1,5),xsd:int) as ?endYear)
       BIND(if(bound(?startYear) && bound(?endYear) , (?endYear), 0) as ?totYears)
    }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('startTime', '-3299-01-01T00:00:00Z'), ('endTime', '-1099-01-01T00:00:00Z'), ('totYears', '-1099')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('startTime', '-2000-01-01T00:00:00Z'), ('endTime', '-1099-01-01T00:00:00Z'), ('totYears', '-1099')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('sName', 'Ancient Egypt'), ('startTime', '-4000-01-01T00:00:00Z'), ('endTime', '0395-01-01T00:00:00Z'), ('totYears', '0395-')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('sName', 'Ancient Greece'), ('startTime', '-1200-01-01T00:00:00Z'), ('endTime', '0600-01-01T00:00:00Z'), ('totYears', '0600-')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('sName', 'Indus Valley Civilization'), ('startTime', '-2999-01-01T00:00:00Z'), ('endTime', '-1799-01-01T00:00:00Z'), ('totYears', '-1799')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('sName', 'Israelite perio

In [7]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'

queryString = """
SELECT DISTINCT ?s ?sName ?startTime ?endTime ?totYears
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280.
   OPTIONAL {?s sc:name ?sName}.
   
   OPTIONAL {
       ?s wdt:P580 ?startTime.
       ?s wdt:P582 ?endTime
       BIND(strdt(substr(str(?startTime), 1,5),xsd:int) as ?startYear)
       BIND(strdt(substr(str(?endTime), 1,5),xsd:int) as ?endYear)
       BIND(if(bound(?startYear) && bound(?endYear) , xsd:int(?endYear)-xsd:int(?startYear), 0) as ?totYears)
    }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('startTime', '-3299-01-01T00:00:00Z'), ('endTime', '-1099-01-01T00:00:00Z'), ('totYears', '2200')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('startTime', '-2000-01-01T00:00:00Z'), ('endTime', '-1099-01-01T00:00:00Z'), ('totYears', '901')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('sName', 'Ancient Egypt'), ('startTime', '-4000-01-01T00:00:00Z'), ('endTime', '0395-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('sName', 'Ancient Greece'), ('startTime', '-1200-01-01T00:00:00Z'), ('endTime', '0600-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('sName', 'Indus Valley Civilization'), ('startTime', '-2999-01-01T00:00:00Z'), ('endTime', '-1799-01-01T00:00:00Z'), ('totYears', '1200')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('sName', 'Israelite period'), ('startTime', '-1199-01-01T00:00:00Z'), ('end

In [8]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'

queryString = """
SELECT DISTINCT ?s ?sName ?startTime ?endTime ?totYears
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280.
   OPTIONAL {?s sc:name ?sName}.
   
   OPTIONAL {
       ?s wdt:P580 ?startTime.
       ?s wdt:P582 ?endTime
       BIND(strdt(substr(str(?startTime), 1,5),xsd:int) as ?startYear)
       BIND(strdt(substr(str(?endTime), 1,5),xsd:int) as ?endYear)
       BIND(if(bound(?startYear) && bound(?endYear) , strdt(xsd:int(?endYear)-xsd:int(?startYear),xsd:int), 0) as ?totYears)
    }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('startTime', '-3299-01-01T00:00:00Z'), ('endTime', '-1099-01-01T00:00:00Z'), ('totYears', '2200')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('startTime', '-2000-01-01T00:00:00Z'), ('endTime', '-1099-01-01T00:00:00Z'), ('totYears', '901')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('sName', 'Ancient Egypt'), ('startTime', '-4000-01-01T00:00:00Z'), ('endTime', '0395-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('sName', 'Ancient Greece'), ('startTime', '-1200-01-01T00:00:00Z'), ('endTime', '0600-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('sName', 'Indus Valley Civilization'), ('startTime', '-2999-01-01T00:00:00Z'), ('endTime', '-1799-01-01T00:00:00Z'), ('totYears', '1200')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('sName', 'Israelite period'), ('startTime', '-1199-01-01T00:00:00Z'), ('end

#### Coping with positive & negative years

In [13]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'

queryString = """
SELECT DISTINCT ?s ?sName ?startTime ?endTime ?startYear ?endYear ?totYears
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280.
   OPTIONAL {?s sc:name ?sName}.
   
   OPTIONAL {
       ?s wdt:P580 ?startTime.
       ?s wdt:P582 ?endTime
       BIND(substr(str(?startTime), 1,5) as ?startYearLiteral)
       BIND(substr(str(?endTime), 1,5) as ?endYearLiteral)
       
       BIND(if(strEnds(?startYearLiteral,"-"),substr(?startYearLiteral,1,4),?startYearLiteral) as ?startYear)
       BIND(if(strEnds(?endYearLiteral,"-"),substr(?endYearLiteral,1,4),?endYearLiteral) as ?endYear)
       
       BIND(if(bound(?startYear) && bound(?endYear) , strdt(xsd:int(?endYear)-xsd:int(?startYear),xsd:int), 0) as ?totYears)
    }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('startTime', '-3299-01-01T00:00:00Z'), ('endTime', '-1099-01-01T00:00:00Z'), ('startYear', '-3299'), ('endYear', '-1099'), ('totYears', '2200')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('startTime', '-2000-01-01T00:00:00Z'), ('endTime', '-1099-01-01T00:00:00Z'), ('startYear', '-2000'), ('endYear', '-1099'), ('totYears', '901')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('sName', 'Ancient Egypt'), ('startTime', '-4000-01-01T00:00:00Z'), ('endTime', '0395-01-01T00:00:00Z'), ('startYear', '-4000'), ('endYear', '0395'), ('totYears', '4395')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('sName', 'Ancient Greece'), ('startTime', '-1200-01-01T00:00:00Z'), ('endTime', '0600-01-01T00:00:00Z'), ('startYear', '-1200'), ('endYear', '0600'), ('totYears', '1800')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('sName', 'Indus Valley Civilization'), (

#### Found a possible solution with BIND + if + substr

In [14]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'

queryString = """
SELECT DISTINCT ?s ?sName ?totYears
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280.
   OPTIONAL {?s sc:name ?sName}.
   
   OPTIONAL {
       ?s wdt:P580 ?startTime.
       ?s wdt:P582 ?endTime
       BIND(substr(str(?startTime), 1,5) as ?startYearLiteral)
       BIND(substr(str(?endTime), 1,5) as ?endYearLiteral)
       
       BIND(if(strEnds(?startYearLiteral,"-"),substr(?startYearLiteral,1,4),?startYearLiteral) as ?startYear)
       BIND(if(strEnds(?endYearLiteral,"-"),substr(?endYearLiteral,1,4),?endYearLiteral) as ?endYear)
       
       BIND(if(bound(?startYear) && bound(?endYear) , strdt(xsd:int(?endYear)-xsd:int(?startYear),xsd:int), 0) as ?totYears)
    }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('totYears', '2200')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('totYears', '901')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('sName', 'Ancient Egypt'), ('totYears', '4395')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('sName', 'Ancient Greece'), ('totYears', '1800')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('sName', 'Indus Valley Civilization'), ('totYears', '1200')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('sName', 'Israelite period'), ('totYears', '614')]
[('s', 'http://www.wikidata.org/entity/Q35355'), ('sName', 'Sumer')]
[('s', 'http://www.wikidata.org/entity/Q25558247'), ('sName', 'Chinese civilization')]
[('s', 'http://www.wikidata.org/entity/Q38060'), ('sName', 'Gaul')]
[('s', 'http://www.wikidata.org/entity/Q173082'), ('sName', 'Dacia')]
[('s', 'http://www.wikidata.org/entity/Q2429397'), ('sName', 'Ancient Cart

#### Looking for alternatived for substr 

In [19]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?totYears
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280.
   OPTIONAL {?s sc:name ?sName}.
   
   OPTIONAL {
       ?s wdt:P580 ?startTime.
       ?s wdt:P582 ?endTime.
       
       BIND((xsd:dateTime(?endTime)-xsd:dateTime(?startTime)) as ?totYears)
    }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('totYears', '69426720000')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('totYears', '28433376000')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('sName', 'Ancient Egypt'), ('totYears', '138664051200')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('sName', 'Ancient Greece'), ('totYears', '56772057600')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('sName', 'Indus Valley Civilization'), ('totYears', '37869120000')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('sName', 'Israelite period'), ('totYears', '19376323200')]
[('s', 'http://www.wikidata.org/entity/Q35355'), ('sName', 'Sumer')]
[('s', 'http://www.wikidata.org/entity/Q25558247'), ('sName', 'Chinese civilization')]
[('s', 'http://www.wikidata.org/entity/Q38060'), ('sName', 'Gaul')]
[('s', 'http://www.wikidata.org/entity/Q173082'), ('sName', 'Dacia')]
[('s', 'http://www.wikidata.o

In [20]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?totYears ?startTime ?endTime
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280.
   OPTIONAL {?s sc:name ?sName}.
   
   OPTIONAL {
       ?s wdt:P580 ?startTime.
       ?s wdt:P582 ?endTime.
       
       BIND((xsd:dateTime(?endTime)-xsd:dateTime(?startTime)) as ?totYears)
    }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('totYears', '69426720000'), ('startTime', '-3299-01-01T00:00:00Z'), ('endTime', '-1099-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('totYears', '28433376000'), ('startTime', '-2000-01-01T00:00:00Z'), ('endTime', '-1099-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('sName', 'Ancient Egypt'), ('totYears', '138664051200'), ('startTime', '-4000-01-01T00:00:00Z'), ('endTime', '0395-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('sName', 'Ancient Greece'), ('totYears', '56772057600'), ('startTime', '-1200-01-01T00:00:00Z'), ('endTime', '0600-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('sName', 'Indus Valley Civilization'), ('totYears', '37869120000'), ('startTime', '-2999-01-01T00:00:00Z'), ('endTime', '-1799-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q155105

In [23]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'

# WOW now it does not give any error for dateTime conversion for ?startYear
queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?totYears ?startYear #year(xsd:dateTime(?startTime))# year(?endTime)
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280.
   OPTIONAL {?s sc:name ?sName}.
   
   OPTIONAL {
       ?s wdt:P580 ?startTime.
       ?s wdt:P582 ?endTime.
       
       BIND((xsd:dateTime(?endTime)-xsd:dateTime(?startTime)) as ?totYears)
       BIND(year(xsd:dateTime(?startTime)) as ?startYear)
    }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('totYears', '69426720000'), ('startYear', '-3299')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('totYears', '28433376000'), ('startYear', '-2000')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('sName', 'Ancient Egypt'), ('totYears', '138664051200'), ('startYear', '-4000')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('sName', 'Ancient Greece'), ('totYears', '56772057600'), ('startYear', '-1200')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('sName', 'Indus Valley Civilization'), ('totYears', '37869120000'), ('startYear', '-2999')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('sName', 'Israelite period'), ('totYears', '19376323200'), ('startYear', '-1199')]
[('s', 'http://www.wikidata.org/entity/Q35355'), ('sName', 'Sumer')]
[('s', 'http://www.wikidata.org/entity/Q25558247'), ('sName', 'Chinese civilization')]
[('s', 'http://www.wiki

In [24]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?totYears ?startYear ?endYear#year(xsd:dateTime(?startTime))# year(?endTime)
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280.
   OPTIONAL {?s sc:name ?sName}.
   
   OPTIONAL {
       ?s wdt:P580 ?startTime.
       ?s wdt:P582 ?endTime.
       
       BIND((xsd:dateTime(?endTime)-xsd:dateTime(?startTime)) as ?totYears)
       BIND(year(xsd:dateTime(?startTime)) as ?startYear)
       BIND(year(xsd:dateTime(?endTime)) as ?endYear)
    }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('totYears', '69426720000'), ('startYear', '-3299'), ('endYear', '-1099')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('totYears', '28433376000'), ('startYear', '-2000'), ('endYear', '-1099')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('sName', 'Ancient Egypt'), ('totYears', '138664051200'), ('startYear', '-4000'), ('endYear', '395')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('sName', 'Ancient Greece'), ('totYears', '56772057600'), ('startYear', '-1200'), ('endYear', '600')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('sName', 'Indus Valley Civilization'), ('totYears', '37869120000'), ('startYear', '-2999'), ('endYear', '-1799')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('sName', 'Israelite period'), ('totYears', '19376323200'), ('startYear', '-1199'), ('endYear', '-585')]
[('s', 'http://www.wikidata.org/entity/Q35355'), ('s

In [26]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?totYears
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280.
   OPTIONAL {?s sc:name ?sName}.
   
   OPTIONAL {
       ?s wdt:P580 ?startTime.
       ?s wdt:P582 ?endTime.
       
       BIND((year(xsd:dateTime(?endTime))-year(xsd:dateTime(?startTime))) as ?totYears)
    }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('totYears', '2200')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('totYears', '901')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('sName', 'Ancient Egypt'), ('totYears', '4395')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('sName', 'Ancient Greece'), ('totYears', '1800')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('sName', 'Indus Valley Civilization'), ('totYears', '1200')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('sName', 'Israelite period'), ('totYears', '614')]
[('s', 'http://www.wikidata.org/entity/Q35355'), ('sName', 'Sumer')]
[('s', 'http://www.wikidata.org/entity/Q25558247'), ('sName', 'Chinese civilization')]
[('s', 'http://www.wikidata.org/entity/Q38060'), ('sName', 'Gaul')]
[('s', 'http://www.wikidata.org/entity/Q173082'), ('sName', 'Dacia')]
[('s', 'http://www.wikidata.org/entity/Q2429397'), ('sName', 'Ancient Cart

#### Found 

In [38]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?s ?sName ?totYears
WHERE {
   # bind something
   ?s wdt:P31 wd:Q28171280.
   ?s sc:name ?sName.
   
   OPTIONAL {
       ?s wdt:P580 ?startTime.
       ?s wdt:P582 ?endTime.
       
       BIND((year(xsd:dateTime(?endTime))-year(xsd:dateTime(?startTime))) as ?totYears)
    }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('totYears', '901')]
[('s', 'http://www.wikidata.org/entity/Q134178'), ('sName', 'Minoan civilization'), ('totYears', '2200')]
[('s', 'http://www.wikidata.org/entity/Q11768'), ('sName', 'Ancient Egypt'), ('totYears', '4395')]
[('s', 'http://www.wikidata.org/entity/Q11772'), ('sName', 'Ancient Greece'), ('totYears', '1800')]
[('s', 'http://www.wikidata.org/entity/Q42534'), ('sName', 'Indus Valley Civilization'), ('totYears', '1200')]
[('s', 'http://www.wikidata.org/entity/Q1551054'), ('sName', 'Israelite period'), ('totYears', '614')]
[('s', 'http://www.wikidata.org/entity/Q35355'), ('sName', 'Sumer')]
[('s', 'http://www.wikidata.org/entity/Q25558247'), ('sName', 'Chinese civilization')]
[('s', 'http://www.wikidata.org/entity/Q38060'), ('sName', 'Gaul')]
[('s', 'http://www.wikidata.org/entity/Q173082'), ('sName', 'Dacia')]
[('s', 'http://www.wikidata.org/entity/Q2429397'), ('sName', 'Ancient Cart

### Final query for this task

In [5]:
# Find all the ancient civilizations. Return the IRI and name of the civilization and also their duration (in number of years) if it is available.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName ?totYears
WHERE {
   # bind something
   ?civilization wdt:P31 wd:Q28171280.
   ?civilization sc:name ?civilizationName.
   
   OPTIONAL {
       ?civilization wdt:P580 ?startTime.
       ?civilization wdt:P582 ?endTime.
       
       BIND((year(xsd:dateTime(?endTime))-year(xsd:dateTime(?startTime))) as ?totYears)
    }
   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q134178'), ('civilizationName', 'Minoan civilization'), ('totYears', '901')]
[('civilization', 'http://www.wikidata.org/entity/Q134178'), ('civilizationName', 'Minoan civilization'), ('totYears', '2200')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('totYears', '4395')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('totYears', '1800')]
[('civilization', 'http://www.wikidata.org/entity/Q42534'), ('civilizationName', 'Indus Valley Civilization'), ('totYears', '1200')]
[('civilization', 'http://www.wikidata.org/entity/Q1551054'), ('civilizationName', 'Israelite period'), ('totYears', '614')]
[('civilization', 'http://www.wikidata.org/entity/Q35355'), ('civilizationName', 'Sumer')]
[('civilization', 'http://www.wikidata.org/entity/Q25558247'), ('civilizationName', 'Chinese civilization')]
[('civilization', 'http://www.wikida

## Task 4
Identify the BGP for archeologist.

In [40]:
# Identify the BGP for archeologist.

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'
# --- end old findings ---

# EXPANDING poseidon wd:Q41127


queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q41127 ?p ?o. #poseidon
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1299'), ('pName', 'depicted by'), ('o', 'http://www.wikidata.org/entity/Q107043392'), ('oName', 'Poseidon or Corybant statue at Elis')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q106727050'), ('oName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19180675'), ('oName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q30059240'), ('oName', "Real'nyj slovar' klassicheskih drevnostej po Ljubkeru")]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q3181656'), ('oName', 'The Nuttall Encyclopædia')]
[('p', 'http://www.wikidata.

In [41]:
# Identify the BGP for archeologist.

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'
# --- end old findings ---

# EXPANDING poseidon wd:Q41127
queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q41127 ?p ?o. #poseidon
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
}
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1299'), ('pName', 'depicted by'), ('o', 'http://www.wikidata.org/entity/Q107043392'), ('oName', 'Poseidon or Corybant statue at Elis')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q106727050'), ('oName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19180675'), ('oName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q30059240'), ('oName', "Real'nyj slovar' klassicheskih drevnostej po Ljubkeru")]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q3181656'), ('oName', 'The Nuttall Encyclopædia')]
[('p', 'http://www.wikidata.

In [43]:
# Identify the BGP for archeologist.

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'
# --- end old findings ---

# EXPANDING Meidum Pyramid wd:Q13444093

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q13444093 ?p ?o.
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q30683'), ('oName', 'Beni Suef Governorate')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after'), ('o', 'http://www.wikidata.org/entity/Q1066424'), ('oName', 'Meidum')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q79'), ('oName', 'Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P186'), ('pName', 'made from material'), ('o', 'http://www.wikidata.org/entity/Q23757'), ('oName', 'limestone')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('o', 'http://www.wikidata.org/entity/Q1066424'), ('oName', 'Meidum')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1456099'), ('oName', 'step pyramid')]
[('p', 'http://www.wikidata.org/prop/direct/P366'

In [44]:
# Identify the BGP for archeologist.

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'
# --- end old findings ---

# EXPANDING Meidum Pyramid wd:Q13444093

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   ?o ?p wd:Q13444093.
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P971'), ('pName', 'category combines topics'), ('o', 'http://www.wikidata.org/entity/Q75129112'), ('oName', 'Category:Interior of Meidum Pyramid')]
[('p', 'http://www.w3.org/2002/07/owl#sameAs'), ('o', 'http://www.wikidata.org/entity/Q16590785')]
[('p', 'http://www.w3.org/2002/07/owl#sameAs'), ('o', 'http://www.wikidata.org/entity/Q23947029')]
[('p', 'http://schema.org/about'), ('o', 'https://www.wikidata.org/wiki/Special:EntityData/Q13444093')]
4


In [46]:
# Identify the BGP for archeologist.

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'
# --- end old findings ---

# EXPANDING wd:Q11768 (Ancient Egypt')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q11768 ?p ?o.
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal"), ('o', 'http://www.wikidata.org/entity/Q8440998'), ('oName', 'Portal:Ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q1138524'), ('oName', 'Paulys Realenzyklopädie der klassischen Altertumswissenschaft')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q4173137'), ('oName', 'Jewish Encyclopedia of Brockhaus and Efron')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q17378135'), ('oName', 'Great Soviet Encyclopedia (1969–1978)')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pName', 'follows'), ('o', 'http://www.wikidata.org/entity/Q714601'), ('oName', 'Predynastic Period of Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P155

In [47]:
# Identify the BGP for archeologist.

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'
# --- end old findings ---

# wd:Q11768 (Ancient Egypt') wdt:P155 ('follows')             wd:Q714601 ('Predynastic Period of Egypt')
#                            wdt:P155 ('follows')             wd:Q208520, ('Prehistoric Egypt')
#                            wdt:P2184 ('history of topic')   wd:Q766876 ('history of ancient Egypt')
#                            wdt:P2579 ('studied by')         wd:Q4752891 ('ancient Near Eastern studies')

# EXPANDING wd:Q4752891 ('ancient Near Eastern studies')
queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q4752891 ?p ?o.
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2578'), ('pName', 'studies'), ('o', 'http://www.wikidata.org/entity/Q11768'), ('oName', 'Ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P2578'), ('pName', 'studies'), ('o', 'http://www.wikidata.org/entity/Q269678'), ('oName', 'ancient Near East')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q23019185'), ('oName', 'Near Eastern studies')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11862829'), ('oName', 'academic discipline')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q258212'), ('oName', 'Near Eastern archaeology')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q145903'), ('oName', 'egyptology')]
[('p', 'http://www.wikidata.org/prop/direct/P52

In [48]:
# Identify the BGP for archeologist.

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'
# --- end old findings ---

# wd:Q11768 (Ancient Egypt') wdt:P155 ('follows')             wd:Q714601 ('Predynastic Period of Egypt')
#                            wdt:P155 ('follows')             wd:Q208520, ('Prehistoric Egypt')
#                            wdt:P2184 ('history of topic')   wd:Q766876 ('history of ancient Egypt')
#                            wdt:P2579 ('studied by')         wd:Q4752891 ('ancient Near Eastern studies')

# wd:Q4752891 ('ancient Near Eastern studies') wdt:P31 ('instance of') wd:Q11862829 ('academic discipline')

# EXPANDING wd:Q11862829 ('academic discipline')
queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q11862829 ?p ?o.
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('o', 'http://www.wikidata.org/entity/Q336'), ('oName', 'science')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('pName', 'Wikidata property'), ('o', 'http://www.wikidata.org/entity/P2579'), ('oName', 'studied by')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from'), ('o', 'http://www.wikidata.org/entity/Q82957'), ('oName', 'Discipline')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pName', 'has list'), ('o', 'http://www.wikidata.org/entity/Q221173'), ('oName', 'ocupen mi codigo perros')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1047113'), ('oName', 'specialty')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q24017414'), ('oName', 'second-order class')]
[('p', 'http://www.wikidata.org/prop/direct/P460'), ('pName'

In [49]:
# Identify the BGP for archeologist.

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'
# --- end old findings ---

# wd:Q11768 (Ancient Egypt') wdt:P155 ('follows')             wd:Q714601 ('Predynastic Period of Egypt')
#                            wdt:P155 ('follows')             wd:Q208520, ('Prehistoric Egypt')
#                            wdt:P2184 ('history of topic')   wd:Q766876 ('history of ancient Egypt')
#                            wdt:P2579 ('studied by')         wd:Q4752891 ('ancient Near Eastern studies')

# wd:Q4752891 ('ancient Near Eastern studies') wdt:P31 ('instance of') wd:Q11862829 ('academic discipline')

# EXPANDING wd:Q11862829 ('academic discipline') - BACKWARD
queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   ?o ?p wd:Q11862829.
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q101018399'), ('oName', 'University of California, Davis, Department of East Asian Languages and Cultures')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('o', 'http://www.wikidata.org/entity/Q1759955'), ('oName', 'multidisciplinarity')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pName', 'facet of'), ('o', 'http://www.wikidata.org/entity/Q849359'), ('oName', 'interdisciplinarity')]
[('p', 'http://www.wikidata.org/prop/direct/P1552'), ('pName', 'has quality'), ('o', 'http://www.wikidata.org/entity/Q207137'), ('oName', 'curriculum')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q80083'), ('oName', 'humanities')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q184784'), ('o

In [50]:
# Identify the BGP for archeologist.

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'
# --- end old findings ---

# wd:Q11768 (Ancient Egypt') wdt:P155 ('follows')             wd:Q714601 ('Predynastic Period of Egypt')
#                            wdt:P155 ('follows')             wd:Q208520, ('Prehistoric Egypt')
#                            wdt:P2184 ('history of topic')   wd:Q766876 ('history of ancient Egypt')
#                            wdt:P2579 ('studied by')         wd:Q4752891 ('ancient Near Eastern studies')

# wd:Q4752891 ('ancient Near Eastern studies') wdt:P31 ('instance of') wd:Q11862829 ('academic discipline')

# EXPANDING wd:Q11862829 ('academic discipline') - BACKWARD 
# + FITERING

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   ?o ?p wd:Q11862829.
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
    FILTER (regex(?oName,"arche*"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q184784'), ('oName', 'architectural drawing')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q4785472'), ('oName', 'archaeological sub-discipline')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q811338'), ('oName', 'buildings archaeology')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q107859286'), ('oName', 'neuroimaging research')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q47844'), ('oName', 'landscape architecture')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q96658223'), ('oName', 'polar research')]
[('p', 'htt

In [51]:
# Identify the BGP for archeologist.

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'
# --- end old findings ---

# wd:Q11768 (Ancient Egypt') wdt:P155 ('follows')             wd:Q714601 ('Predynastic Period of Egypt')
#                            wdt:P155 ('follows')             wd:Q208520, ('Prehistoric Egypt')
#                            wdt:P2184 ('history of topic')   wd:Q766876 ('history of ancient Egypt')
#                            wdt:P2579 ('studied by')         wd:Q4752891 ('ancient Near Eastern studies')

# wd:Q4752891 ('ancient Near Eastern studies') wdt:P31 ('instance of') wd:Q11862829 ('academic discipline')

# wd:Q23498 ('archaeology')  wdt:P31 ('instance of') wd:Q11862829 ('academic discipline')

# EXPANDING wd:Q23498 ('archaeology') - BACKWARD 

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   ?o ?p wd:Q23498.
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
    #FILTER (regex(?oName,"arche*"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q17181063'), ('oName', 'Richard Walter')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q17617360'), ('oName', 'Sergey Kuzminykh')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q4514244'), ('oName', 'Yevgeny Chernykh')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q1046137'), ('oName', 'Nikolay Veselovsky')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q22908764'), ('oName', 'Leslie Grinsell')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q1377678'), ('oName', 'Pyotr Kozlov')]
[('p', 'http://www.wikidata.

In [52]:
# Identify the BGP for archeologist.

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'
# --- end old findings ---

# wd:Q11768 (Ancient Egypt') wdt:P155 ('follows')             wd:Q714601 ('Predynastic Period of Egypt')
#                            wdt:P155 ('follows')             wd:Q208520, ('Prehistoric Egypt')
#                            wdt:P2184 ('history of topic')   wd:Q766876 ('history of ancient Egypt')
#                            wdt:P2579 ('studied by')         wd:Q4752891 ('ancient Near Eastern studies')

# wd:Q4752891 ('ancient Near Eastern studies') wdt:P31 ('instance of') wd:Q11862829 ('academic discipline')

# wd:Q23498 ('archaeology')  wdt:P31 ('instance of') wd:Q11862829 ('academic discipline')

# EXPANDING wd:Q23498 ('archaeology') - FORWARD
queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q23498 ?p ?o.
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
    #FILTER (regex(?oName,"arche*"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal"), ('o', 'http://www.wikidata.org/entity/Q7076022'), ('oName', 'Portal:Archaeology')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q4173137'), ('oName', 'Jewish Encyclopedia of Brockhaus and Efron')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q2657718'), ('oName', 'Armenian Soviet Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q29551022'), ('oName', 'Dictionnaire du patrimoine rennais')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19180675'), ('oName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343

#### FOUND wd:Q3621491 ('archaeologist')

In [53]:
# Identify the BGP for archeologist.

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'
# --- end old findings ---

# wd:Q11768 (Ancient Egypt') wdt:P155 ('follows')             wd:Q714601 ('Predynastic Period of Egypt')
#                            wdt:P155 ('follows')             wd:Q208520, ('Prehistoric Egypt')
#                            wdt:P2184 ('history of topic')   wd:Q766876 ('history of ancient Egypt')
#                            wdt:P2579 ('studied by')         wd:Q4752891 ('ancient Near Eastern studies')

# wd:Q4752891 ('ancient Near Eastern studies') wdt:P31 ('instance of') wd:Q11862829 ('academic discipline')

# wd:Q23498 ('archaeology')  wdt:P31 ('instance of') wd:Q11862829 ('academic discipline')

# FOUND wd:Q23498 ('archaeology') wdt:P3095 ('practiced by') wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q23498 wdt:P3095 ?o.
   OPTIONAL {
       ?o sc:name ?oName.
    }   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3621491'), ('oName', 'archaeologist')]
1


#### Prob: archeologist != archaeologist

In [54]:
# Identify the BGP for archeologist.

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'
# --- end old findings ---

# wd:Q11768 (Ancient Egypt') wdt:P155 ('follows')             wd:Q714601 ('Predynastic Period of Egypt')
#                            wdt:P155 ('follows')             wd:Q208520, ('Prehistoric Egypt')
#                            wdt:P2184 ('history of topic')   wd:Q766876 ('history of ancient Egypt')
#                            wdt:P2579 ('studied by')         wd:Q4752891 ('ancient Near Eastern studies')

# wd:Q4752891 ('ancient Near Eastern studies') wdt:P31 ('instance of') wd:Q11862829 ('academic discipline')

# wd:Q23498 ('archaeology')  wdt:P31 ('instance of') wd:Q11862829 ('academic discipline')

# wd:Q23498 ('archaeology') --wdt:P3095 ('practiced by')--> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q23498 wdt:P3095 ?o.
   OPTIONAL {
       ?o sc:name ?oName.
    }   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3621491'), ('oName', 'archaeologist')]
1


#### FINDING if there is a BGP for archeologist as requested or is the same thing and there is an typo while writing the request

In [57]:
#Identify the BGP for archeologist.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

# wd:Q11768 wdt:P2579 ('studied by') wd:Q4752891 ('ancient Near Eastern studies')
# wd:Q4752891 wdt:P527 ('has part') wd:Q145903 ('egyptology')
queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q4752891 ?p ?o.
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
    #FILTER (regex(?oName,"arche*"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2578'), ('pName', 'studies'), ('o', 'http://www.wikidata.org/entity/Q11768'), ('oName', 'Ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P2578'), ('pName', 'studies'), ('o', 'http://www.wikidata.org/entity/Q269678'), ('oName', 'ancient Near East')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q23019185'), ('oName', 'Near Eastern studies')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q11862829'), ('oName', 'academic discipline')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q258212'), ('oName', 'Near Eastern archaeology')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part'), ('o', 'http://www.wikidata.org/entity/Q145903'), ('oName', 'egyptology')]
[('p', 'http://www.wikidata.org/prop/direct/P52

In [59]:
#Identify the BGP for archeologist.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

# wd:Q11768 wdt:P2579 ('studied by') wd:Q4752891 ('ancient Near Eastern studies')
# wd:Q4752891 wdt:P527 ('has part') wd:Q145903 ('egyptology')
# wd:Q145903 wdt:P279 ('subclass of') wd:Q23498 ('archaeology')
queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q145903 ?p ?o.
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
    #FILTER (regex(?oName,"arche*"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q3181656'), ('oName', 'The Nuttall Encyclopædia')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19180675'), ('oName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q2657718'), ('oName', 'Armenian Soviet Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q602358'), ('oName', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P2578'), ('pName', 'studies'), ('o', 'http://www.wikidata.org/entity/Q135028'), ('oName', 'Nubia')]
[('p', 'http://www.wikidata.org/prop/direct/P2578'), ('pName', 'studies'), ('o', 'http://www.wik

In [63]:
#Identify the BGP for archeologist.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

# wd:Q11768 wdt:P2579 ('studied by') wd:Q4752891 ('ancient Near Eastern studies')
# wd:Q4752891 wdt:P527 ('has part') wd:Q145903 ('egyptology')
# wd:Q145903 wdt:P279 ('subclass of') wd:Q23498 ('archaeology')
queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q23498 ?p ?o.
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
}
LIMIT 150
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal"), ('o', 'http://www.wikidata.org/entity/Q7076022'), ('oName', 'Portal:Archaeology')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q4173137'), ('oName', 'Jewish Encyclopedia of Brockhaus and Efron')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q2657718'), ('oName', 'Armenian Soviet Encyclopedia')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q29551022'), ('oName', 'Dictionnaire du patrimoine rennais')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19180675'), ('oName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343

In [65]:
#Identify the BGP for archeologist.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

# wd:Q11768 wdt:P2579 ('studied by') wd:Q4752891 ('ancient Near Eastern studies')
# wd:Q4752891 wdt:P527 ('has part') wd:Q145903 ('egyptology')
# wd:Q145903 wdt:P279 ('subclass of') wd:Q23498 ('archaeology')
queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   ?o ?p wd:Q23498 .
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
    FILTER (?p not in (wdt:P101, wdt:P106,wdt:P1269))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('o', 'http://www.wikidata.org/entity/Q683974'), ('oName', 'Ernest-Aimé Feydeau')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('o', 'http://www.wikidata.org/entity/Q30081010'), ('oName', 'Prehistoric Textiles')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('o', 'http://www.wikidata.org/entity/Q21685950'), ('oName', 'Integrating Archaeology')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('o', 'http://www.wikidata.org/entity/Q71937877'), ('oName', 'SPARQL Unicorn')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('o', 'http://www.wikidata.org/entity/Q74823867'), ('oName', 'profileAAR')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('o', 'http://www.wikidata.org/entity/Q26926958'), ('oName', 'Hispania Antiqua')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'g

In [68]:
#Identify the BGP for archeologist.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

# wd:Q11768 wdt:P2579 ('studied by') wd:Q4752891 ('ancient Near Eastern studies')
# wd:Q4752891 wdt:P527 ('has part') wd:Q145903 ('egyptology')
# wd:Q145903 wdt:P279 ('subclass of') wd:Q23498 ('archaeology')
queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   ?o ?p wd:Q23498 .
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
    FILTER (?p not in (wdt:P101, wdt:P106,wdt:P1269))
    FILTER (regex(?oName,"archeol.*","i"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after'), ('o', 'http://www.wikidata.org/entity/Q43386302'), ('oName', 'Archeologická')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location'), ('o', 'http://www.wikidata.org/entity/Q56325219'), ('oName', 'Archeology of Photography Foundation')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q75084710'), ('oName', 'archeology of the future')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('pName', 'main subject'), ('o', 'http://www.wikidata.org/entity/Q29031417'), ('oName', 'Archeology and the evolution of human behavior')]
[('p', 'http://www.wikidata.org/prop/direct/P921'), ('pName', 'main subject'), ('o', 'http://www.wikidata.org/entity/Q30149438'), ('oName', 'Human spiruridiasis due to Physaloptera spp. (Nematoda: Physalopteridae) in a grave of the Shahr-e Sukhteh archeological site of the Bronze Age (2800–2500 BC)

In [70]:
#Identify the BGP for archeologist.

# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

# wd:Q11768 wdt:P2579 ('studied by') wd:Q4752891 ('ancient Near Eastern studies')
# wd:Q4752891 wdt:P527 ('has part') wd:Q145903 ('egyptology')
# wd:Q145903 wdt:P279 ('subclass of') wd:Q23498 ('archaeology')
queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q75084710 ?p ?o .
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q23498'), ('oName', 'archaeology')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q64448878'), ('oName', 'futuristic fiction')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1697305'), ('oName', 'narrative motif')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://schema.org/name'), ('o', 'archeology of the future')]
5


#### Can't find any archeologist. Internet says that archaeologist (with an 'a') is the correct spelling, wrt archeologist.

### Final query for this task

In [6]:
# Identify the BGP for archeologist.

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
#                                P580 ('start time'), '-4000-01-01T00:00:00Z'
#                                P582 ('end time'), '0395-01-01T00:00:00Z'
# --- end old findings ---

# wd:Q11768 (Ancient Egypt') wdt:P155 ('follows')             wd:Q714601 ('Predynastic Period of Egypt')
#                            wdt:P155 ('follows')             wd:Q208520, ('Prehistoric Egypt')
#                            wdt:P2184 ('history of topic')   wd:Q766876 ('history of ancient Egypt')
#                            wdt:P2579 ('studied by')         wd:Q4752891 ('ancient Near Eastern studies')

# wd:Q4752891 ('ancient Near Eastern studies') wdt:P31 ('instance of') wd:Q11862829 ('academic discipline')

# wd:Q23498 ('archaeology')  wdt:P31 ('instance of') wd:Q11862829 ('academic discipline')

# wd:Q23498 ('archaeology') --wdt:P3095 ('practiced by')--> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   wd:Q23498 wdt:P3095 ?o. # wd:Q23498 ('archaeology') --wdt:P3095 ('practiced by')--> wd:Q3621491 ('archaeologist')
   OPTIONAL {
       ?o sc:name ?oName.
    }   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q3621491'), ('oName', 'archaeologist')]
1


## Task 5
For each ancient civilization find the number of archeologists who worked on it (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

In [71]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# EXAPANDING wd:Q28171280 ('ancient civilization')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q28171280 ?p ?o.
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }   
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q8432'), ('oName', 'civilization')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P3916'), ('o', 'http://vocabularies.unesco.org/thesaurus/concept2326')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P8408'), ('o', 'http://kbpedia.org/kko/rc/AncientCivilization')]
[('p', 'http://www.wikidata.org/prop/direct/P1051'), ('o', '5071')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('o', 'Ancient-Civilizations')]
[('p', 'http://www.wikidata.org/prop/direct/P3916'), ('o', 'concept2326')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('o', 'AncientCivilization')]
[('p', 'http://schema.org/name'), ('o', 'ancient civilization')]
9


In [74]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# EXAPANDING wd:Q11768 ('Ancient Egypt')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   VALUES(?civilization){
       (wd:Q11768)#('Ancient Egypt')
   }
   ?civilization wdt:P31 wd:Q28171280;
           ?p ?o.
   OPTIONAL {
       ?civilization sc:name ?civilizationName.
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2579'), ('pName', 'studied by'), ('o', 'http://www.wikidata.org/entity/Q4752891'), ('oName', 'ancient Near Eastern studies')]
[('p', 'http://www.wikidata.org/prop/direct/P5125'), ('pName', 'Wikimedia outline'), ('o', 'http://www.wikidata.org/entity/Q7112549'), ('oName', 'outline of ancient Egypt')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q17378135'), ('oName', 'Great Soviet Encyclopedia (1969–1978)')]
[('p', 'http://www.wikidata.org/prop/direct/P6885'), ('pName', 'historical region'), ('o', 'http://www.wikidata.org/entity/Q269678'), ('oName', 'ancient Near East')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1620908'), ('oName', 'historical region')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q113852

In [81]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# EXAPANDING wd:Q11768 ('Ancient Egypt') BACKWARD

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   VALUES(?civilization){
       (wd:Q11768)#('Ancient Egypt')
   }
   ?civilization wdt:P31 wd:Q28171280.
   ?o ?p ?civilization.
   OPTIONAL {
       ?civilization sc:name ?civilizationName.
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2647'), ('pName', 'source of material'), ('o', 'http://www.wikidata.org/entity/Q340032'), ('oName', 'mumia')]
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction'), ('o', 'http://www.wikidata.org/entity/Q732358'), ('oName', 'nomarch')]
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction'), ('o', 'http://www.wikidata.org/entity/Q37110'), ('oName', 'pharaoh')]
[('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction'), ('o', 'http://www.wikidata.org/entity/Q1226867'), ('oName', 'Dioiketes')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q4793175'), ('oName', 'Armenian Egyptology Centre')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q61942824'), ('oName', 'Eugene Cruz-Uribe')]
[('p', 'http://

In [82]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

# wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---
# wd:Q631410 ('Nikephoros') --wdt:P101 ('field of work')--> wd:Q11768 (Ancient Egypt')

# EXAPANDING wd:Q11768 ('Ancient Egypt') with wdt:P101 ('field of work')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName
WHERE {
   # bind something
   VALUES(?civilization){
       (wd:Q11768)#('Ancient Egypt')
   }
   ?civilization wdt:P31 wd:Q28171280.
   ?o wdt:P101 ?civilization. #'field of work'
   OPTIONAL {
       ?civilization sc:name ?civilizationName.
       ?o sc:name ?oName.
    }
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q61942824'), ('oName', 'Eugene Cruz-Uribe')]
[('o', 'http://www.wikidata.org/entity/Q631410'), ('oName', 'Nikephoros')]
[('o', 'http://www.wikidata.org/entity/Q4793175'), ('oName', 'Armenian Egyptology Centre')]
[('o', 'http://www.wikidata.org/entity/Q56000728'), ('oName', 'Roger Rémondon')]
[('o', 'http://www.wikidata.org/entity/Q18115652'), ('oName', 'Gratien-Michel Ollivier-Beauregard')]
5


In [84]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---
# wd:Q631410 ('Nikephoros') --wdt:P101 ('field of work')--> wd:Q11768 (Ancient Egypt')

# getting all ancient civilization's archeologists, who has wdt:P101 ('field of work') a civilization

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName (COUNT(*) as ?archaeologist) #?o ?oName
WHERE {
   # bind something
   #VALUES(?civilization){
   #    (wd:Q11768)#('Ancient Egypt')
   #}
   ?civilization wdt:P31 wd:Q28171280.
   ?o wdt:P101 ?civilization. #'field of work'
   
   ?civilization sc:name ?civilizationName.
   ?o sc:name ?oName.
}
GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q11767'), ('civilizationName', 'Mesopotamia'), ('archaeologist', '3')]
[('civilization', 'http://www.wikidata.org/entity/Q193748'), ('civilizationName', 'history of ancient Israel and Judah'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '29')]
[('civilization', 'http://www.wikidata.org/entity/Q1551054'), ('civilizationName', 'Israelite period'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '8')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '5')]
[('civilization', 'http://www.wikidata.org/entity/Q41741'), ('civilizationName', 'Thrace'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q38060'), ('civilizationName', 'Gaul'), ('archaeologist

In [88]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---
# wd:Q631410 ('Nikephoros') --wdt:P101 ('field of work')--> wd:Q11768 (Ancient Egypt')

# getting all ancient civilization's archeologists, who has wdt:P101 ('field of work') a civilization
# checking if we are talking about humans

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?student ?studentName ?p ?pName ?o ?oName# ?civilization ?civilizationName (COUNT(*) as ?archaeologist) 
WHERE {
   # bind something
   #VALUES(?civilization){
   #    (wd:Q11768)#('Ancient Egypt')
   #}
   ?civilization wdt:P31 wd:Q28171280.
   ?student wdt:P101 ?civilization. #'field of work'
   
   ?civilization sc:name ?civilizationName.
   ?student sc:name ?studentName.
   
   #checking if its human
   ?student ?p ?o.
   
   OPTIONAL{
   ?p sc:name ?pName.
   ?o sc:name ?oName.
   }
}
#GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('student', 'http://www.wikidata.org/entity/Q2342814'), ('studentName', 'Alexandre Bertrand'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q38060'), ('oName', 'Gaul')]
[('student', 'http://www.wikidata.org/entity/Q3035177'), ('studentName', 'Dominique Garcia'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q38060'), ('oName', 'Gaul')]
[('student', 'http://www.wikidata.org/entity/Q346765'), ('studentName', 'Albert Grenier'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q38060'), ('oName', 'Gaul')]
[('student', 'http://www.wikidata.org/entity/Q3035177'), ('studentName', 'Dominique Garcia'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q695827'), ('oName', 'protohistory')]
[('student', 'http://www

In [90]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---
# wd:Q631410 ('Nikephoros') --wdt:P101 ('field of work')--> wd:Q11768 (Ancient Egypt')

# getting all ancient civilization's archeologists, who has wdt:P101 ('field of work') a civilization
# checking if we are talking about humans
# TAKING ONLY archaeologists now

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName (COUNT(*) as ?archaeologist) 
WHERE {
   # bind something
   #VALUES(?civilization){
   #    (wd:Q11768)#('Ancient Egypt')
   #}
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
   ?student wdt:P101 ?civilization; #'field of work'
            sc:name ?studentName;
            wdt:P106 wd:Q3621491. #P106 'occupation'-> Q3621491 ('archaeologist)
}
GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q193748'), ('civilizationName', 'history of ancient Israel and Judah'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q11767'), ('civilizationName', 'Mesopotamia'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '7')]
[('civilization', 'http://www.wikidata.org/entity/Q1551054'), ('civilizationName', 'Israelite period'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '7')]
[('civilization', 'http://www.wikidata.org/entity/Q41741'), ('civilizationName', 'Thrace'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q38060'), ('civilizationName', 'Gaul'), ('archaeologist', '3')]
[('civilization', 'http://www.wikidata.org/entity/Q28567'), ('civilizationName', 'Maya civilization'), ('archaeolog

#### We have very few archaeologist, lets find other ways to find them (other than field of work)

In [91]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?student ?studentName ?p ?pName #?civilization ?civilizationName (COUNT(*) as ?archaeologist) 
WHERE {
   # bind something
   #VALUES(?civilization){
   #    (wd:Q11768)#('Ancient Egypt')
   #}
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
   ?student ?p ?civilization; #'field of work'
            sc:name ?studentName;
            wdt:P106 wd:Q3621491. #P106 'occupation'-> Q3621491 ('archaeologist)
    ?p sc:name ?pName.
}
#GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('student', 'http://www.wikidata.org/entity/Q2342814'), ('studentName', 'Alexandre Bertrand'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('student', 'http://www.wikidata.org/entity/Q1985476'), ('studentName', 'Charles Thomas Newton'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('student', 'http://www.wikidata.org/entity/Q1523942'), ('studentName', 'Gilbert Charles-Picard'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('student', 'http://www.wikidata.org/entity/Q64563606'), ('studentName', "Vincent Robin d'Arba Desborough"), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('student', 'http://www.wikidata.org/entity/Q2959998'), ('studentName', 'Charles Picard'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('student', 'http://www.wikidata.org/entity/Q107185282'), ('studentName', 'Orlando Cerasuolo'), ('p

In [2]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName ?p ?pName ?civilizationName
WHERE {
   # bind something
   #VALUES(?civilization){
   #    (wd:Q11768)#('Ancient Egypt')
   #}
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
                 
   ?o ?p ?civilization.
   
   OPTIONAL{
       ?o sc:name ?oName.
       ?p sc:name ?pName.
    }
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q9076903'), ('oName', 'Shutrukid Dynasty'), ('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('civilizationName', 'Elam')]
[('o', 'http://www.wikidata.org/entity/Q16202253'), ('oName', 'Neterkheperre Meryptah called Pipi II'), ('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('civilizationName', 'Ancient Egypt')]
[('o', 'http://www.wikidata.org/entity/Q4804543'), ('oName', 'Ashakhet II'), ('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('civilizationName', 'Ancient Egypt')]
[('o', 'http://www.wikidata.org/entity/Q17319063'), ('oName', 'Wenennefer'), ('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('civilizationName', 'Ancient Egypt')]
[('o', 'http://www.wikidata.org/entity/Q17319103'), ('oName', 'Yuyu'), ('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pName', 'time period'), ('civilizationName', 'Ancient Egypt')]
[('o', 'ht

In [3]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName ?p ?pName ?civilizationName
WHERE {
   # bind something
   #VALUES(?civilization){
   #    (wd:Q11768)#('Ancient Egypt')
   #}
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
                 
   ?o ?p ?civilization.
   
   OPTIONAL{
       ?o sc:name ?oName.
       ?p sc:name ?pName.
    }
    FILTER (?p not in (wdt:P2348, wdt:P17,wdt:P19))
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q524447'), ('oName', 'Standard of Ur'), ('p', 'http://www.wikidata.org/prop/direct/P135'), ('pName', 'movement'), ('civilizationName', 'Sumer')]
[('o', 'http://www.wikidata.org/entity/Q2915613'), ('oName', 'Ram in a Thicket'), ('p', 'http://www.wikidata.org/prop/direct/P135'), ('pName', 'movement'), ('civilizationName', 'Sumer')]
[('o', 'http://www.wikidata.org/entity/Q65091764'), ('oName', 'Bull Headed Lyre of Ur'), ('p', 'http://www.wikidata.org/prop/direct/P135'), ('pName', 'movement'), ('civilizationName', 'Sumer')]
[('o', 'http://www.wikidata.org/entity/Q2915613'), ('oName', 'Ram in a Thicket'), ('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('civilizationName', 'Sumer')]
[('o', 'http://www.wikidata.org/entity/Q65091764'), ('oName', 'Bull Headed Lyre of Ur'), ('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('civilizationName', 'Sumer')]
[('o', 'http://www.wikidata.org/entity/Q105794982'),

In [98]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName ?p ?pName ?civilizationName
WHERE {
   # bind something
   #VALUES(?civilization){
   #    (wd:Q11768)#('Ancient Egypt')
   #}
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
                 
   ?o ?p ?civilization.
   
   OPTIONAL{
       ?o sc:name ?oName.
       ?p sc:name ?pName.
    }
    FILTER (?p not in (wdt:P2348, wdt:P17,wdt:P19,wdt:P20))
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q524447'), ('oName', 'Standard of Ur'), ('p', 'http://www.wikidata.org/prop/direct/P135'), ('pName', 'movement'), ('civilizationName', 'Sumer')]
[('o', 'http://www.wikidata.org/entity/Q2915613'), ('oName', 'Ram in a Thicket'), ('p', 'http://www.wikidata.org/prop/direct/P135'), ('pName', 'movement'), ('civilizationName', 'Sumer')]
[('o', 'http://www.wikidata.org/entity/Q65091764'), ('oName', 'Bull Headed Lyre of Ur'), ('p', 'http://www.wikidata.org/prop/direct/P135'), ('pName', 'movement'), ('civilizationName', 'Sumer')]
[('o', 'http://www.wikidata.org/entity/Q2915613'), ('oName', 'Ram in a Thicket'), ('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('civilizationName', 'Sumer')]
[('o', 'http://www.wikidata.org/entity/Q65091764'), ('oName', 'Bull Headed Lyre of Ur'), ('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre'), ('civilizationName', 'Sumer')]
[('o', 'http://www.wikidata.org/entity/Q105794982'),

In [99]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName ?p ?pName
WHERE {
   # bind something
   #VALUES(?civilization){
   #    (wd:Q11768)#('Ancient Egypt')
   #}
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
                 
   ?o ?p ?civilization.
   
   OPTIONAL{
       ?o sc:name ?oName.
       ?p sc:name ?pName.
    }
    FILTER (?p not in (wdt:P2348, wdt:P17,wdt:P19,wdt:P20))
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q524447'), ('oName', 'Standard of Ur'), ('p', 'http://www.wikidata.org/prop/direct/P135'), ('pName', 'movement')]
[('o', 'http://www.wikidata.org/entity/Q2915613'), ('oName', 'Ram in a Thicket'), ('p', 'http://www.wikidata.org/prop/direct/P135'), ('pName', 'movement')]
[('o', 'http://www.wikidata.org/entity/Q65091764'), ('oName', 'Bull Headed Lyre of Ur'), ('p', 'http://www.wikidata.org/prop/direct/P135'), ('pName', 'movement')]
[('o', 'http://www.wikidata.org/entity/Q2915613'), ('oName', 'Ram in a Thicket'), ('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre')]
[('o', 'http://www.wikidata.org/entity/Q65091764'), ('oName', 'Bull Headed Lyre of Ur'), ('p', 'http://www.wikidata.org/prop/direct/P136'), ('pName', 'genre')]
[('o', 'http://www.wikidata.org/entity/Q105794982'), ('oName', "Who was the world's first author? - Soraya Field Fiorio"), ('p', 'http://www.wikidata.org/prop/direct/P180'), ('pName', 'depicts')]
[('o', 'htt

In [100]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---
# [('o', 'http://www.wikidata.org/entity/Q244746'), ('oName', 'Larsa'), ('p', 'http://www.wikidata.org/prop/direct/P2596'), ('pName', 'culture')]

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   wd:Q244746 ?p ?o.
   
   OPTIONAL{
       ?o sc:name ?oName.
       ?p sc:name ?pName.
    }
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2596'), ('pName', 'culture'), ('o', 'http://www.wikidata.org/entity/Q35355'), ('oName', 'Sumer')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q486972'), ('oName', 'human settlement')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q602358'), ('oName', 'Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P421'), ('pName', 'located in time zone'), ('o', 'http://www.wikidata.org/entity/Q6760'), ('oName', 'UTC+03:00')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country'), ('o', 'http://www.wikidata.org/entity/Q796'), ('oName', 'Iraq')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q839954'), ('oName', 'archaeological site')]
[('p', 'http://www.wikidata.org/prop/direc

In [101]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---
# [('o', 'http://www.wikidata.org/entity/Q244746'), ('oName', 'Larsa'), ('p', 'http://www.wikidata.org/prop/direct/P2596'), ('pName', 'culture')]

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   ?o ?p wd:Q244746.
   
   OPTIONAL{
       ?o sc:name ?oName.
       ?p sc:name ?pName.
    }
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity'), ('o', 'http://www.wikidata.org/entity/Q19309800'), ('oName', 'Ebabbar')]
[('p', 'http://www.wikidata.org/prop/direct/P189'), ('pName', 'location of discovery'), ('o', 'http://www.wikidata.org/entity/Q7980851'), ('oName', 'Weld-Blundell Prism')]
[('p', 'http://www.wikidata.org/prop/direct/P1071'), ('pName', 'location of creation'), ('o', 'http://www.wikidata.org/entity/Q7980851'), ('oName', 'Weld-Blundell Prism')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after'), ('o', 'http://www.wikidata.org/entity/Q13577667'), ('oName', 'Isin-Larsa period')]
[('p', 'http://www.wikidata.org/prop/direct/P189'), ('pName', 'location of discovery'), ('o', 'http://www.wikidata.org/entity/Q87240223'), ('oName', 'winged goddess-AO 6501')]
[('p', 'http://www.wikidata.org/prop/direct/P189'), ('pName', 'location of discovery'), ('o', 'http://www.wikidata.org/enti

In [103]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName ?p ?pName
WHERE {
   # bind something
   #VALUES(?civilization){
   #    (wd:Q11768)#('Ancient Egypt')
   #}
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
                 
   ?o ?p ?civilization.
   
   OPTIONAL{
       ?o sc:name ?oName.
       ?p sc:name ?pName.
    }
    FILTER (?p not in (wdt:P2348, wdt:P17,wdt:P19,wdt:P20,wdt:P136,wdt:P2596))
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q524447'), ('oName', 'Standard of Ur'), ('p', 'http://www.wikidata.org/prop/direct/P135'), ('pName', 'movement')]
[('o', 'http://www.wikidata.org/entity/Q2915613'), ('oName', 'Ram in a Thicket'), ('p', 'http://www.wikidata.org/prop/direct/P135'), ('pName', 'movement')]
[('o', 'http://www.wikidata.org/entity/Q65091764'), ('oName', 'Bull Headed Lyre of Ur'), ('p', 'http://www.wikidata.org/prop/direct/P135'), ('pName', 'movement')]
[('o', 'http://www.wikidata.org/entity/Q105794982'), ('oName', "Who was the world's first author? - Soraya Field Fiorio"), ('p', 'http://www.wikidata.org/prop/direct/P180'), ('pName', 'depicts')]
[('o', 'http://www.wikidata.org/entity/Q16281782'), ('oName', 'Sümer'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('o', 'http://www.wikidata.org/entity/Q36790'), ('oName', 'Sumerian'), ('p', 'http://www.wikidata.org/prop/direct/P2341'), ('pName', 'indigenous to')]
[('o', 'http://www.

In [106]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName ?p ?pName
WHERE {
   # bind something
   #VALUES(?civilization){
   #    (wd:Q11768)#('Ancient Egypt')
   #}
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
                 
   ?o ?p ?civilization.
   
   OPTIONAL{
       ?o sc:name ?oName.
       ?p sc:name ?pName.
    }
    FILTER (?p not in (wdt:P2348, wdt:P17,wdt:P19,wdt:P20,wdt:P136,wdt:P2596,wdt:P27))
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q524447'), ('oName', 'Standard of Ur'), ('p', 'http://www.wikidata.org/prop/direct/P135'), ('pName', 'movement')]
[('o', 'http://www.wikidata.org/entity/Q2915613'), ('oName', 'Ram in a Thicket'), ('p', 'http://www.wikidata.org/prop/direct/P135'), ('pName', 'movement')]
[('o', 'http://www.wikidata.org/entity/Q65091764'), ('oName', 'Bull Headed Lyre of Ur'), ('p', 'http://www.wikidata.org/prop/direct/P135'), ('pName', 'movement')]
[('o', 'http://www.wikidata.org/entity/Q105794982'), ('oName', "Who was the world's first author? - Soraya Field Fiorio"), ('p', 'http://www.wikidata.org/prop/direct/P180'), ('pName', 'depicts')]
[('o', 'http://www.wikidata.org/entity/Q16281782'), ('oName', 'Sümer'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pName', 'different from')]
[('o', 'http://www.wikidata.org/entity/Q36790'), ('oName', 'Sumerian'), ('p', 'http://www.wikidata.org/prop/direct/P2341'), ('pName', 'indigenous to')]
[('o', 'http://www.

In [107]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') wdt:P921 ('main subject')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName ?p ?pName
WHERE {
   # bind something
   #VALUES(?civilization){
   #    (wd:Q11768)#('Ancient Egypt')
   #}
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
                 
   ?article wdt:P921 ?civilization. #mainSubject
   ?article ?p ?o.
   
   OPTIONAL{
       ?o sc:name ?oName.
       ?p sc:name ?pName.
    }
    #FILTER (?p not in (wdt:P2348, wdt:P17,wdt:P19,wdt:P20,wdt:P136,wdt:P2596,wdt:P27))
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q2520013'), ('oName', 'sculpture in the round'), ('p', 'http://www.wikidata.org/prop/direct/P921'), ('pName', 'main subject')]
[('o', 'http://www.wikidata.org/entity/Q21481766'), ('oName', 'academic chapter'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of')]
[('o', 'http://www.wikidata.org/entity/Q3083196'), ('oName', 'French of France'), ('p', 'http://www.wikidata.org/prop/direct/P407'), ('pName', 'language of work or name')]
[('o', 'http://www.wikidata.org/entity/Q743255'), ('oName', 'Barbegal aqueduct'), ('p', 'http://www.wikidata.org/prop/direct/P921'), ('pName', 'main subject')]
[('o', 'http://www.wikidata.org/entity/Q636481'), ('oName', 'paleoethnobotany'), ('p', 'http://www.wikidata.org/prop/direct/P921'), ('pName', 'main subject')]
[('o', 'http://www.wikidata.org/entity/Q27096638'), ('oName', 'Room 304'), ('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location')]
[('o', 'http://www.wikidata

In [108]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')

# main subject --> author --> is an archaeologist?

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?o ?oName ?p ?pName
WHERE {
   # bind something
   #VALUES(?civilization){
   #    (wd:Q11768)#('Ancient Egypt')
   #}
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
                 
   ?article wdt:P921 ?civilization. #mainSubject
   ?article wdt:P50 ?author. #author
   ?author ?p ?o.
   
   OPTIONAL{
       ?o sc:name ?oName.
       ?p sc:name ?pName.
    }
    #FILTER (?p not in (wdt:P2348, wdt:P17,wdt:P19,wdt:P20,wdt:P136,wdt:P2596,wdt:P27))
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q987744'), ('oName', 'Edward E. Smith Memorial Award'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received')]
[('o', 'http://www.wikidata.org/entity/Q2294364'), ('oName', 'Slobodan Ćurčić'), ('p', 'http://www.wikidata.org/prop/direct/P184'), ('pName', 'doctoral advisor')]
[('o', 'http://www.wikidata.org/entity/Q1499767'), ('oName', 'Jean-Baptiste'), ('p', 'http://www.wikidata.org/prop/direct/P735'), ('pName', 'given name')]
[('o', 'http://www.wikidata.org/entity/Q15501913'), ('oName', 'Nikola'), ('p', 'http://www.wikidata.org/prop/direct/P735'), ('pName', 'given name')]
[('o', 'http://www.wikidata.org/entity/Q15983985'), ('oName', 'classical archaeologist'), ('p', 'http://www.wikidata.org/prop/direct/P106'), ('pName', 'occupation')]
[('o', 'http://www.wikidata.org/entity/Q161976'), ('oName', 'University of Würzburg'), ('p', 'http://www.wikidata.org/prop/direct/P108'), ('pName', 'employer')]
[('o', 'http://www.wiki

In [113]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist')

# main subject --> author --> is an archaeologist?
queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilizationName ?articleName ?authorName ?occupation ?occupationName #?p ?pName
WHERE {
   # bind something
   #VALUES(?civilization){
   #    (wd:Q11768)#('Ancient Egypt')
   #}
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
                 
   ?article wdt:P921 ?civilization. #mainSubject
   ?article wdt:P50 ?author. #author
   ?author wdt:P106 ?occupation. #occupation
   
   VALUES (?occupation){
       (wd:Q3621491)#, ('oName', 'archaeologist')]
       (wd:Q15983985)#, ('oName', 'classical archaeologist')]
   }

   
   OPTIONAL{
       ?occupation sc:name ?occupationName.
       ?author sc:name ?authorName.
       #?p sc:name ?pName.
    }
    #FILTER (?p not in (wdt:P2348, wdt:P17,wdt:P19,wdt:P20,wdt:P136,wdt:P2596,wdt:P27))
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilizationName', 'Thrace'), ('authorName', 'Bernhard Weisser'), ('occupation', 'http://www.wikidata.org/entity/Q15983985'), ('occupationName', 'classical archaeologist')]
[('civilizationName', 'Gaul'), ('authorName', 'Peter Guest'), ('occupation', 'http://www.wikidata.org/entity/Q15983985'), ('occupationName', 'classical archaeologist')]
[('civilizationName', 'Gaul'), ('authorName', 'Nico Roymans'), ('occupation', 'http://www.wikidata.org/entity/Q15983985'), ('occupationName', 'classical archaeologist')]
[('civilizationName', 'Dacia'), ('authorName', 'William Hanson'), ('occupation', 'http://www.wikidata.org/entity/Q15983985'), ('occupationName', 'classical archaeologist')]
[('civilizationName', 'Gaul'), ('authorName', 'Greg Woolf'), ('occupation', 'http://www.wikidata.org/entity/Q15983985'), ('occupationName', 'classical archaeologist')]
[('civilizationName', 'Gaul'), ('authorName', 'Simon Esmonde Cleary'), ('occupation', 'http://www.wikidata.org/entity/Q15983985'), ('occ

##### Attention we don't have only wd:Q3621491 ('archaeologist') but also sub-categories

In [114]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
    wd:Q15983985 ?p ?o.   
   OPTIONAL{
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q2468727'), ('oName', 'classical scholar')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q3621491'), ('oName', 'archaeologist')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q28640'), ('oName', 'profession')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q1047113'), ('oName', 'specialty')]
[('p', 'http://www.wikidata.org/prop/direct/P425'), ('pName', 'field of this occupation'), ('o', 'http://www.wikidata.org/entity/Q1669589'), ('oName', 'classical archaeology')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://schema.org/name'), ('o', 'classical archaeologist')]
7


In [115]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
    ?o ?p wd:Q3621491 .   
   OPTIONAL{
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q364239'), ('oName', 'William Camden')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q9176125'), ('oName', 'Bohdan Janusz')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q102139'), ('oName', 'Margrethe II of Denmark')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q20609875'), ('oName', 'Numan Negmatov')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q1677608'), ('oName', 'Jacob Pinkerfield')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work'), ('o', 'http://www.wikidata.org/entity/Q1821977'), ('oName', 'Lev Perovski')]
[('p', 'http://www.wikidata.or

In [117]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
    ?o ?p wd:Q3621491 .   
   OPTIONAL{
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }
    FILTER (?p not in (wdt:P101, wdt:P106))
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q47482514'), ('oName', 'archaeobotanist')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q51034797'), ('oName', 'computational archaeologist')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q15983985'), ('oName', 'classical archaeologist')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q17488316'), ('oName', 'prehistorian')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q4529715'), ('oName', 'Hittitologist')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q26224000'), ('oName', 'castellologist')]
[('p', 'http://www.wikida

In [3]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# query of the previous day

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
    ?o ?p wd:Q3621491 .   
   OPTIONAL{
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }
    FILTER (?p not in (wdt:P101, wdt:P106))
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q47482514'), ('oName', 'archaeobotanist')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q51034797'), ('oName', 'computational archaeologist')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q15983985'), ('oName', 'classical archaeologist')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q17488316'), ('oName', 'prehistorian')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q4529715'), ('oName', 'Hittitologist')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q26224000'), ('oName', 'castellologist')]
[('p', 'http://www.wikida

In [4]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
    ?o ?p wd:Q3621491 .   
   OPTIONAL{
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }
    #P279'), ('pName', 'subclass of'
    FILTER (?p in (wdt:P279))
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q5177606'), ('oName', 'County Archaeologist')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q26132815'), ('oName', 'byzantinist')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q22917056'), ('oName', 'engraved gem researcher')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q17488316'), ('oName', 'prehistorian')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q54862887'), ('oName', 'archaeological monument keeper')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q107737256'), ('oName', 'historian of archaeology')]
[('p', '

In [6]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?specializedArchaeologist ?specializedArchaeologistName
WHERE {
    ?specializedArchaeologist wdt:P279 wd:Q3621491 .   
    #OPTIONAL{
       ?specializedArchaeologist sc:name ?specializedArchaeologistName.
    #}
    #P279'), ('pName', 'subclass of'
    #FILTER (?p in (wdt:P279))
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('specializedArchaeologist', 'http://www.wikidata.org/entity/Q47482514'), ('specializedArchaeologistName', 'archaeobotanist')]
[('specializedArchaeologist', 'http://www.wikidata.org/entity/Q51034797'), ('specializedArchaeologistName', 'computational archaeologist')]
[('specializedArchaeologist', 'http://www.wikidata.org/entity/Q15983985'), ('specializedArchaeologistName', 'classical archaeologist')]
[('specializedArchaeologist', 'http://www.wikidata.org/entity/Q17488316'), ('specializedArchaeologistName', 'prehistorian')]
[('specializedArchaeologist', 'http://www.wikidata.org/entity/Q4529715'), ('specializedArchaeologistName', 'Hittitologist')]
[('specializedArchaeologist', 'http://www.wikidata.org/entity/Q26224000'), ('specializedArchaeologistName', 'castellologist')]
[('specializedArchaeologist', 'http://www.wikidata.org/entity/Q1350189'), ('specializedArchaeologistName', 'egyptologist')]
[('specializedArchaeologist', 'http://www.wikidata.org/entity/Q26132815'), ('specialize

#### Partial solution: getting all the articles written by an archaeologist or any subclass of archaeologist

In [7]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilizationName ?articleName ?authorName ?occupation ?occupationName #?p ?pName
WHERE {
   # bind something
   #VALUES(?civilization){
   #    (wd:Q11768)#('Ancient Egypt')
   #}
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
                 
   ?article wdt:P921 ?civilization. #mainSubject
   ?article wdt:P50 ?author. #author
   ?author wdt:P106 ?occupation. #occupation
   ?specializedArchaeologist wdt:P279 wd:Q3621491 .
   
   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
   #VALUES (?occupation){
   #    (wd:Q3621491)#, ('oName', 'archaeologist')]
   #    (wd:Q15983985)#, ('oName', 'classical archaeologist')]
   #}

   
   OPTIONAL{
       ?occupation sc:name ?occupationName.
       ?author sc:name ?authorName.
       #?p sc:name ?pName.
    }
    #FILTER (?p not in (wdt:P2348, wdt:P17,wdt:P19,wdt:P20,wdt:P136,wdt:P2596,wdt:P27))
}

LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilizationName', 'Thrace'), ('authorName', 'Bernhard Weisser'), ('occupation', 'http://www.wikidata.org/entity/Q15983985'), ('occupationName', 'classical archaeologist')]
[('civilizationName', 'Gaul'), ('authorName', 'Peter Guest'), ('occupation', 'http://www.wikidata.org/entity/Q15983985'), ('occupationName', 'classical archaeologist')]
[('civilizationName', 'Gaul'), ('authorName', 'Nico Roymans'), ('occupation', 'http://www.wikidata.org/entity/Q15983985'), ('occupationName', 'classical archaeologist')]
[('civilizationName', 'Dacia'), ('authorName', 'William Hanson'), ('occupation', 'http://www.wikidata.org/entity/Q15983985'), ('occupationName', 'classical archaeologist')]
[('civilizationName', 'Gaul'), ('authorName', 'Greg Woolf'), ('occupation', 'http://www.wikidata.org/entity/Q15983985'), ('occupationName', 'classical archaeologist')]
[('civilizationName', 'Gaul'), ('authorName', 'Simon Esmonde Cleary'), ('occupation', 'http://www.wikidata.org/entity/Q15983985'), ('occ

In [8]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName (COUNT(*) as ?archaeologist) 
WHERE {
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
                 
   ?article wdt:P921 ?civilization. #mainSubject
   ?article wdt:P50 ?author. #author
   ?author wdt:P106 ?occupation. #occupation
   ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Archaeologist
   
   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))

   OPTIONAL{
       ?occupation sc:name ?occupationName.
       ?author sc:name ?authorName.
    }
}

GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q11767'), ('civilizationName', 'Mesopotamia'), ('archaeologist', '33')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '133')]
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '34')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '204')]
[('civilization', 'http://www.wikidata.org/entity/Q41741'), ('civilizationName', 'Thrace'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q38060'), ('civilizationName', 'Gaul'), ('archaeologist', '467')]
[('civilization', 'http://www.wikidata.org/entity/Q134178'), ('civilizationName', 'Minoan civilization'), ('archaeologist', '66')]
[('civiliza

#### Putting together field of work with authors of articles

In [12]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# PUTTING together 'field of work' and 'main subject'

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName (COUNT(*) as ?archaeologist) 
WHERE {
   # bind something
   #VALUES(?civilization){
   #    (wd:Q11768)#('Ancient Egypt')
   #}
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
   ?student wdt:P101 ?civilization; #'field of work'
            sc:name ?studentName;
            wdt:P106 ?occupation. #occupation
   
   ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Archaeologist  ('Q3621491')   
   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))

}
GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q193748'), ('civilizationName', 'history of ancient Israel and Judah'), ('archaeologist', '33')]
[('civilization', 'http://www.wikidata.org/entity/Q11767'), ('civilizationName', 'Mesopotamia'), ('archaeologist', '33')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '236')]
[('civilization', 'http://www.wikidata.org/entity/Q1551054'), ('civilizationName', 'Israelite period'), ('archaeologist', '67')]
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '231')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q41741'), ('civilizationName', 'Thrace'), ('archaeologist', '33')]
[('civilization', 'http://www.wikidata.org/entity/Q38060'), ('civilizationName', 'Gaul'), ('archae

In [13]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# PUTTING together 'field of work' and 'main subject' 
# trying to add more student with 'field of work'

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName (COUNT(*) as ?archaeologist) 
WHERE {
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
                 
   #finding Archaeologist who has worked on ("field of work") the civilization
   ?student wdt:P101 ?civilization; #'field of work'
            sc:name ?studentName;
            wdt:P106 ?occupation1. #P106 'occupation'-> Q3621491 ('archaeologist)
            
   #finding Archaeologist who wrote some article on the civilization
   ?article wdt:P921 ?civilization. #mainSubject
   ?article wdt:P50 ?author. #author
   ?author wdt:P106 ?occupation. #occupation
   
   ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Archaeologist
   
   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
   FILTER (?occupation1 in (?specializedArchaeologist, wd:Q3621491))

   OPTIONAL{
       ?occupation sc:name ?occupationName.
       ?author sc:name ?authorName.
    }
}

GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q11767'), ('civilizationName', 'Mesopotamia'), ('archaeologist', '33')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '951')]
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '238')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '24')]
[('civilization', 'http://www.wikidata.org/entity/Q41741'), ('civilizationName', 'Thrace'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q38060'), ('civilizationName', 'Gaul'), ('archaeologist', '1401')]
[('civilization', 'http://www.wikidata.org/entity/Q134178'), ('civilizationName', 'Minoan civilization'), ('archaeologist', '6')]
[('civilization', 'http://www.wikidata.org/entity/Q28567'), ('civilizationName', 'Maya civilization'), ('archaeologist', '231')

In [15]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# PUTTING together 'field of work' and 'main subject' 
# trying to add more student with 'field of work'
# ? DUPLICATES

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?student ?studentName ?civilization ?civilizationName #?civilization ?civilizationName (COUNT(*) as ?archaeologist) 
WHERE {
   #finding Archaeologist who has worked on ("field of work") the civilization
   {
       SELECT DISTINCT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
           
           ?student wdt:P101 ?civilization; #'field of work'
                sc:name ?studentName;
                wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 
                
                
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
            
   #finding Archaeologist who wrote some article on the civilization
   {
       SELECT DISTINCT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
   
           ?article wdt:P921 ?civilization. #mainSubject
           ?article wdt:P50 ?student. #author
           ?student wdt:P106 ?occupation; #occupation  
                    sc:name ?studentName.    
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
}
#GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [16]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# PUTTING together 'field of work' and 'main subject' 
# trying to add more student with 'field of work'
# ? DUPLICATES

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT * #?student ?studentName ?civilization ?civilizationName #?civilization ?civilizationName (COUNT(*) as ?archaeologist) 
WHERE {
   #finding Archaeologist who has worked on ("field of work") the civilization
   {
       SELECT DISTINCT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
           
           ?student wdt:P101 ?civilization; #'field of work'
                sc:name ?studentName;
                wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 
                
                
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
            
   #finding Archaeologist who wrote some article on the civilization
   {
       SELECT DISTINCT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
   
           ?article wdt:P921 ?civilization. #mainSubject
           ?article wdt:P50 ?student. #author
           ?student wdt:P106 ?occupation; #occupation  
                    sc:name ?studentName.    
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
}
#GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [17]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# PUTTING together 'field of work' and 'main subject' 
# trying to add more student with 'field of work'
# ? DUPLICATES

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT * #?student ?studentName ?civilization ?civilizationName #?civilization ?civilizationName (COUNT(*) as ?archaeologist) 
WHERE {
   #finding Archaeologist who has worked on ("field of work") the civilization
   {
       SELECT DISTINCT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
           
           ?student wdt:P101 ?civilization; #'field of work'
                sc:name ?studentName;
                wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 
                
                
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
        UNION    
   #finding Archaeologist who wrote some article on the civilization
   {
       SELECT DISTINCT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
   
           ?article wdt:P921 ?civilization. #mainSubject
           ?article wdt:P50 ?student. #author
           ?student wdt:P106 ?occupation; #occupation  
                    sc:name ?studentName.    
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
}
#GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('student', 'http://www.wikidata.org/entity/Q2342814'), ('studentName', 'Alexandre Bertrand'), ('civilization', 'http://www.wikidata.org/entity/Q38060'), ('civilizationName', 'Gaul')]
[('student', 'http://www.wikidata.org/entity/Q1985476'), ('studentName', 'Charles Thomas Newton'), ('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece')]
[('student', 'http://www.wikidata.org/entity/Q21032203'), ('studentName', 'Aenne Ohnesorg'), ('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece')]
[('student', 'http://www.wikidata.org/entity/Q1523942'), ('studentName', 'Gilbert Charles-Picard'), ('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage')]
[('student', 'http://www.wikidata.org/entity/Q64563606'), ('studentName', "Vincent Robin d'Arba Desborough"), ('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece')]
[('st

In [18]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# PUTTING together 'field of work' and 'main subject' 
# trying to add more student with 'field of work'
# ? DUPLICATES

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?student ?studentName ?civilization ?civilizationName #?civilization ?civilizationName (COUNT(*) as ?archaeologist) 
WHERE {
   #finding Archaeologist who has worked on ("field of work") the civilization
   {
       SELECT DISTINCT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
           
           ?student wdt:P101 ?civilization; #'field of work'
                sc:name ?studentName;
                wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 
                
                
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
        UNION    
   #finding Archaeologist who wrote some article on the civilization
   {
       SELECT DISTINCT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
   
           ?article wdt:P921 ?civilization. #mainSubject
           ?article wdt:P50 ?student. #author
           ?student wdt:P106 ?occupation; #occupation  
                    sc:name ?studentName.    
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
}
#GROUP BY ?civilization ?civilizationName
ORDER BY ?studentName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('student', 'http://www.wikidata.org/entity/Q101163496'), ('studentName', 'Aaron A. Burke'), ('civilization', 'http://www.wikidata.org/entity/Q1551054'), ('civilizationName', 'Israelite period')]
[('student', 'http://www.wikidata.org/entity/Q21032203'), ('studentName', 'Aenne Ohnesorg'), ('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece')]
[('student', 'http://www.wikidata.org/entity/Q403881'), ('studentName', 'Aidan Dodson'), ('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt')]
[('student', 'http://www.wikidata.org/entity/Q346765'), ('studentName', 'Albert Grenier'), ('civilization', 'http://www.wikidata.org/entity/Q38060'), ('civilizationName', 'Gaul')]
[('student', 'http://www.wikidata.org/entity/Q98969651'), ('studentName', 'Alexander Nagel'), ('civilization', 'http://www.wikidata.org/entity/Q11767'), ('civilizationName', 'Mesopotamia')]
[('student', 'http://www.wikidata.org/entity/Q

In [19]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# PUTTING together 'field of work' and 'main subject' 
# trying to add more student with 'field of work'
# ? DUPLICATES

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?student ?studentName ?civilization ?civilizationName (COUNT(*) AS ?tot) #(COUNT(*) as ?archaeologist) 
WHERE {
   #finding Archaeologist who has worked on ("field of work") the civilization
   {
       SELECT DISTINCT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
           
           ?student wdt:P101 ?civilization; #'field of work'
                sc:name ?studentName;
                wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 
                
                
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
        UNION    
   #finding Archaeologist who wrote some article on the civilization
   {
       SELECT DISTINCT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
   
           ?article wdt:P921 ?civilization. #mainSubject
           ?article wdt:P50 ?student. #author
           ?student wdt:P106 ?occupation; #occupation  
                    sc:name ?studentName.    
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
}
#GROUP BY ?civilization ?civilizationName
ORDER BY DESC(?tot)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('student', 'http://www.wikidata.org/entity/Q1985476'), ('studentName', 'Charles Thomas Newton'), ('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('tot', '1')]
[('student', 'http://www.wikidata.org/entity/Q12228275'), ('studentName', 'Ammar Mahjoubi'), ('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('tot', '1')]
[('student', 'http://www.wikidata.org/entity/Q113937'), ('studentName', 'Fritz Schachermeyr'), ('civilization', 'http://www.wikidata.org/entity/Q134178'), ('civilizationName', 'Minoan civilization'), ('tot', '1')]
[('student', 'http://www.wikidata.org/entity/Q3575342'), ('studentName', 'Zeineb Benzina'), ('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('tot', '1')]
[('student', 'http://www.wikidata.org/entity/Q240788'), ('studentName', 'Edme-François Jomard'), ('civilization', 'http://www.wikidata.org/entity/Q

In [20]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# PUTTING together 'field of work' and 'main subject' 
# trying to add more student with 'field of work'
# ? DUPLICATES

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?civilization ?civilizationName (COUNT(*) as ?archaeologist) 
WHERE {
   #finding Archaeologist who has worked on ("field of work") the civilization
   {
       SELECT DISTINCT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
           
           ?student wdt:P101 ?civilization; #'field of work'
                sc:name ?studentName;
                wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 
                
                
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
        UNION    
   #finding Archaeologist who wrote some article on the civilization
   {
       SELECT DISTINCT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
   
           ?article wdt:P921 ?civilization. #mainSubject
           ?article wdt:P50 ?student. #author
           ?student wdt:P106 ?occupation; #occupation  
                    sc:name ?studentName.    
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
}
GROUP BY ?civilization ?civilizationName
ORDER BY DESC(?archaeologist)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q38060'), ('civilizationName', 'Gaul'), ('archaeologist', '19')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '12')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '10')]
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '8')]
[('civilization', 'http://www.wikidata.org/entity/Q28567'), ('civilizationName', 'Maya civilization'), ('archaeologist', '7')]
[('civilization', 'http://www.wikidata.org/entity/Q134178'), ('civilizationName', 'Minoan civilization'), ('archaeologist', '3')]
[('civilization', 'http://www.wikidata.org/entity/Q11767'), ('civilizationName', 'Mesopotamia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '2')]
[('civ

In [21]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# PUTTING together 'field of work' and 'main subject' 
# trying to add more student with 'field of work'
# ? DUPLICATES

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?civilization ?civilizationName (COUNT(*) as ?archaeologist) 
WHERE {
   #finding Archaeologist who has worked on ("field of work") the civilization
   {
       SELECT DISTINCT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
           
           ?student wdt:P101 ?civilization; #'field of work'
                sc:name ?studentName;
                wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 
                
                
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
        UNION    
   #finding Archaeologist who wrote some article on the civilization
   {
       SELECT DISTINCT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
   
           ?article wdt:P921 ?civilization. #mainSubject
           ?article wdt:P50 ?student. #author
           ?student wdt:P106 ?occupation; #occupation  
                    sc:name ?studentName.    
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
}
GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q11767'), ('civilizationName', 'Mesopotamia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q193748'), ('civilizationName', 'history of ancient Israel and Judah'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '12')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q1551054'), ('civilizationName', 'Israelite period'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '8')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '10')]
[('civilization', 'http://www.wikidata.org/entity/Q41741'), ('civilizationName', 'Thrace'), ('archaeolog

In [22]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# PUTTING together 'field of work' and 'main subject' 
# trying to add more student with 'field of work'
# ? DUPLICATES

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?civilization ?civilizationName (COUNT(*) as ?archaeologist) 
WHERE {
   #finding Archaeologist who has worked on ("field of work") the civilization
   {
       SELECT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
           
           ?student wdt:P101 ?civilization; #'field of work'
                sc:name ?studentName;
                wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 
                
                
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
        UNION    
   #finding Archaeologist who wrote some article on the civilization
   {
       SELECT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
   
           ?article wdt:P921 ?civilization. #mainSubject
           ?article wdt:P50 ?student. #author
           ?student wdt:P106 ?occupation; #occupation  
                    sc:name ?studentName.    
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
}
GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q193748'), ('civilizationName', 'history of ancient Israel and Judah'), ('archaeologist', '33')]
[('civilization', 'http://www.wikidata.org/entity/Q11767'), ('civilizationName', 'Mesopotamia'), ('archaeologist', '66')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '369')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q1551054'), ('civilizationName', 'Israelite period'), ('archaeologist', '67')]
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '265')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '206')]
[('civilization', 'http://www.wikidata.org/entity/Q41741'), ('civilizationName', 'Thrace'), ('arc

In [3]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# PUTTING together 'field of work' and 'main subject' 
# trying to add more student with 'field of work'
# ? DUPLICATES

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?civilizationName ?studentName ?student #?civilization ?civilizationName (COUNT(*) as ?archaeologist) 
WHERE {
   #finding Archaeologist who has worked on ("field of work") the civilization
   {
       SELECT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
           
           ?student wdt:P101 ?civilization; #'field of work'
                sc:name ?studentName;
                wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 
                
                
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
        UNION    
   #finding Archaeologist who wrote some article on the civilization
   {
       SELECT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
   
           ?article wdt:P921 ?civilization. #mainSubject
           ?article wdt:P50 ?student. #author
           ?student wdt:P106 ?occupation; #occupation  
                    sc:name ?studentName.    
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
}
#GROUP BY ?civilization ?civilizationName
ORDER BY ?studentName ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilizationName', 'Israelite period'), ('studentName', 'Aaron A. Burke'), ('student', 'http://www.wikidata.org/entity/Q101163496')]
[('civilizationName', 'Israelite period'), ('studentName', 'Aaron A. Burke'), ('student', 'http://www.wikidata.org/entity/Q101163496')]
[('civilizationName', 'Israelite period'), ('studentName', 'Aaron A. Burke'), ('student', 'http://www.wikidata.org/entity/Q101163496')]
[('civilizationName', 'Israelite period'), ('studentName', 'Aaron A. Burke'), ('student', 'http://www.wikidata.org/entity/Q101163496')]
[('civilizationName', 'Israelite period'), ('studentName', 'Aaron A. Burke'), ('student', 'http://www.wikidata.org/entity/Q101163496')]
[('civilizationName', 'Israelite period'), ('studentName', 'Aaron A. Burke'), ('student', 'http://www.wikidata.org/entity/Q101163496')]
[('civilizationName', 'Israelite period'), ('studentName', 'Aaron A. Burke'), ('student', 'http://www.wikidata.org/entity/Q101163496')]
[('civilizationName', 'Israelite period'

In [4]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# PUTTING together 'field of work' and 'main subject' 
# trying to add more student with 'field of work'
# ? DUPLICATES

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilizationName ?studentName ?student #?civilization ?civilizationName (COUNT(*) as ?archaeologist) 
WHERE {
   #finding Archaeologist who has worked on ("field of work") the civilization
   {
       SELECT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
           
           ?student wdt:P101 ?civilization; #'field of work'
                sc:name ?studentName;
                wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 
                
                
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
        UNION    
   #finding Archaeologist who wrote some article on the civilization
   {
       SELECT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
   
           ?article wdt:P921 ?civilization. #mainSubject
           ?article wdt:P50 ?student. #author
           ?student wdt:P106 ?occupation; #occupation  
                    sc:name ?studentName.    
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
}
#GROUP BY ?civilization ?civilizationName
ORDER BY ?studentName ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilizationName', 'Israelite period'), ('studentName', 'Aaron A. Burke'), ('student', 'http://www.wikidata.org/entity/Q101163496')]
[('civilizationName', 'Ancient Greece'), ('studentName', 'Aenne Ohnesorg'), ('student', 'http://www.wikidata.org/entity/Q21032203')]
[('civilizationName', 'Ancient Egypt'), ('studentName', 'Aidan Dodson'), ('student', 'http://www.wikidata.org/entity/Q403881')]
[('civilizationName', 'Gaul'), ('studentName', 'Albert Grenier'), ('student', 'http://www.wikidata.org/entity/Q346765')]
[('civilizationName', 'Mesopotamia'), ('studentName', 'Alexander Nagel'), ('student', 'http://www.wikidata.org/entity/Q98969651')]
[('civilizationName', 'Gaul'), ('studentName', 'Alexandre Bertrand'), ('student', 'http://www.wikidata.org/entity/Q2342814')]
[('civilizationName', 'Maya civilization'), ('studentName', 'Alfonso Lacadena García-Gallo'), ('student', 'http://www.wikidata.org/entity/Q48306177')]
[('civilizationName', 'Ancient Carthage'), ('studentName', 'Ammar 

In [5]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# PUTTING together 'field of work' and 'main subject' 
# trying to add more student with 'field of work'
# ? DUPLICATES

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName ?studentName ?student 
WHERE {
   #finding Archaeologist who has worked on ("field of work") the civilization
   {
       SELECT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
           
           ?student wdt:P101 ?civilization; #'field of work'
                sc:name ?studentName;
                wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 
                
                
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
        UNION    
   #finding Archaeologist who wrote some article on the civilization
   {
       SELECT ?student ?studentName ?civilization ?civilizationName
       WHERE{
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.
           ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')
   
           ?article wdt:P921 ?civilization. #mainSubject
           ?article wdt:P50 ?student. #author
           ?student wdt:P106 ?occupation; #occupation  
                    sc:name ?studentName.    
           FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
        }
    }
}
#GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q38060'), ('civilizationName', 'Gaul'), ('studentName', 'Alexandre Bertrand'), ('student', 'http://www.wikidata.org/entity/Q2342814')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('studentName', 'Charles Thomas Newton'), ('student', 'http://www.wikidata.org/entity/Q1985476')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('studentName', 'Aenne Ohnesorg'), ('student', 'http://www.wikidata.org/entity/Q21032203')]
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('studentName', 'Gilbert Charles-Picard'), ('student', 'http://www.wikidata.org/entity/Q1523942')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('studentName', "Vincent Robin d'Arba Desborough"), ('student', 'http://www.wikidata.org/entity/Q64563606')]
[('ci

#### Partial combined sol

In [6]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# PUTTING together 'field of work' and 'main subject' 
# trying to add more student with 'field of work'
# DUPLICATES --> REMOVED using another inner query === same results DISTINCS inside each ! ---> look above, 5th cel from here

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?civilization ?civilizationName (COUNT(*) as ?archaeologist) WHERE {
    SELECT DISTINCT ?civilization ?civilizationName ?studentName ?student 
    WHERE {
       #finding Archaeologist who has worked on ("field of work") the civilization
       {
           SELECT ?student ?studentName ?civilization ?civilizationName
           WHERE{
               ?civilization wdt:P31 wd:Q28171280;
                             sc:name ?civilizationName.
               ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')

               ?student wdt:P101 ?civilization; #'field of work'
                    sc:name ?studentName;
                    wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 


               FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
            }
        }
            UNION    
       #finding Archaeologist who wrote some article on the civilization
       {
           SELECT ?student ?studentName ?civilization ?civilizationName
           WHERE{
               ?civilization wdt:P31 wd:Q28171280;
                             sc:name ?civilizationName.
               ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')

               ?article wdt:P921 ?civilization. #mainSubject
               ?article wdt:P50 ?student. #author
               ?student wdt:P106 ?occupation; #occupation  
                        sc:name ?studentName.    
               FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
            }
        }
    }
}
GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q11767'), ('civilizationName', 'Mesopotamia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q193748'), ('civilizationName', 'history of ancient Israel and Judah'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '12')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q1551054'), ('civilizationName', 'Israelite period'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '8')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '10')]
[('civilization', 'http://www.wikidata.org/entity/Q41741'), ('civilizationName', 'Thrace'), ('archaeolog

#### WITH ONLY Field of work we have less matchings, TODO: check if you can go further in subclasses of archaeologis

In [7]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?civilization ?civilizationName (COUNT(*) as ?archaeologist) WHERE {
    SELECT DISTINCT ?civilization ?civilizationName ?studentName ?student 
    WHERE {
       #finding Archaeologist who has worked on ("field of work") the civilization
       {
           SELECT ?student ?studentName ?civilization ?civilizationName
           WHERE{
               ?civilization wdt:P31 wd:Q28171280;
                             sc:name ?civilizationName.
               ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')

               ?student wdt:P101 ?civilization; #'field of work'
                    sc:name ?studentName;
                    wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 


               FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
            }
        }
    }
}
GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q11767'), ('civilizationName', 'Mesopotamia'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q193748'), ('civilizationName', 'history of ancient Israel and Judah'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '9')]
[('civilization', 'http://www.wikidata.org/entity/Q1551054'), ('civilizationName', 'Israelite period'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '7')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q41741'), ('civilizationName', 'Thrace'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q38060'), ('civilizationName', 'Gaul'), ('archaeologist'

#### FINDING civilization without any archaeologist

In [9]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?civilization ?civilizationName (COUNT(*) as ?archaeologist) WHERE 
{
    {
        ?civilizationOuter wdt:P31 wd:Q28171280;
                sc:name ?civilizationOuterName.
    }
    UNION
    {
        SELECT DISTINCT ?civilization ?civilizationName ?studentName ?student 
        WHERE {
           #finding Archaeologist who has worked on ("field of work") the civilization
           {
               SELECT ?student ?studentName ?civilization ?civilizationName
               WHERE{
                   ?civilization wdt:P31 wd:Q28171280;
                                 sc:name ?civilizationName.
                   ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')

                   ?student wdt:P101 ?civilization; #'field of work'
                        sc:name ?studentName;
                        wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 


                   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
               }
           }
           UNION    
           #finding Archaeologist who wrote some article on the civilization
           {
               SELECT ?student ?studentName ?civilization ?civilizationName
               WHERE{
                   ?civilization wdt:P31 wd:Q28171280;
                                 sc:name ?civilizationName.
                   ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')

                   ?article wdt:P921 ?civilization. #mainSubject
                   ?article wdt:P50 ?student. #author
                   ?student wdt:P106 ?occupation; #occupation  
                            sc:name ?studentName.    
                   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
                }
            }
        }
    }
}
GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('archaeologist', '25')]
[('civilization', 'http://www.wikidata.org/entity/Q134178'), ('civilizationName', 'Minoan civilization'), ('archaeologist', '3')]
[('civilization', 'http://www.wikidata.org/entity/Q11767'), ('civilizationName', 'Mesopotamia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q32047'), ('civilizationName', 'Illyria'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q41741'), ('civilizationName', 'Thrace'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q38060'), ('civilizationName', 'Gaul'), ('archaeologist', '19')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '10')]
[('civilization', 'http://www.wikidata.org/entity/Q1551054'), ('civilizationName', 'Israelite period'), ('archaeologist', '2')]

In [10]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?civilization ?civilizationName (COUNT(*) as ?archaeologist) WHERE 
{
    {
        ?civilization wdt:P31 wd:Q28171280;
                sc:name ?civilizationName.
    }
    UNION
    {
        SELECT DISTINCT ?civilization ?civilizationName ?studentName ?student 
        WHERE {
           #finding Archaeologist who has worked on ("field of work") the civilization
           {
               SELECT ?student ?studentName ?civilization ?civilizationName
               WHERE{
                   ?civilization wdt:P31 wd:Q28171280;
                                 sc:name ?civilizationName.
                   ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')

                   ?student wdt:P101 ?civilization; #'field of work'
                        sc:name ?studentName;
                        wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 


                   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
               }
           }
           UNION    
           #finding Archaeologist who wrote some article on the civilization
           {
               SELECT ?student ?studentName ?civilization ?civilizationName
               WHERE{
                   ?civilization wdt:P31 wd:Q28171280;
                                 sc:name ?civilizationName.
                   ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')

                   ?article wdt:P921 ?civilization. #mainSubject
                   ?article wdt:P50 ?student. #author
                   ?student wdt:P106 ?occupation; #occupation  
                            sc:name ?studentName.    
                   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
                }
            }
        }
    }
}
GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q193748'), ('civilizationName', 'history of ancient Israel and Judah'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q11767'), ('civilizationName', 'Mesopotamia'), ('archaeologist', '3')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '3')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '13')]
[('civilization', 'http://www.wikidata.org/entity/Q1364601'), ('civilizationName', 'Gallo-Roman culture'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q768212'), ('civilizationName', 'Turco-Mongol'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q28405'), ('civilizationName', 'Gojoseon'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q1551054'), ('civilizationName', 'Israelite period'), ('archae

In [11]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# we have an additional count 

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName (COUNT(?studentName) as ?archaeologist) WHERE 
{
    {
        ?civilization wdt:P31 wd:Q28171280;
                sc:name ?civilizationName.
    }
    UNION
    {
        SELECT DISTINCT ?civilization ?civilizationName ?studentName ?student 
        WHERE {
           #finding Archaeologist who has worked on ("field of work") the civilization
           {
               SELECT ?student ?studentName ?civilization ?civilizationName
               WHERE{
                   ?civilization wdt:P31 wd:Q28171280;
                                 sc:name ?civilizationName.
                   ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')

                   ?student wdt:P101 ?civilization; #'field of work'
                        sc:name ?studentName;
                        wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 


                   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
               }
           }
           UNION    
           #finding Archaeologist who wrote some article on the civilization
           {
               SELECT ?student ?studentName ?civilization ?civilizationName
               WHERE{
                   ?civilization wdt:P31 wd:Q28171280;
                                 sc:name ?civilizationName.
                   ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')

                   ?article wdt:P921 ?civilization. #mainSubject
                   ?article wdt:P50 ?student. #author
                   ?student wdt:P106 ?occupation; #occupation  
                            sc:name ?studentName.    
                   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
                }
            }
        }
    }
}
GROUP BY ?civilization ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q193748'), ('civilizationName', 'history of ancient Israel and Judah'), ('archaeologist', '1')]
[('civilization', 'http://www.wikidata.org/entity/Q11767'), ('civilizationName', 'Mesopotamia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '12')]
[('civilization', 'http://www.wikidata.org/entity/Q1364601'), ('civilizationName', 'Gallo-Roman culture'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q768212'), ('civilizationName', 'Turco-Mongol'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q28405'), ('civilizationName', 'Gojoseon'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q1551054'), ('civilizationName', 'Israelite period'), ('archae

In [12]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName (COUNT(?studentName) as ?archaeologist) WHERE 
{
    {
        ?civilization wdt:P31 wd:Q28171280;
                sc:name ?civilizationName.
    }
    UNION
    {
        SELECT DISTINCT ?civilization ?civilizationName ?studentName ?student 
        WHERE {
           #finding Archaeologist who has worked on ("field of work") the civilization
           {
               SELECT ?student ?studentName ?civilization ?civilizationName
               WHERE{
                   ?civilization wdt:P31 wd:Q28171280;
                                 sc:name ?civilizationName.
                   ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')

                   ?student wdt:P101 ?civilization; #'field of work'
                        sc:name ?studentName;
                        wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 


                   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
               }
           }
           UNION    
           #finding Archaeologist who wrote some article on the civilization
           {
               SELECT ?student ?studentName ?civilization ?civilizationName
               WHERE{
                   ?civilization wdt:P31 wd:Q28171280;
                                 sc:name ?civilizationName.
                   ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')

                   ?article wdt:P921 ?civilization. #mainSubject
                   ?article wdt:P50 ?student. #author
                   ?student wdt:P106 ?occupation; #occupation  
                            sc:name ?studentName.    
                   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
                }
            }
        }
    }
}
GROUP BY ?civilization ?civilizationName
ORDER BY ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '8')]
[('civilization', 'http://www.wikidata.org/entity/Q4752820'), ('civilizationName', 'Ancient Crete'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '10')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '12')]
[('civilization', 'http://www.wikidata.org/entity/Q3327860'), ('civilizationName', 'Celtic kingdom of Ireland'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q25558247'), ('civilizationName', 'Chinese civilization'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q128904'), ('civilizationName', 'Elam'), ('archaeologi

In [13]:
# For each ancient civilization find the number of archeologists who worked on it (return also the civilizations with no archeologists) 
# (the result set must be triples of civilization IRI, label and #archeologist).

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?civilization ?civilizationName WHERE 
{
    ?civilization wdt:P31 wd:Q28171280;
                    sc:name ?civilizationName.
}
ORDER BY ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage')]
[('civilization', 'http://www.wikidata.org/entity/Q4752820'), ('civilizationName', 'Ancient Crete')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece')]
[('civilization', 'http://www.wikidata.org/entity/Q3327860'), ('civilizationName', 'Celtic kingdom of Ireland')]
[('civilization', 'http://www.wikidata.org/entity/Q25558247'), ('civilizationName', 'Chinese civilization')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia')]
[('civilization', 'http://www.wikidata.org/entity/Q128904'), ('civilizationName', 'Elam')]
[('civilization', 'http://www.wikidata.org/entity/Q1364601'), ('civilizationName', 'Gallo-Roman culture')]
[('civilization', 'http://www.wikidata.org/entity/Q38060'), ('civilizationNa

In [8]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

# DISTINCT on studentName

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName (COUNT(DISTINCT ?studentName) as ?archaeologist) WHERE 
{
    # need for civilizations with no archaeologists
    {
        ?civilization wdt:P31 wd:Q28171280;
                sc:name ?civilizationName.
    }
    UNION
    {
        SELECT DISTINCT ?civilization ?civilizationName ?studentName ?student 
        WHERE {
           #finding Archaeologist who has worked on ("field of work") the civilization
           {
               SELECT ?student ?studentName ?civilization ?civilizationName
               WHERE{
                   ?civilization wdt:P31 wd:Q28171280;
                                 sc:name ?civilizationName.
                   ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')

                   ?student wdt:P101 ?civilization; #'field of work'
                        sc:name ?studentName;
                        wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 


                   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
               }
           }
           UNION    
           #finding Archaeologist who wrote some article on the civilization
           {
               SELECT ?student ?studentName ?civilization ?civilizationName
               WHERE{
                   ?civilization wdt:P31 wd:Q28171280;
                                 sc:name ?civilizationName.
                   ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')

                   ?article wdt:P921 ?civilization. #mainSubject
                   ?article wdt:P50 ?student. #author
                   ?student wdt:P106 ?occupation; #occupation  
                            sc:name ?studentName.    
                   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
                }
            }
        }
    }
}
GROUP BY ?civilization ?civilizationName
ORDER BY ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '8')]
[('civilization', 'http://www.wikidata.org/entity/Q4752820'), ('civilizationName', 'Ancient Crete'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '10')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '12')]
[('civilization', 'http://www.wikidata.org/entity/Q3327860'), ('civilizationName', 'Celtic kingdom of Ireland'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q25558247'), ('civilizationName', 'Chinese civilization'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q128904'), ('civilizationName', 'Elam'), ('archaeologi

#### FINDING ALTERNATIVE SOLUTION
##### USING path patterns

In [10]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName (COUNT(DISTINCT ?studentName) as ?archaeologist) WHERE 
{
    # need for civilizations with no archaeologists
    {
        ?civilization wdt:P31 wd:Q28171280;
                sc:name ?civilizationName.
    }
    UNION
    {
        SELECT DISTINCT ?civilization ?civilizationName ?studentName ?student 
        WHERE {
           #finding Archaeologist who has worked on ("field of work") the civilization
           {
               SELECT ?student ?studentName ?civilization ?civilizationName
               WHERE{
                   ?civilization wdt:P31 wd:Q28171280;
                                 sc:name ?civilizationName.
                                 
                   ?student wdt:P101 ?civilization; #'field of work'
                        sc:name ?studentName;
                        wdt:P106/wdt:P279* wd:Q3621491. #P106 'occupation'-> P279* 'sub Class' of Q3621491 ('archaeologist')
               }
           }
           UNION    
           #finding Archaeologist who wrote some article on the civilization
           {
               SELECT ?student ?studentName ?civilization ?civilizationName
               WHERE{
                   ?civilization wdt:P31 wd:Q28171280;
                                 sc:name ?civilizationName.

                   ?article wdt:P921 ?civilization. #mainSubject
                   ?article wdt:P50 ?student. #author
                   
                   ?student sc:name ?studentName;
                           wdt:P106/wdt:P279* wd:Q3621491. #P106 'occupation'-> P279* 'sub Class' of Q3621491 ('archaeologist'); #occupation  
                }
            }
        }
    }
}
GROUP BY ?civilization ?civilizationName
ORDER BY ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '8')]
[('civilization', 'http://www.wikidata.org/entity/Q4752820'), ('civilizationName', 'Ancient Crete'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '10')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '12')]
[('civilization', 'http://www.wikidata.org/entity/Q3327860'), ('civilizationName', 'Celtic kingdom of Ireland'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q25558247'), ('civilizationName', 'Chinese civilization'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q128904'), ('civilizationName', 'Elam'), ('archaeologi

##### SAME RESULT USING path patterns
##### Now trying to remove UNIONs

In [11]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName (COUNT(DISTINCT ?studentName) as ?archaeologist) WHERE 
{
    # need for civilizations with no archaeologists
    {
        ?civilization wdt:P31 wd:Q28171280;
                sc:name ?civilizationName.
    }
    UNION
    {
        SELECT DISTINCT ?civilization ?civilizationName ?studentName ?student 
        WHERE {
           ?civilization wdt:P31 wd:Q28171280;
                         sc:name ?civilizationName.

           ?student wdt:P101|^wdt:P50/wdt:P921 ?civilization; #'field of work' | #author/#mainSubject
                sc:name ?studentName;
                wdt:P106/wdt:P279* wd:Q3621491. #P106 'occupation'-> P279* 'sub Class' of Q3621491 ('archaeologist')
       }
    }
}
GROUP BY ?civilization ?civilizationName
ORDER BY ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '8')]
[('civilization', 'http://www.wikidata.org/entity/Q4752820'), ('civilizationName', 'Ancient Crete'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '10')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '12')]
[('civilization', 'http://www.wikidata.org/entity/Q3327860'), ('civilizationName', 'Celtic kingdom of Ireland'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q25558247'), ('civilizationName', 'Chinese civilization'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q128904'), ('civilizationName', 'Elam'), ('archaeologi

##### SAME RESULT. Removed inner UNION
##### Trying to remove outer UNION

In [12]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName (COUNT(DISTINCT ?studentName) as ?archaeologist) WHERE 
{
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
   OPTIONAL{
    ?student wdt:P101|^wdt:P50/wdt:P921 ?civilization; #'field of work' | #author/#mainSubject
        sc:name ?studentName;
        wdt:P106/wdt:P279* wd:Q3621491. #P106 'occupation'-> P279* 'sub Class' of Q3621491 ('archaeologist')
   }
}
GROUP BY ?civilization ?civilizationName
ORDER BY ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '8')]
[('civilization', 'http://www.wikidata.org/entity/Q4752820'), ('civilizationName', 'Ancient Crete'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '10')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '12')]
[('civilization', 'http://www.wikidata.org/entity/Q3327860'), ('civilizationName', 'Celtic kingdom of Ireland'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q25558247'), ('civilizationName', 'Chinese civilization'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q128904'), ('civilizationName', 'Elam'), ('archaeologi

##### SAME RESULT. Removed outer UNION

##### CHECKING, WITHOUT SUBCLASSES

In [13]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName (COUNT(DISTINCT ?studentName) as ?archaeologist) WHERE 
{
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
   OPTIONAL{
    ?student wdt:P101|^wdt:P50/wdt:P921 ?civilization; #'field of work' | #author/#mainSubject
        sc:name ?studentName;
        wdt:P106 wd:Q3621491. #P106 'occupation'-> P279* 'sub Class' of Q3621491 ('archaeologist')
   }
}
GROUP BY ?civilization ?civilizationName
ORDER BY ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '8')]
[('civilization', 'http://www.wikidata.org/entity/Q4752820'), ('civilizationName', 'Ancient Crete'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '6')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '10')]
[('civilization', 'http://www.wikidata.org/entity/Q3327860'), ('civilizationName', 'Celtic kingdom of Ireland'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q25558247'), ('civilizationName', 'Chinese civilization'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q128904'), ('civilizationName', 'Elam'), ('archaeologis

##### CHECKING, WITHOUT mainSubject

In [14]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName (COUNT(DISTINCT ?studentName) as ?archaeologist) WHERE 
{
   ?civilization wdt:P31 wd:Q28171280;
                 sc:name ?civilizationName.
   OPTIONAL{
    ?student wdt:P101 ?civilization; #'field of work' | #author/#mainSubject
        sc:name ?studentName;
        wdt:P106/wdt:P279* wd:Q3621491. #P106 'occupation'-> P279* 'sub Class' of Q3621491 ('archaeologist')
   }
}
GROUP BY ?civilization ?civilizationName
ORDER BY ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '7')]
[('civilization', 'http://www.wikidata.org/entity/Q4752820'), ('civilizationName', 'Ancient Crete'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '9')]
[('civilization', 'http://www.wikidata.org/entity/Q3327860'), ('civilizationName', 'Celtic kingdom of Ireland'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q25558247'), ('civilizationName', 'Chinese civilization'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q128904'), ('civilizationName', 'Elam'), ('archaeologist

### Final query for this task

#### Proposed solution: for 'who worked on it' I'm finding all DISTINCT archaeologists:

1) whoever has 'field of work' an ancient civilization
2) whoever wrote an article with 'main subject' on an ancient civilization

In [15]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# commented step by step

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName (COUNT(DISTINCT ?studentName) as ?archaeologist) WHERE 
{
   # getting all ancient civilization
   ?civilization wdt:P31 wd:Q28171280;                  # --wdt:P31 ('instance of')--> wd:Q28171280 ('ancient civilization')
                 sc:name ?civilizationName.
                 
   # optional only for who has archeologists who worked on a civilization
   OPTIONAL{
   
        # getting all students who 
            # 1) has field of work the civilization 
            # OR 
            # 2) who are the author of an article with the main subject the civilization
        # AND the student must have archaeologist or any of its subclass as occupation
            
        ?student wdt:P101|^wdt:P50/wdt:P921 ?civilization;  # --wdt:P101 ('field of work')--> 
                                                            # <--wdt:P50 (author')-- [] --wdt:P921 ('main subject')->
                sc:name ?studentName;
                wdt:P106/wdt:P279* wd:Q3621491.         # --wdt:P106 ('occupation')--> / --wdt:P279 ('sub Class')-->* wd:Q3621491 ('archaeologist')
   }
}
GROUP BY ?civilization ?civilizationName
ORDER BY ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '8')]
[('civilization', 'http://www.wikidata.org/entity/Q4752820'), ('civilizationName', 'Ancient Crete'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '10')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '12')]
[('civilization', 'http://www.wikidata.org/entity/Q3327860'), ('civilizationName', 'Celtic kingdom of Ireland'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q25558247'), ('civilizationName', 'Chinese civilization'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q128904'), ('civilizationName', 'Elam'), ('archaeologi

### OLD SOLUTION WITH UNIONS AND WITHOUT PATH PATTERNS

In [7]:
# For each ancient civilization find the number of archeologists who worked on it 
#    (return also the civilizations with no archeologists) (the result set must be triples of civilization IRI, label and #archeologist).

# --- old findings ---
# wd:Q13444093 wdt:P88 ('commissioned by') wd:Q189371 ('Sneferu');
#                                          wd :Q220240 ('Huni');

# wd:Q189371 wdt:P27 ('country of citizenship'), wd:Q11768 ('Ancient Egypt')
#            wdt:P2348 ('time period') wd:Q11768 (Ancient Egypt')

#  wd:Q11768 (Ancient Egypt') wdt:P31 ('instance of') wd:Q28171280 ('ancient civilization')
# --- end old findings ---

# wdt:P921 ('main subject') wd:Q55079391 ('Le couronnement du rempart de Saint-Lézer (Castrum Bigorra) en Novempopulanie') 
# wdt:P50 (author') wd:Q55110166 ('Kim Bowes')
# ?author --wdt:P106 ('occupation')-> wd:Q15983985 ('classical archaeologist') --wdt:P279 ('subclass of')-> wd:Q3621491 ('archaeologist')

queryString = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT DISTINCT ?civilization ?civilizationName (COUNT(?studentName) as ?archaeologist) WHERE 
{
    # need for civilizations with no archaeologists
    {
        ?civilization wdt:P31 wd:Q28171280;
                sc:name ?civilizationName.
    }
    UNION
    {
        SELECT DISTINCT ?civilization ?civilizationName ?studentName ?student 
        WHERE {
           #finding Archaeologist who has worked on ("field of work") the civilization
           {
               SELECT ?student ?studentName ?civilization ?civilizationName
               WHERE{
                   ?civilization wdt:P31 wd:Q28171280;
                                 sc:name ?civilizationName.
                   ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')

                   ?student wdt:P101 ?civilization; #'field of work'
                        sc:name ?studentName;
                        wdt:P106 ?occupation. #P106 'occupation'-> Q3621491 ('archaeologist) 


                   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
               }
           }
           UNION    
           #finding Archaeologist who wrote some article on the civilization
           {
               SELECT ?student ?studentName ?civilization ?civilizationName
               WHERE{
                   ?civilization wdt:P31 wd:Q28171280;
                                 sc:name ?civilizationName.
                   ?specializedArchaeologist wdt:P279 wd:Q3621491 . #sub Class of Q3621491 ('archaeologist')

                   ?article wdt:P921 ?civilization. #mainSubject
                   ?article wdt:P50 ?student. #author
                   ?student wdt:P106 ?occupation; #occupation  
                            sc:name ?studentName.    
                   FILTER (?occupation in (?specializedArchaeologist, wd:Q3621491))
                }
            }
        }
    }
}
GROUP BY ?civilization ?civilizationName
ORDER BY ?civilizationName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('civilization', 'http://www.wikidata.org/entity/Q2429397'), ('civilizationName', 'Ancient Carthage'), ('archaeologist', '8')]
[('civilization', 'http://www.wikidata.org/entity/Q4752820'), ('civilizationName', 'Ancient Crete'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q11768'), ('civilizationName', 'Ancient Egypt'), ('archaeologist', '10')]
[('civilization', 'http://www.wikidata.org/entity/Q11772'), ('civilizationName', 'Ancient Greece'), ('archaeologist', '12')]
[('civilization', 'http://www.wikidata.org/entity/Q3327860'), ('civilizationName', 'Celtic kingdom of Ireland'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q25558247'), ('civilizationName', 'Chinese civilization'), ('archaeologist', '0')]
[('civilization', 'http://www.wikidata.org/entity/Q173082'), ('civilizationName', 'Dacia'), ('archaeologist', '2')]
[('civilization', 'http://www.wikidata.org/entity/Q128904'), ('civilizationName', 'Elam'), ('archaeologi

## Task 6
Find the BGP for deity.

In [15]:
# Find the BGP for deity. (divinità/dio)
# | `wd:Q41127`| Poseidon   | node |

# Exploring from Poseidon

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q41127 ?p ?o.
   # get the label
   ?p sc:name ?pName.
   OPTIONAL {?p sc:name ?pName}.
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1299'), ('pName', 'depicted by'), ('o', 'http://www.wikidata.org/entity/Q107043392'), ('oName', 'Poseidon or Corybant statue at Elis')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q4532138'), ('oName', 'Granat Encyclopedic Dictionary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q30059240'), ('oName', "Real'nyj slovar' klassicheskih drevnostej po Ljubkeru")]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q106727050'), ('oName', 'Metropolitan Museum of Art Tagging Vocabulary')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q19180675'), ('oName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('p', 'http://www.wik

In [21]:
# Find the BGP for deity. (divinità/dio)
# | `wd:Q41127`| Poseidon   | node |

# Exploring from Poseidon, counting children just for fun

queryString = """
SELECT DISTINCT (COUNT(*) as ?numChildren)
WHERE {
   # bind something
   wd:Q41127 wdt:P40 ?o.
   # get the label
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('numChildren', '149')]
1


In [23]:
# Find the BGP for deity. (divinità/dio)
# wd:Q41127 ('Poseidon') wdt:P31 ('instance of') wd:Q22989102 ('Greek deity')
#                        wdt:P31 ('instance of') wd:Q1916821 ('water deity')

# Exploring from Poseidon
queryString = """
SELECT DISTINCT *
WHERE {
   # bind something
   wd:Q41127 wdt:P31 ?o.
   # get the label
   OPTIONAL {?o sc:name ?oName}.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1916821'), ('oName', 'water deity')]
[('o', 'http://www.wikidata.org/entity/Q22989102'), ('oName', 'Greek deity')]
2


In [25]:
# Find the BGP for deity. (divinità/dio)
# wd:Q41127 ('Poseidon') wdt:P31 ('instance of') wd:Q22989102 ('Greek deity')
#                        wdt:P31 ('instance of') wd:Q1916821 ('water deity')

# Exploring wather deity
queryString = """
SELECT DISTINCT *
WHERE {
   # bind something
   wd:Q1916821 ?p ?o.
   # get the label
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('o', 'http://www.wikidata.org/entity/Q13405593'), ('pName', 'subclass of'), ('oName', 'nature deity')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('o', 'http://www.wikidata.org/entity/Q1936706'), ('pName', 'subclass of'), ('oName', 'water sprite')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('o', 'http://www.wikidata.org/entity/Q8163944'), ('pName', "topic's main category"), ('oName', 'Category:Water deities')]
[('p', 'http://www.wikidata.org/prop/direct/P2925'), ('o', 'http://www.wikidata.org/entity/Q283'), ('pName', 'domain of saint or deity'), ('oName', 'water')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2671'), ('o', 'http://g.co/kg/g/121wt28p')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('o', '/g/121wt28p')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('o', 'Water deities'

In [26]:
# Find the BGP for deity. (divinità/dio)
# wd:Q41127 ('Poseidon') wdt:P31 ('instance of') wd:Q22989102 ('Greek deity')
#                        wdt:P31 ('instance of') wd:Q1916821 ('water deity')
# wd:Q1916821 ('water deity') --wdt:P279 ('subclass of') -> wd:Q13405593 ('nature deity')
# Exploring Q13405593 ('nature deity')

queryString = """
SELECT DISTINCT *
WHERE {
   # bind something
   wd:Q13405593 ?p ?o.
   # get the label
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('o', 'http://www.wikidata.org/entity/Q6630292'), ('pName', 'has list'), ('oName', 'list of nature deities')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('o', 'http://www.wikidata.org/entity/Q178885'), ('pName', 'subclass of'), ('oName', 'deity')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('o', 'http://www.wikidata.org/entity/Q20828805'), ('pName', 'subclass of'), ('oName', 'sprite')]
[('p', 'http://www.wikidata.org/prop/direct/P460'), ('o', 'http://www.wikidata.org/entity/Q739737'), ('pName', 'said to be the same as'), ('oName', 'Cernunnos')]
[('p', 'http://www.wikidata.org/prop/direct/P460'), ('o', 'http://www.wikidata.org/entity/Q1786347'), ('pName', 'said to be the same as'), ('oName', 'elemental')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('o', 'http://www.wikidata.org/entity/Q1916821'), ('pName', 'has part'), ('oName', 'water deity')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('o'

In [28]:
# Find the BGP for deity. (divinità/dio)
# wd:Q41127 ('Poseidon') wdt:P31 ('instance of') wd:Q22989102 ('Greek deity')
#                        wdt:P31 ('instance of') wd:Q1916821 ('water deity')
# wd:Q1916821 ('water deity') --wdt:P279 ('subclass of')--> wd:Q13405593 ('nature deity')
# wd:Q13405593 ('nature deity') --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# SOL
queryString = """
SELECT DISTINCT *
WHERE {
   # bind something
   wd:Q13405593 wdt:P279 ?o.
   # get the label
   OPTIONAL {
       ?o sc:name ?oName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q20828805'), ('oName', 'sprite')]
[('o', 'http://www.wikidata.org/entity/Q178885'), ('oName', 'deity')]
2


### Final query for this task

In [28]:
# Find the BGP for deity. (divinità/dio)
# wd:Q41127 ('Poseidon') wdt:P31 ('instance of') wd:Q22989102 ('Greek deity')
#                        wdt:P31 ('instance of') wd:Q1916821 ('water deity')
# wd:Q1916821 ('water deity') --wdt:P279 ('subclass of')--> wd:Q13405593 ('nature deity')
# wd:Q13405593 ('nature deity') --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

queryString = """
SELECT DISTINCT *
WHERE {
   # bind something
   wd:Q13405593 wdt:P279 ?o.
   # get the label
   OPTIONAL {
       ?o sc:name ?oName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q20828805'), ('oName', 'sprite')]
[('o', 'http://www.wikidata.org/entity/Q178885'), ('oName', 'deity')]
2


## Task 7
List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

In [29]:
# List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

# --- old findings ---
# wd:Q41127 ('Poseidon') wdt:P31 ('instance of') wd:Q22989102 ('Greek deity')
#                        wdt:P31 ('instance of') wd:Q1916821 ('water deity')
# wd:Q1916821 ('water deity') --wdt:P279 ('subclass of')--> wd:Q13405593 ('nature deity')
# wd:Q13405593 ('nature deity') --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')
# --- end old findings ---

# Exploring Greek deity

queryString = """
SELECT DISTINCT *
WHERE {
   # bind something
   wd:Q22989102 ?p ?o.
   # get the label
   OPTIONAL {
       ?o sc:name ?oName.
       ?p sc:name ?pName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('o', 'http://www.wikidata.org/entity/Q95074'), ('oName', 'fictional character'), ('pName', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P2596'), ('o', 'http://www.wikidata.org/entity/Q11772'), ('oName', 'Ancient Greece'), ('pName', 'culture')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('o', 'http://www.wikidata.org/entity/Q22988604'), ('oName', 'mythological Greek character'), ('pName', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('o', 'http://www.wikidata.org/entity/Q178885'), ('oName', 'deity'), ('pName', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('o', 'http://www.wikidata.org/entity/Q34726'), ('oName', 'Greek mythology'), ('pName', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('o', 'http://www.wikidata.org/entity/Q855270'), ('oName', 'religion in ancient Greece'), ('pName', 'part of')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [30]:
# List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

# --- old findings ---
# wd:Q41127 ('Poseidon') wdt:P31 ('instance of') wd:Q22989102 ('Greek deity')
#                        wdt:P31 ('instance of') wd:Q1916821 ('water deity')
# wd:Q1916821 ('water deity') --wdt:P279 ('subclass of')--> wd:Q13405593 ('nature deity')
# wd:Q13405593 ('nature deity') --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')
# --- end old findings ---

# Exploring Greek deity backward

queryString = """
SELECT DISTINCT *
WHERE {
   # bind something
   ?s ?p wd:Q22989102.
   # get the label
   OPTIONAL {
       ?s sc:name ?sName.
       ?p sc:name ?pName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q5081127'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('sName', 'Acmon'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q366533'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('sName', 'Adranus'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q3362052'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('sName', 'Pandora'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q6612'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('sName', 'Charon'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q2144233'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('sName', 'Pheme'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q1120723'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('sName', 'Pheme'), ('pName', 'instance of')]
[('s', 'http://www.wikidata.org/entity/Q1862189'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('sName'

In [31]:
# List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

# --- old findings ---
# wd:Q41127 ('Poseidon') wdt:P31 ('instance of') wd:Q22989102 ('Greek deity')
#                        wdt:P31 ('instance of') wd:Q1916821 ('water deity')
# wd:Q1916821 ('water deity') --wdt:P279 ('subclass of')--> wd:Q13405593 ('nature deity')
# wd:Q13405593 ('nature deity') --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')
# --- end old findings ---

# Exploring from Greek deity backward

queryString = """
SELECT DISTINCT *
WHERE {
   # bind something
   ?s wdt:P279 wd:Q22989102.
   # get the label
   OPTIONAL {
       ?s sc:name ?sName.
       #?p sc:name ?pName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1134605'), ('sName', 'Telchines')]
[('s', 'http://www.wikidata.org/entity/Q247617'), ('sName', 'Curetes')]
[('s', 'http://www.wikidata.org/entity/Q3346693'), ('sName', 'Greek nymph')]
[('s', 'http://www.wikidata.org/entity/Q45713'), ('sName', 'titan')]
[('s', 'http://www.wikidata.org/entity/Q1157625'), ('sName', 'daemon')]
[('s', 'http://www.wikidata.org/entity/Q201411'), ('sName', 'Horae')]
[('s', 'http://www.wikidata.org/entity/Q101609'), ('sName', 'Twelve Olympians')]
[('s', 'http://www.wikidata.org/entity/Q184353'), ('sName', 'Charites')]
[('s', 'http://www.wikidata.org/entity/Q878099'), ('sName', 'Greek primordial deity')]
[('s', 'http://www.wikidata.org/entity/Q66016'), ('sName', 'Muses')]
[('s', 'http://www.wikidata.org/entity/Q180287'), ('sName', 'Moirae')]
[('s', 'http://www.wikidata.org/entity/Q191192'), ('sName', 'Giants')]
[('s', 'http://www.wikidata.org/entity/Q578584'), ('sName', 'Oneiroi')]
[('s', 'http://www.wikidata.org/e

In [32]:
# List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

# --- old findings ---
# wd:Q41127 ('Poseidon') wdt:P31 ('instance of') wd:Q22989102 ('Greek deity')
#                        wdt:P31 ('instance of') wd:Q1916821 ('water deity')
# wd:Q1916821 ('water deity') --wdt:P279 ('subclass of')--> wd:Q13405593 ('nature deity')
# wd:Q13405593 ('nature deity') --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')
# --- end old findings ---

# wd:Q101609 (Twelve Olympians') --wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity').

# Exploring wd:Q101609 (Twelve Olympians')

queryString = """
SELECT DISTINCT ?sName ?pName ?s ?p
WHERE {
   # bind something
   ?s ?p wd:Q101609.
   # get the label
   OPTIONAL {
       ?s sc:name ?sName.
       ?p sc:name ?pName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sName', 'Apollo'), ('pName', 'part of'), ('s', 'http://www.wikidata.org/entity/Q37340'), ('p', 'http://www.wikidata.org/prop/direct/P361')]
[('sName', 'Athena'), ('pName', 'part of'), ('s', 'http://www.wikidata.org/entity/Q37122'), ('p', 'http://www.wikidata.org/prop/direct/P361')]
[('sName', 'Hera'), ('pName', 'part of'), ('s', 'http://www.wikidata.org/entity/Q38012'), ('p', 'http://www.wikidata.org/prop/direct/P361')]
[('sName', 'Artemis'), ('pName', 'part of'), ('s', 'http://www.wikidata.org/entity/Q39503'), ('p', 'http://www.wikidata.org/prop/direct/P361')]
[('sName', 'Hephaestus'), ('pName', 'part of'), ('s', 'http://www.wikidata.org/entity/Q44384'), ('p', 'http://www.wikidata.org/prop/direct/P361')]
[('sName', 'Hermes'), ('pName', 'part of'), ('s', 'http://www.wikidata.org/entity/Q41484'), ('p', 'http://www.wikidata.org/prop/direct/P361')]
[('sName', 'Category:Twelve Olympians'), ('pName', "category's main topic"), ('s', 'http://www.wikidata.org/entity/Q7452446'), ('p'

In [33]:
# List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

# --- old findings ---
# wd:Q41127 ('Poseidon') wdt:P31 ('instance of') wd:Q22989102 ('Greek deity')
#                        wdt:P31 ('instance of') wd:Q1916821 ('water deity')
# wd:Q1916821 ('water deity') --wdt:P279 ('subclass of')--> wd:Q13405593 ('nature deity')
# wd:Q13405593 ('nature deity') --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')
# --- end old findings ---

# wd:Q101609 (Twelve Olympians') --wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity').

# Exploring wd:Q101609 (Twelve Olympians')
# [wrong result bcoz ?p]

queryString = """
SELECT DISTINCT ?sName ?pName ?s ?p
WHERE {
   # bind something
   ?s wdt:P361 wd:Q101609.
   # get the label
   OPTIONAL {
       ?s sc:name ?sName.
       ?p sc:name ?pName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('sName', 'Hestia'), ('pName', 'software version identifier'), ('s', 'http://www.wikidata.org/entity/Q41419'), ('p', 'http://www.wikidata.org/entity/P348')]
[('sName', 'Hestia'), ('pName', 'tabular software version'), ('s', 'http://www.wikidata.org/entity/Q41419'), ('p', 'http://www.wikidata.org/entity/P4669')]
[('sName', 'Hestia'), ('pName', 'has edition or translation'), ('s', 'http://www.wikidata.org/entity/Q41419'), ('p', 'http://www.wikidata.org/entity/P747')]
[('sName', 'Hestia'), ('pName', 'Fonsi Nieto'), ('s', 'http://www.wikidata.org/entity/Q41419'), ('p', 'http://www.wikidata.org/entity/Q1002728')]
[('sName', 'Hestia'), ('pName', "Al Stewart's albums in chronological order"), ('s', 'http://www.wikidata.org/entity/Q41419'), ('p', 'http://www.wikidata.org/entity/Q100326534')]
[('sName', 'Hestia'), ('pName', 'Category:Adelina Lopes Vieira'), ('s', 'http://www.wikidata.org/entity/Q41419'), ('p', 'http://www.wikidata.org/entity/Q100327667')]
[('sName', 'Hestia'), ('pName'

In [34]:
# List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

# --- old findings ---
# wd:Q41127 ('Poseidon') wdt:P31 ('instance of') wd:Q22989102 ('Greek deity')
#                        wdt:P31 ('instance of') wd:Q1916821 ('water deity')
# wd:Q1916821 ('water deity') --wdt:P279 ('subclass of')--> wd:Q13405593 ('nature deity')
# wd:Q13405593 ('nature deity') --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')
# --- end old findings ---

# wd:Q101609 (Twelve Olympians') --wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity').

# Exploring wd:Q101609 (Twelve Olympians') backward

queryString = """
SELECT DISTINCT ?s ?sName 
WHERE {
   # bind something
   ?s wdt:P361 wd:Q101609.
   # get the label
   OPTIONAL {
       ?s sc:name ?sName.
       #?p sc:name ?pName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q37340'), ('sName', 'Apollo')]
[('s', 'http://www.wikidata.org/entity/Q37122'), ('sName', 'Athena')]
[('s', 'http://www.wikidata.org/entity/Q38012'), ('sName', 'Hera')]
[('s', 'http://www.wikidata.org/entity/Q39503'), ('sName', 'Artemis')]
[('s', 'http://www.wikidata.org/entity/Q44384'), ('sName', 'Hephaestus')]
[('s', 'http://www.wikidata.org/entity/Q41484'), ('sName', 'Hermes')]
[('s', 'http://www.wikidata.org/entity/Q41127'), ('sName', 'Poseidon')]
[('s', 'http://www.wikidata.org/entity/Q41419'), ('sName', 'Hestia')]
[('s', 'http://www.wikidata.org/entity/Q34201'), ('sName', 'Zeus')]
[('s', 'http://www.wikidata.org/entity/Q35500'), ('sName', 'Aphrodite')]
[('s', 'http://www.wikidata.org/entity/Q40730'), ('sName', 'Demeter')]
[('s', 'http://www.wikidata.org/entity/Q40901'), ('sName', 'Ares')]
12


### Final query for this task

In [35]:
# List all the twelve olympian greek deities (the result set must a list of 12 elements of couples of deity IRI and label).

# wd:Q41127 ('Poseidon')         --wdt:P31  ('instance of')-->  wd:Q22989102 ('Greek deity') .
# wd:Q101609 (Twelve Olympians') --wdt:P279 ('subclass of')-->  wd:Q22989102 ('Greek deity').
# SOLs                           --wdt:P361 ('part of')-->      wd:Q101609 (Twelve Olympians')

queryString = """
SELECT DISTINCT ?deity ?deityName
WHERE {
   # bind something
   ?deity wdt:P361 wd:Q101609. #?deity --'part of'--> 'Twelve Olympians'
   # get the label
   OPTIONAL {
       ?deity sc:name ?deityName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('deity', 'http://www.wikidata.org/entity/Q37340'), ('deityName', 'Apollo')]
[('deity', 'http://www.wikidata.org/entity/Q37122'), ('deityName', 'Athena')]
[('deity', 'http://www.wikidata.org/entity/Q38012'), ('deityName', 'Hera')]
[('deity', 'http://www.wikidata.org/entity/Q39503'), ('deityName', 'Artemis')]
[('deity', 'http://www.wikidata.org/entity/Q44384'), ('deityName', 'Hephaestus')]
[('deity', 'http://www.wikidata.org/entity/Q41484'), ('deityName', 'Hermes')]
[('deity', 'http://www.wikidata.org/entity/Q41127'), ('deityName', 'Poseidon')]
[('deity', 'http://www.wikidata.org/entity/Q41419'), ('deityName', 'Hestia')]
[('deity', 'http://www.wikidata.org/entity/Q34201'), ('deityName', 'Zeus')]
[('deity', 'http://www.wikidata.org/entity/Q35500'), ('deityName', 'Aphrodite')]
[('deity', 'http://www.wikidata.org/entity/Q40730'), ('deityName', 'Demeter')]
[('deity', 'http://www.wikidata.org/entity/Q40901'), ('deityName', 'Ares')]
12


## Task 8
Compare ancient Greece with ancient Egypt and find how many deities each one has.

In [37]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has.

# --- old findings ---
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# Exploring wd:Q22989102 ('Greek deity') to find ancient Greece and a commom point with ancient Egypt

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q22989102 ?p ?o.
   # get the label
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2596'), ('pName', 'culture'), ('o', 'http://www.wikidata.org/entity/Q11772'), ('oName', 'Ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q178885'), ('oName', 'deity')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q95074'), ('oName', 'fictional character')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q22988604'), ('oName', 'mythological Greek character')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q855270'), ('oName', 'religion in ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q34726'), ('oName', 'Greek mythology')]
[('p', 'http://www.wikidata.org/prop/direct/P

In [38]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has.

# --- old findings ---
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# Exploring wd:Q11772 ('Ancient Greece') to find a commom point with ancient Egypt

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName
WHERE {
   # bind something
   wd:Q11772 ?p ?o.
   # get the label
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal"), ('o', 'http://www.wikidata.org/entity/Q10566332'), ('oName', 'Portal:Ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('o', 'http://www.wikidata.org/entity/Q3181656'), ('oName', 'The Nuttall Encyclopædia')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('o', 'http://www.wikidata.org/entity/Q6054993'), ('oName', 'Template:Ancient Greece topics')]
[('p', 'http://www.wikidata.org/prop/direct/P1792'), ('pName', 'category of associated people'), ('o', 'http://www.wikidata.org/entity/Q6692116'), ('oName', 'Category:Ancient Greeks')]
[('p', 'http://www.wikidata.org/prop/direct/P2184'), ('pName', 'history of topic'), ('o', 'http://www.wikidata.org/entity/Q7798'), ('oName', 'history of ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P2341'), ('pName', 'indigenous to'), ('o', 'http

In [39]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has.

# --- old findings ---
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# Exploring wd:Q11772 ('Ancient Greece') to find a commom point with ancient Egypt - BACKWARD

queryString = """
SELECT DISTINCT ?o ?oName ?p ?pName 
WHERE {
   # bind something
   ?o ?p wd:Q11772.
   # get the label
   OPTIONAL {
       ?p sc:name ?pName.
       ?o sc:name ?oName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q6054993'), ('oName', 'Template:Ancient Greece topics'), ('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic')]
[('o', 'http://www.wikidata.org/entity/Q1978195'), ('oName', 'Nesiarch'), ('p', 'http://www.wikidata.org/prop/direct/P1001'), ('pName', 'applies to jurisdiction')]
[('o', 'http://www.wikidata.org/entity/Q499357'), ('oName', 'Andreas Luther'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('o', 'http://www.wikidata.org/entity/Q2732854'), ('oName', 'Epigraphical Museum'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('o', 'http://www.wikidata.org/entity/Q4446512'), ('oName', 'Igor Surikov'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[('o', 'http://www.wikidata.org/entity/Q1985476'), ('oName', 'Charles Thomas Newton'), ('p', 'http://www.wikidata.org/prop/direct/P101'), ('pName', 'field of work')]
[(

In [41]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has.

# --- old findings ---
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# Exploring wd:Q11772 ('Ancient Greece') to find a commom point with ancient Egypt --> already done in task 1
# Getting all deities from Ancient Greece

queryString = """
SELECT DISTINCT ?o ?oName #?p ?pName 
WHERE {
   # bind something
   ?o wdt:P2596 wd:Q11772.
   # get the label
   OPTIONAL {
       #?p sc:name ?pName.
       ?o sc:name ?oName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q26314'), ('oName', 'Eleusinian Mysteries')]
[('o', 'http://www.wikidata.org/entity/Q122602'), ('oName', 'Tanagra figurine')]
[('o', 'http://www.wikidata.org/entity/Q17591052'), ('oName', 'Phlyax Vases')]
[('o', 'http://www.wikidata.org/entity/Q17621289'), ('oName', 'Malla (Crete)')]
[('o', 'http://www.wikidata.org/entity/Q17622009'), ('oName', 'Aulon')]
[('o', 'http://www.wikidata.org/entity/Q17623020'), ('oName', 'Bromiscus')]
[('o', 'http://www.wikidata.org/entity/Q17627339'), ('oName', 'Hysiae')]
[('o', 'http://www.wikidata.org/entity/Q197019'), ('oName', 'Temple of Zeus in Olympia')]
[('o', 'http://www.wikidata.org/entity/Q211354'), ('oName', 'pederasty')]
[('o', 'http://www.wikidata.org/entity/Q15940498'), ('oName', 'Therapne')]
[('o', 'http://www.wikidata.org/entity/Q242741'), ('oName', 'Erechtheion')]
[('o', 'http://www.wikidata.org/entity/Q1132537'), ('oName', 'Old Comedy')]
[('o', 'http://www.wikidata.org/entity/Q456547'), ('oNam

In [44]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# Getting all deities from Ancient Greece and finding a common starting point 

queryString = """
SELECT DISTINCT ?specialDeity ?specialDeityName #?o ?oName #?p ?pName 
WHERE {
   # bind something
   ?specialDeity wdt:P2596 wd:Q11772;
       wdt:P279 wd:Q178885.
       
   # get the label
   OPTIONAL {
       ?specialDeity sc:name ?specialDeityName.
       #?p sc:name ?pName.
       #?o sc:name ?oName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [45]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# Getting all deities from Ancient Greece and finding a common starting point 


queryString = """
SELECT DISTINCT ?specialDeity ?specialDeityName #?o ?oName #?p ?pName 
WHERE {
   # bind something
   ?specialDeity wdt:P2596 wd:Q11772;
                 wdt:P279 wd:Q178885.
       
   # get the label
   OPTIONAL {
       ?specialDeity sc:name ?specialDeityName.
       #?p sc:name ?pName.
       #?o sc:name ?oName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity')]
1


#### Found a common starting point

In [46]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# Getting all deities from Ancient Greece and finding a common starting point 

queryString = """
SELECT DISTINCT ?specialDeity ?specialDeityName #?o ?oName #?p ?pName 
WHERE {
   # bind something
   VALUES (?culture){
       (wd:Q11772)
       (wd:Q11768)
   }
   ?specialDeity wdt:P2596 ?culture;
                 wdt:P279 wd:Q178885.
       
   # get the label
   OPTIONAL {
       ?specialDeity sc:name ?specialDeityName.
       #?p sc:name ?pName.
       #?o sc:name ?oName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('specialDeity', 'http://www.wikidata.org/entity/Q146083'), ('specialDeityName', 'Ancient Egyptian deity')]
[('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity')]
2


In [48]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# Getting all deities from Ancient Egypt

queryString = """
SELECT DISTINCT ?specialDeity ?specialDeityName #?o ?oName #?p ?pName 
WHERE {
   # bind something
   ?specialDeity wdt:P279 wd:Q178885.
       
   # get the label
   OPTIONAL {
       ?specialDeity sc:name ?specialDeityName.
       #?p sc:name ?pName.
       #?o sc:name ?oName.
    }.
}
ORDER BY ?specialDeityName
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('specialDeity', 'http://www.wikidata.org/entity/Q146083'), ('specialDeityName', 'Ancient Egyptian deity')]
[('specialDeity', 'http://www.wikidata.org/entity/Q3549810'), ('specialDeityName', 'Anito')]
[('specialDeity', 'http://www.wikidata.org/entity/Q676379'), ('specialDeityName', 'Anunnaki')]
[('specialDeity', 'http://www.wikidata.org/entity/Q3774531'), ('specialDeityName', 'Atua')]
[('specialDeity', 'http://www.wikidata.org/entity/Q32066628'), ('specialDeityName', 'Aztec deity')]
[('specialDeity', 'http://www.wikidata.org/entity/Q26701417'), ('specialDeityName', 'Balabhadra')]
[('specialDeity', 'http://www.wikidata.org/entity/Q178149'), ('specialDeityName', 'Bodhisattva')]
[('specialDeity', 'http://www.wikidata.org/entity/Q65122124'), ('specialDeityName', 'Buddhist deity')]
[('specialDeity', 'http://www.wikidata.org/entity/Q104439732'), ('specialDeityName', 'Caboclo')]
[('specialDeity', 'http://www.wikidata.org/entity/Q465434'), ('specialDeityName', 'Celtic deity')]
[('spec

In [49]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# Expanding specialDeity to get complete list of deities

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885.                 
    ?s ?p ?specialDeity.   
   
    # get the label
    OPTIONAL {
        ?specialDeity sc:name ?specialDeityName.
        ?p sc:name ?pName.
        ?s sc:name ?sName.
    }.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q6573004'), ('sName', 'Heliopolitans'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('s', 'http://www.wikidata.org/entity/Q108092253'), ('sName', 'Ancient Egyptian pantheon'), ('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class')]
[('s', 'http://www.wikidata.org/entity/Q48803719'), ('sName', 'Cavern deities of the underworld'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q48803736'), ('sName', 'Gate deities of the underworld'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q9777847'), ('sName', 'Category:Egyptian deities'), ('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic")]
[('s', 'http://www.wikidata.org/entity/Q7214264'), ('sName', 'Category:Egyptian gods'), ('p', 'http://www.wikidata.org/prop/direc

In [50]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# Expanding specialDeity to get complete list of deities
# Filtering - wdt:P31 'instance of'

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885.                 
    ?s ?p ?specialDeity.   
   
    # get the label
    OPTIONAL {
        ?specialDeity sc:name ?specialDeityName.
        ?p sc:name ?pName.
        ?s sc:name ?sName.
    }.
    FILTER (?p not in (wdt:P31))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q6573004'), ('sName', 'Heliopolitans'), ('p', 'http://www.wikidata.org/prop/direct/P144'), ('pName', 'based on')]
[('s', 'http://www.wikidata.org/entity/Q108092253'), ('sName', 'Ancient Egyptian pantheon'), ('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class')]
[('s', 'http://www.wikidata.org/entity/Q48803719'), ('sName', 'Cavern deities of the underworld'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q48803736'), ('sName', 'Gate deities of the underworld'), ('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of')]
[('s', 'http://www.wikidata.org/entity/Q9777847'), ('sName', 'Category:Egyptian deities'), ('p', 'http://www.wikidata.org/prop/direct/P301'), ('pName', "category's main topic")]
[('s', 'http://www.wikidata.org/entity/Q7214264'), ('sName', 'Category:Egyptian gods'), ('p', 'http://www.wikidata.org/prop/direc

In [51]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# Expanding specialDeity to get complete list of deities
# Expanding Q516588 ('list of Greek gods and goddesses') to get a complete list of greek deities, secodary sol


queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName # ?s ?sName 
WHERE {              
    wd:Q516588 ?p ?o.   
   
    # get the label
    OPTIONAL {
        #?s sc:name ?sName.
        ?p sc:name ?pName.
        ?o sc:name ?oName.
    }.
    #FILTER (?p not in (wdt:P31))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P140'), ('pName', 'religion'), ('o', 'http://www.wikidata.org/entity/Q855270'), ('oName', 'religion in ancient Greece')]
[('p', 'http://www.wikidata.org/prop/direct/P1754'), ('pName', 'category related to list'), ('o', 'http://www.wikidata.org/entity/Q54803387'), ('oName', 'Category:Mythological Greek characters')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q13406463'), ('oName', 'Wikimedia list article')]
[('p', 'http://www.wikidata.org/prop/direct/P360'), ('pName', 'is a list of'), ('o', 'http://www.wikidata.org/entity/Q22989102'), ('oName', 'Greek deity')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q20059824'), ('oName', 'list of mythological Greek characters')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/p

In [52]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# Expanding specialDeity to get complete list of deities
# Expanding Q516588 ('list of Greek gods and goddesses') to get a complete list of greek deities, secodary sol -- backward

queryString = """
SELECT DISTINCT ?s ?sName ?p ?pName #?o ?oName # ?s ?sName 
WHERE {              
    ?s ?p wd:Q516588.   
   
    # get the label
    OPTIONAL {
        ?s sc:name ?sName.
        ?p sc:name ?pName.
        #?o sc:name ?oName.
    }.
    #FILTER (?p not in (wdt:P31))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q20059824'), ('sName', 'list of mythological Greek characters'), ('p', 'http://www.wikidata.org/prop/direct/P527'), ('pName', 'has part')]
[('s', 'http://www.wikidata.org/entity/Q9932522'), ('sName', 'Category:Greek gods'), ('p', 'http://www.wikidata.org/prop/direct/P1753'), ('pName', 'list related to category')]
[('s', 'http://www.wikidata.org/entity/Q54803387'), ('sName', 'Category:Mythological Greek characters'), ('p', 'http://www.wikidata.org/prop/direct/P1753'), ('pName', 'list related to category')]
[('s', 'http://www.wikidata.org/entity/Q12053312'), ('p', 'http://www.w3.org/2002/07/owl#sameAs')]
[('s', 'http://www.wikidata.org/entity/Q16157635'), ('p', 'http://www.w3.org/2002/07/owl#sameAs')]
[('s', 'http://www.wikidata.org/entity/Q25622199'), ('p', 'http://www.w3.org/2002/07/owl#sameAs')]
[('s', 'http://www.wikidata.org/entity/Q25700769'), ('p', 'http://www.w3.org/2002/07/owl#sameAs')]
[('s', 'https://www.wikidata.org/wiki/Special:

In [54]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# Expanding specialDeity to get complete list of deities
# Expanding Q516588 ('list of Greek gods and goddesses') can't find the list
# back do prev sol, Expanding specialDeity through wdt:P279 subclass of

queryString = """
SELECT DISTINCT ?s ?sName # ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885.                 
    ?s wdt:P279 ?specialDeity.   
   
    # get the label
    OPTIONAL {
        ?specialDeity sc:name ?specialDeityName.
        #?p sc:name ?pName.
        ?s sc:name ?sName.
    }.
    #FILTER (?p not in (wdt:P31))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q48803736'), ('sName', 'Gate deities of the underworld')]
[('s', 'http://www.wikidata.org/entity/Q48803719'), ('sName', 'Cavern deities of the underworld')]
[('s', 'http://www.wikidata.org/entity/Q21707266'), ('sName', 'Antithei')]
[('s', 'http://www.wikidata.org/entity/Q23015925'), ('sName', 'demigod of Greek mythology')]
[('s', 'http://www.wikidata.org/entity/Q184353'), ('sName', 'Charites')]
[('s', 'http://www.wikidata.org/entity/Q12481383'), ('sName', 'Greek underworld deities')]
[('s', 'http://www.wikidata.org/entity/Q45713'), ('sName', 'titan')]
[('s', 'http://www.wikidata.org/entity/Q6019476'), ('sName', 'Furies')]
[('s', 'http://www.wikidata.org/entity/Q191192'), ('sName', 'Giants')]
[('s', 'http://www.wikidata.org/entity/Q11905703'), ('sName', 'Anthelioi')]
[('s', 'http://www.wikidata.org/entity/Q5709717'), ('sName', 'Astra Planeta')]
[('s', 'http://www.wikidata.org/entity/Q247617'), ('sName', 'Curetes')]
[('s', 'http://www.wikida

In [55]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q21707266 ('Antithei')
# wd:Q66016 ('Muses')

# Getting all deities from Ancient Egypt & Ancient Greece
# Expanding specialDeity to get complete list of deities
# Expanding Q516588 ('list of Greek gods and goddesses') can't find the list
# back do prev sol, Expanding specialDeity through wdt:P279 subclass of
# Expanding wd:Q21707266 ('Antithei')

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName 
WHERE {              
    wd:Q21707266 ?p ?o.   
   
    # get the label
    OPTIONAL {
        #?specialDeity sc:name ?specialDeityName.
        ?p sc:name ?pName.
        ?o sc:name ?oName.
    }.
    #FILTER (?p not in (wdt:P31))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q22989102'), ('oName', 'Greek deity')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pName', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q28061975'), ('oName', 'group of Greek mythical characters')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q337547'), ('oName', 'ancient Roman religion')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('pName', 'part of'), ('o', 'http://www.wikidata.org/entity/Q855270'), ('oName', 'religion in ancient Greece')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct/P2272'), ('o', 'Antithei')]
[('p', 'http://schema.org/name'), ('o', 'Antithei')]
7


In [56]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q21707266 ('Antithei')
# wd:Q66016 ('Muses')

# Getting all deities from Ancient Egypt & Ancient Greece
# Expanding specialDeity to get complete list of deities
# Expanding Q516588 ('list of Greek gods and goddesses') can't find the list
# back do prev sol, Expanding specialDeity through wdt:P279 subclass of
# Expanding wd:Q21707266 ('Antithei') --backward

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName 
WHERE {              
    ?o ?p wd:Q21707266.   
   
    # get the label
    OPTIONAL {
        #?specialDeity sc:name ?specialDeityName.
        ?p sc:name ?pName.
        ?o sc:name ?oName.
    }.
    #FILTER (?p not in (wdt:P31))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://schema.org/about'), ('o', 'https://www.wikidata.org/wiki/Special:EntityData/Q21707266')]
1


In [57]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q21707266 ('Antithei')
# wd:Q66016 ('Muses')

# Getting all deities from Ancient Egypt & Ancient Greece
# Expanding specialDeity to get complete list of deities
# Expanding Q516588 ('list of Greek gods and goddesses') can't find the list
# back do prev sol, Expanding specialDeity through wdt:P279 subclass of
# Expanding wd:Q66016 ('Muses') --backward

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName 
WHERE {              
    ?o ?p wd:Q66016.   
   
    # get the label
    OPTIONAL {
        #?specialDeity sc:name ?specialDeityName.
        ?p sc:name ?pName.
        ?o sc:name ?oName.
    }.
    #FILTER (?p not in (wdt:P31))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pName', 'template has topic'), ('o', 'http://www.wikidata.org/entity/Q6818632'), ('oName', 'Template:Footer Nine Muses')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after'), ('o', 'http://www.wikidata.org/entity/Q3516783'), ('oName', 'Teatro delle Muse')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after'), ('o', 'http://www.wikidata.org/entity/Q17461272'), ('oName', 'Muzenplein')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('pName', 'named after'), ('o', 'http://www.wikidata.org/entity/Q28223644'), ('oName', 'Théâtre des Muses')]
[('p', 'http://www.wikidata.org/prop/direct/P180'), ('pName', 'depicts'), ('o', 'http://www.wikidata.org/entity/Q19912180'), ('oName', 'The Muse: History')]
[('p', 'http://www.wikidata.org/prop/direct/P180'), ('pName', 'depicts'), ('o', 'http://www.wikidata.org/entity/Q19925641'), ('oName', 'Euterpe')]
[('p', 'http://www.wikidata.org/prop

In [58]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q21707266 ('Antithei')
# wd:Q66016 ('Muses')

# wd:Q176283('Aoide')
# wd:Q505207('Melete')
# wd:Q690388('Mneme')
# wd:Q103975('Calliope')
# wd:Q103992('Melpomene')
# wd:Q104002('Urania')
# wd:Q104012('Euterpe')
# wd:Q104042('Terpsichore')
# wd:Q104413('Erato')
# wd:Q104425('Polyhymnia')
# wd:Q108019('Thalia')
# wd:Q5957283('Hypate')
# wd:Q6821184('Mese')
# wd:Q3523555('Thelxinoë')
# wd:Q9049602('Nete')
# wd:Q12899635('Dia')
# wd:Q12899764('Enope')
# wd:Q12899831('Eunice')
# wd:Q12900215('Helice') -- wdt:P31 ('instance of')--> wd:Q66016 ('Muses')

# Getting all deities from Ancient Egypt & Ancient Greece
# Expanding specialDeity to get complete list of deities
# Expanding Q516588 ('list of Greek gods and goddesses') can't find the list
# back do prev sol, Expanding specialDeity through wdt:P279 subclass of
# Expanding wd:Q66016 ('Muses') --backward
# Trying avoid going to in depth, checking if greek deity contains already the muses

queryString = """
SELECT DISTINCT ?p ?pName ?o ?oName 
WHERE {              
    ?o ?p wd:Q22989102.    #greek deity
   
    # get the label
    OPTIONAL {
        #?specialDeity sc:name ?specialDeityName.
        ?p sc:name ?pName.
        ?o sc:name ?oName.
    }.
    #FILTER (?p not in (wdt:P31))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('pName', 'has parts of the class'), ('o', 'http://www.wikidata.org/entity/Q28919351'), ('oName', 'Gods')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1134605'), ('oName', 'Telchines')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q247617'), ('oName', 'Curetes')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q3346693'), ('oName', 'Greek nymph')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q45713'), ('oName', 'titan')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q1157625'), ('oName', 'daemon')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'), ('o'

In [60]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q21707266 ('Antithei')
# wd:Q66016 ('Muses')

# wd:Q176283('Aoide')
# wd:Q505207('Melete')
# wd:Q690388('Mneme')
# wd:Q103975('Calliope')
# wd:Q103992('Melpomene')
# wd:Q104002('Urania')
# wd:Q104012('Euterpe')
# wd:Q104042('Terpsichore')
# wd:Q104413('Erato')
# wd:Q104425('Polyhymnia')
# wd:Q108019('Thalia')
# wd:Q5957283('Hypate')
# wd:Q6821184('Mese')
# wd:Q3523555('Thelxinoë')
# wd:Q9049602('Nete')
# wd:Q12899635('Dia')
# wd:Q12899764('Enope')
# wd:Q12899831('Eunice')
# wd:Q12900215('Helice') -- wdt:P31 ('instance of')--> wd:Q66016 ('Muses')

# wd:Q66016 ('Muses') --wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# Expanding specialDeity to get complete list of deities
# Expanding Q516588 ('list of Greek gods and goddesses') can't find the list
# back do prev sol, Expanding specialDeity through wdt:P279 subclass of
# Expanding wd:Q66016 ('Muses') --backward
# Trying avoid going to in depth, checking if greek deity contains already the muses, withoud subclass of


queryString = """
SELECT DISTINCT ?o ?oName 
WHERE {              
    ?o wdt:P31 wd:Q22989102.    #greek deity
   
    # get the label
    OPTIONAL {
        #?specialDeity sc:name ?specialDeityName.
        #?p sc:name ?pName.
        ?o sc:name ?oName.
    }.
    #FILTER (?p not in (wdt:P31))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q5081127'), ('oName', 'Acmon')]
[('o', 'http://www.wikidata.org/entity/Q366533'), ('oName', 'Adranus')]
[('o', 'http://www.wikidata.org/entity/Q3362052'), ('oName', 'Pandora')]
[('o', 'http://www.wikidata.org/entity/Q6612'), ('oName', 'Charon')]
[('o', 'http://www.wikidata.org/entity/Q2144233'), ('oName', 'Pheme')]
[('o', 'http://www.wikidata.org/entity/Q1120723'), ('oName', 'Pheme')]
[('o', 'http://www.wikidata.org/entity/Q1862189'), ('oName', 'Paean')]
[('o', 'http://www.wikidata.org/entity/Q1775444'), ('oName', 'Phosphorus')]
[('o', 'http://www.wikidata.org/entity/Q178417'), ('oName', 'Eos')]
[('o', 'http://www.wikidata.org/entity/Q59697'), ('oName', 'Hesperus')]
[('o', 'http://www.wikidata.org/entity/Q1084868'), ('oName', 'Lyssa')]
[('o', 'http://www.wikidata.org/entity/Q1294659'), ('oName', 'Telesphorus')]
[('o', 'http://www.wikidata.org/entity/Q18423351'), ('oName', 'Basileia')]
[('o', 'http://www.wikidata.org/entity/Q185670'), ('oNa

In [61]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q21707266 ('Antithei')
# wd:Q66016 ('Muses')

# wd:Q176283('Aoide')
# wd:Q505207('Melete')
# wd:Q690388('Mneme')
# wd:Q103975('Calliope')
# wd:Q103992('Melpomene')
# wd:Q104002('Urania')
# wd:Q104012('Euterpe')
# wd:Q104042('Terpsichore')
# wd:Q104413('Erato')
# wd:Q104425('Polyhymnia')
# wd:Q108019('Thalia')
# wd:Q5957283('Hypate')
# wd:Q6821184('Mese')
# wd:Q3523555('Thelxinoë')
# wd:Q9049602('Nete')
# wd:Q12899635('Dia')
# wd:Q12899764('Enope')
# wd:Q12899831('Eunice')
# wd:Q12900215('Helice') -- wdt:P31 ('instance of')--> wd:Q66016 ('Muses')

# wd:Q66016 ('Muses') --wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# Expanding specialDeity to get complete list of deities
# Expanding Q516588 ('list of Greek gods and goddesses') can't find the list
# back do prev sol, Expanding specialDeity through wdt:P279 subclass of
# Expanding wd:Q66016 ('Muses') --backward
# Trying avoid going to in depth, checking if greek deity contains already the muses, withoud subclass of
# Hardcoded way

queryString = """
SELECT DISTINCT ?o ?oName 
WHERE {              

    VALUES(?o){
        (wd:Q176283) # 'Aoide'
        (wd:Q505207) # 'Melete'
        (wd:Q690388) # 'Mneme'
        (wd:Q103975) # 'Calliope'
        (wd:Q103992) # 'Melpomene'

        (wd:Q104002) # 'Urania'
        (wd:Q104012) # 'Euterpe'
        (wd:Q104042) # 'Terpsichore'
        (wd:Q104413) # 'Erato'

        (wd:Q104425) # 'Polyhymnia'
        (wd:Q108019) # 'Thalia'
        (wd:Q5957283) # 'Hypate'
        (wd:Q6821184) # 'Mese'
        (wd:Q3523555) # 'Thelxinoë'

        (wd:Q9049602) # 'Nete'
        (wd:Q12899635) # 'Dia'
        (wd:Q12899764) # 'Enope'
        (wd:Q12899831) # 'Eunice'
        (wd:Q12900215) # 'Helice'
    }
    ?o wdt:P31 wd:Q22989102.    #greek deity
   
    # get the label
    OPTIONAL {
        #?specialDeity sc:name ?specialDeityName.
        #?p sc:name ?pName.
        ?o sc:name ?oName.
    }.
    #FILTER (?p not in (wdt:P31))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [4]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q21707266 ('Antithei')
# wd:Q66016 ('Muses')

# wd:Q176283('Aoide')
# wd:Q505207('Melete')
# wd:Q690388('Mneme')
# wd:Q103975('Calliope')
# wd:Q103992('Melpomene')
# wd:Q104002('Urania')
# wd:Q104012('Euterpe')
# wd:Q104042('Terpsichore')
# wd:Q104413('Erato')
# wd:Q104425('Polyhymnia')
# wd:Q108019('Thalia')
# wd:Q5957283('Hypate')
# wd:Q6821184('Mese')
# wd:Q3523555('Thelxinoë')
# wd:Q9049602('Nete')
# wd:Q12899635('Dia')
# wd:Q12899764('Enope')
# wd:Q12899831('Eunice')
# wd:Q12900215('Helice') -- wdt:P31 ('instance of')--> wd:Q66016 ('Muses')

# wd:Q66016 ('Muses') --wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# Expanding specialDeity to get complete list of deities
# Expanding Q516588 ('list of Greek gods and goddesses') can't find the list
# back do prev sol, Expanding specialDeity through wdt:P279 subclass of
# Expanding wd:Q66016 ('Muses') --backward
# Trying avoid going to in depth, checking if greek deity contains already the muses, withoud subclass of
# Hardcoded way

queryString = """
SELECT DISTINCT ?o ?oName 
WHERE {              
    ?o wdt:P31 wd:Q22989102.    #greek deity
   
    # get the label
    OPTIONAL {
        ?o sc:name ?oName.
    }.
    FILTER (regex(?oName,"Talia"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [5]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q21707266 ('Antithei')
# wd:Q66016 ('Muses')

# wd:Q176283('Aoide')
# wd:Q505207('Melete')
# wd:Q690388('Mneme')
# wd:Q103975('Calliope')
# wd:Q103992('Melpomene')
# wd:Q104002('Urania')
# wd:Q104012('Euterpe')
# wd:Q104042('Terpsichore')
# wd:Q104413('Erato')
# wd:Q104425('Polyhymnia')
# wd:Q108019('Thalia')
# wd:Q5957283('Hypate')
# wd:Q6821184('Mese')
# wd:Q3523555('Thelxinoë')
# wd:Q9049602('Nete')
# wd:Q12899635('Dia')
# wd:Q12899764('Enope')
# wd:Q12899831('Eunice')
# wd:Q12900215('Helice') -- wdt:P31 ('instance of')--> wd:Q66016 ('Muses')

# wd:Q66016 ('Muses') --wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# Expanding specialDeity to get complete list of deities
# Expanding Q516588 ('list of Greek gods and goddesses') can't find the list
# back do prev sol, Expanding specialDeity through wdt:P279 subclass of
# Expanding wd:Q66016 ('Muses') --backward
# Trying avoid going to in depth, checking if greek deity contains already the muses, withoud subclass of
# Hardcoded way

queryString = """
SELECT DISTINCT ?o ?oName 
WHERE {              
    ?o wdt:P31 wd:Q22989102.    #greek deity
   
    # get the label
    OPTIONAL {
        ?o sc:name ?oName.
    }.
    FILTER (regex(?oName,"Erato"))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [62]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the subclasses of a specialDeity

queryString = """
SELECT DISTINCT ?s ?sName # ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885.                 
    ?s wdt:P279 ?specialDeity.   
   
    # get the label
    OPTIONAL {
        ?specialDeity sc:name ?specialDeityName.
        #?p sc:name ?pName.
        ?s sc:name ?sName.
    }.
    #FILTER (?p not in (wdt:P31))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q48803736'), ('sName', 'Gate deities of the underworld')]
[('s', 'http://www.wikidata.org/entity/Q48803719'), ('sName', 'Cavern deities of the underworld')]
[('s', 'http://www.wikidata.org/entity/Q21707266'), ('sName', 'Antithei')]
[('s', 'http://www.wikidata.org/entity/Q23015925'), ('sName', 'demigod of Greek mythology')]
[('s', 'http://www.wikidata.org/entity/Q184353'), ('sName', 'Charites')]
[('s', 'http://www.wikidata.org/entity/Q12481383'), ('sName', 'Greek underworld deities')]
[('s', 'http://www.wikidata.org/entity/Q45713'), ('sName', 'titan')]
[('s', 'http://www.wikidata.org/entity/Q6019476'), ('sName', 'Furies')]
[('s', 'http://www.wikidata.org/entity/Q191192'), ('sName', 'Giants')]
[('s', 'http://www.wikidata.org/entity/Q11905703'), ('sName', 'Anthelioi')]
[('s', 'http://www.wikidata.org/entity/Q5709717'), ('sName', 'Astra Planeta')]
[('s', 'http://www.wikidata.org/entity/Q247617'), ('sName', 'Curetes')]
[('s', 'http://www.wikida

In [63]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the INCTANCES of a specialDeity

queryString = """
SELECT DISTINCT ?s ?sName ?specialDeity ?specialDeityName # ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885.                 
    ?s wdt:P31 ?specialDeity. 
   
    # get the label
    OPTIONAL {
        ?specialDeity sc:name ?specialDeityName.
        #?p sc:name ?pName.
        ?s sc:name ?sName.
    }.
    #FILTER (?p not in (wdt:P31))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q145915'), ('sName', 'Nekhbet'), ('specialDeity', 'http://www.wikidata.org/entity/Q146083'), ('specialDeityName', 'Ancient Egyptian deity')]
[('s', 'http://www.wikidata.org/entity/Q58373'), ('sName', 'Amun'), ('specialDeity', 'http://www.wikidata.org/entity/Q146083'), ('specialDeityName', 'Ancient Egyptian deity')]
[('s', 'http://www.wikidata.org/entity/Q2620491'), ('sName', 'Werethekau'), ('specialDeity', 'http://www.wikidata.org/entity/Q146083'), ('specialDeityName', 'Ancient Egyptian deity')]
[('s', 'http://www.wikidata.org/entity/Q170569'), ('sName', 'Atum'), ('specialDeity', 'http://www.wikidata.org/entity/Q146083'), ('specialDeityName', 'Ancient Egyptian deity')]
[('s', 'http://www.wikidata.org/entity/Q211611'), ('sName', 'Anuket'), ('specialDeity', 'http://www.wikidata.org/entity/Q146083'), ('specialDeityName', 'Ancient Egyptian deity')]
[('s', 'http://www.wikidata.org/entity/Q146313'), ('sName', 'Sobek'), ('specialDeity', 'http://w

In [64]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the SUBCLASSES|INSTANCES of a specialDeity

queryString = """
SELECT DISTINCT ?s ?sName #?specialDeity ?specialDeityName # ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885.                 
    ?s wdt:P31|wdt:P279 ?specialDeity. 
   
    # get the label
    OPTIONAL {
        ?specialDeity sc:name ?specialDeityName.
        #?p sc:name ?pName.
        ?s sc:name ?sName.
    }.
    #FILTER (?p not in (wdt:P31))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q145915'), ('sName', 'Nekhbet')]
[('s', 'http://www.wikidata.org/entity/Q58373'), ('sName', 'Amun')]
[('s', 'http://www.wikidata.org/entity/Q2620491'), ('sName', 'Werethekau')]
[('s', 'http://www.wikidata.org/entity/Q170569'), ('sName', 'Atum')]
[('s', 'http://www.wikidata.org/entity/Q211611'), ('sName', 'Anuket')]
[('s', 'http://www.wikidata.org/entity/Q146313'), ('sName', 'Sobek')]
[('s', 'http://www.wikidata.org/entity/Q1252904'), ('sName', 'Ra')]
[('s', 'http://www.wikidata.org/entity/Q130227'), ('sName', 'Aten')]
[('s', 'http://www.wikidata.org/entity/Q133343'), ('sName', 'Hathor')]
[('s', 'http://www.wikidata.org/entity/Q845614'), ('sName', 'Maahes')]
[('s', 'http://www.wikidata.org/entity/Q116273'), ('sName', 'Tefnut')]
[('s', 'http://www.wikidata.org/entity/Q1936121'), ('sName', 'Min-Kamutef')]
[('s', 'http://www.wikidata.org/entity/Q1627813'), ('sName', 'Horus the Child')]
[('s', 'http://www.wikidata.org/entity/Q790000'), ('sName'

In [66]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the SUBCLASSES of a specialDeity
# GETTING deities --> subDeities --> specialDeity

queryString = """
SELECT DISTINCT ?subDeity ?subDeityName ?deity ?deityName #?specialDeity ?specialDeityName # ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885.                 
    ?subDeity wdt:P279 ?specialDeity. #subclass of
    ?deity wdt:P31 ?subDeity. # instance of
    
    
   
    # get the label
    OPTIONAL {
        ?specialDeity sc:name ?specialDeityName.
        #?p sc:name ?pName.
        ?subDeity sc:name ?subDeityName.
        ?deity sc:name ?deityName.
    }.
    #FILTER (?p not in (wdt:P31))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('subDeity', 'http://www.wikidata.org/entity/Q247617'), ('subDeityName', 'Curetes'), ('deity', 'http://www.wikidata.org/entity/Q12878563'), ('deityName', 'Scelmis')]
[('subDeity', 'http://www.wikidata.org/entity/Q247617'), ('subDeityName', 'Curetes'), ('deity', 'http://www.wikidata.org/entity/Q60318124'), ('deityName', 'Eleutherus')]
[('subDeity', 'http://www.wikidata.org/entity/Q3346693'), ('subDeityName', 'Greek nymph'), ('deity', 'http://www.wikidata.org/entity/Q1075258'), ('deityName', 'Chloris')]
[('subDeity', 'http://www.wikidata.org/entity/Q3346693'), ('subDeityName', 'Greek nymph'), ('deity', 'http://www.wikidata.org/entity/Q203673'), ('deityName', 'Kallisto')]
[('subDeity', 'http://www.wikidata.org/entity/Q3346693'), ('subDeityName', 'Greek nymph'), ('deity', 'http://www.wikidata.org/entity/Q3086495'), ('deityName', 'Thalia')]
[('subDeity', 'http://www.wikidata.org/entity/Q3346693'), ('subDeityName', 'Greek nymph'), ('deity', 'http://www.wikidata.org/entity/Q3680908')

In [3]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the SUBCLASSES of a specialDeity
# GETTING DISTINCT subDeities --> specialDeity, checking

queryString = """
SELECT DISTINCT ?subDeity ?subDeityName #?deity ?deityName #?specialDeity ?specialDeityName # ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885.                 
    ?subDeity wdt:P279 ?specialDeity. #subclass of
    ?deity wdt:P31 ?subDeity. # instance of
    
    
   
    # get the label
    ?subDeity sc:name ?subDeityName.
    OPTIONAL {
        ?specialDeity sc:name ?specialDeityName.
        #?p sc:name ?pName.
        ?deity sc:name ?deityName.
    }.
    #FILTER (?p not in (wdt:P31))
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('subDeity', 'http://www.wikidata.org/entity/Q247617'), ('subDeityName', 'Curetes')]
[('subDeity', 'http://www.wikidata.org/entity/Q3346693'), ('subDeityName', 'Greek nymph')]
[('subDeity', 'http://www.wikidata.org/entity/Q45713'), ('subDeityName', 'titan')]
[('subDeity', 'http://www.wikidata.org/entity/Q1157625'), ('subDeityName', 'daemon')]
[('subDeity', 'http://www.wikidata.org/entity/Q201411'), ('subDeityName', 'Horae')]
[('subDeity', 'http://www.wikidata.org/entity/Q101609'), ('subDeityName', 'Twelve Olympians')]
[('subDeity', 'http://www.wikidata.org/entity/Q184353'), ('subDeityName', 'Charites')]
[('subDeity', 'http://www.wikidata.org/entity/Q878099'), ('subDeityName', 'Greek primordial deity')]
[('subDeity', 'http://www.wikidata.org/entity/Q66016'), ('subDeityName', 'Muses')]
[('subDeity', 'http://www.wikidata.org/entity/Q180287'), ('subDeityName', 'Moirae')]
[('subDeity', 'http://www.wikidata.org/entity/Q191192'), ('subDeityName', 'Giants')]
[('subDeity', 'http://www.

In [6]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the SUBCLASSES of a specialDeity
# GETTING deity --> subDeities --> specialDeity, checking

queryString = """
SELECT DISTINCT ?subDeity ?subDeityName (COUNT(*) as ?tot) #?deity ?deityName #?specialDeity ?specialDeityName # ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885.                 
    ?subDeity wdt:P279 ?specialDeity. #subclass of
    ?deity wdt:P31 ?subDeity. # instance of
    
    
   
    # get the label
    ?subDeity sc:name ?subDeityName.
    ?specialDeity sc:name ?specialDeityName.
    ?deity sc:name ?deityName.
    OPTIONAL {
        #?p sc:name ?pName.
    }.
    #FILTER (?p not in (wdt:P31))
}
GROUP BY ?subDeity ?subDeityName
ORDER BY desc(?tot)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('subDeity', 'http://www.wikidata.org/entity/Q3346693'), ('subDeityName', 'Greek nymph'), ('tot', '122')]
[('subDeity', 'http://www.wikidata.org/entity/Q191192'), ('subDeityName', 'Giants'), ('tot', '44')]
[('subDeity', 'http://www.wikidata.org/entity/Q3059255'), ('subDeityName', 'Greek sea god'), ('tot', '28')]
[('subDeity', 'http://www.wikidata.org/entity/Q45713'), ('subDeityName', 'titan'), ('tot', '28')]
[('subDeity', 'http://www.wikidata.org/entity/Q66016'), ('subDeityName', 'Muses'), ('tot', '19')]
[('subDeity', 'http://www.wikidata.org/entity/Q878099'), ('subDeityName', 'Greek primordial deity'), ('tot', '17')]
[('subDeity', 'http://www.wikidata.org/entity/Q1157625'), ('subDeityName', 'daemon'), ('tot', '17')]
[('subDeity', 'http://www.wikidata.org/entity/Q476682'), ('subDeityName', 'Anemoi'), ('tot', '10')]
[('subDeity', 'http://www.wikidata.org/entity/Q184353'), ('subDeityName', 'Charites'), ('tot', '8')]
[('subDeity', 'http://www.wikidata.org/entity/Q201411'), ('subD

In [9]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the SUBCLASSES of a specialDeity
# GETTING deity --> subDeities --> specialDeity, checking
# problem with Twelve Olymians --> only 1 deity ?

queryString = """
SELECT DISTINCT ?subDeity ?subDeityName ?deity ?deityName #?specialDeity ?specialDeityName # ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885.                 
    ?subDeity wdt:P279 ?specialDeity. #subclass of
    ?deity wdt:P31 ?subDeity. # instance of
    
    
   
    # get the label
    ?subDeity sc:name ?subDeityName.
    ?specialDeity sc:name ?specialDeityName.
    ?deity sc:name ?deityName.
    OPTIONAL {
        #?p sc:name ?pName.
    }.
    FILTER (?subDeity not in (wd:Q101609))
}
GROUP BY ?subDeity ?subDeityName
ORDER BY desc(?tot)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('subDeity', 'http://www.wikidata.org/entity/Q3346693'), ('subDeityName', 'Greek nymph'), ('deity', 'http://www.wikidata.org/entity/Q1075258'), ('deityName', 'Chloris')]
[('subDeity', 'http://www.wikidata.org/entity/Q45713'), ('subDeityName', 'titan'), ('deity', 'http://www.wikidata.org/entity/Q178417'), ('deityName', 'Eos')]
[('subDeity', 'http://www.wikidata.org/entity/Q878099'), ('subDeityName', 'Greek primordial deity'), ('deity', 'http://www.wikidata.org/entity/Q101322'), ('deityName', 'Tartarus')]
[('subDeity', 'http://www.wikidata.org/entity/Q191192'), ('subDeityName', 'Giants'), ('deity', 'http://www.wikidata.org/entity/Q2303713'), ('deityName', 'Anax')]
[('subDeity', 'http://www.wikidata.org/entity/Q191192'), ('subDeityName', 'Giants'), ('deity', 'http://www.wikidata.org/entity/Q3411759'), ('deityName', 'Pelorus')]
[('subDeity', 'http://www.wikidata.org/entity/Q180262'), ('subDeityName', 'Erinyes'), ('deity', 'http://www.wikidata.org/entity/Q371545'), ('deityName', 'M

In [8]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the SUBCLASSES of a specialDeity
# GETTING deity --> subDeities --> specialDeity, checking
# problem with Twelve Olymians --> only 1 deity ? need to use also wdt:P361 'part of'

queryString = """
SELECT DISTINCT ?subDeity ?subDeityName ?deity ?deityName #?specialDeity ?specialDeityName # ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885.                 
    ?subDeity wdt:P279 ?specialDeity. #subclass of
    ?deity wdt:P31 ?subDeity. # instance of
    
    
   
    # get the label
    ?subDeity sc:name ?subDeityName.
    ?specialDeity sc:name ?specialDeityName.
    ?deity sc:name ?deityName.
    OPTIONAL {
        #?p sc:name ?pName.
    }.
    FILTER (?subDeity in (wd:Q101609))
}
GROUP BY ?subDeity ?subDeityName
ORDER BY desc(?tot)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('subDeity', 'http://www.wikidata.org/entity/Q101609'), ('subDeityName', 'Twelve Olympians'), ('deity', 'http://www.wikidata.org/entity/Q41680'), ('deityName', 'Dionysus')]
1


In [12]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the SUBCLASSES of a specialDeity
# GETTING deity --> subDeities --> specialDeity, checking
# problem with Twelve Olymians --> only 1 deity ? need to use also wdt:P361 'part of'

queryString = """
SELECT DISTINCT ?subDeity ?subDeityName ?deity ?deityName #?specialDeity ?specialDeityName # ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885.                 
    ?subDeity wdt:P279 ?specialDeity. #subclass of
    ?deity wdt:P31|wdt:P361 ?subDeity. # instance of
    
    
   
    # get the label
    ?subDeity sc:name ?subDeityName.
    ?specialDeity sc:name ?specialDeityName.
    ?deity sc:name ?deityName.
    OPTIONAL {
        #?p sc:name ?pName.
    }.
    FILTER (?subDeity in (wd:Q101609))
}
GROUP BY ?subDeity ?subDeityName
ORDER BY asc(?deityName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('subDeity', 'http://www.wikidata.org/entity/Q101609'), ('subDeityName', 'Twelve Olympians'), ('deity', 'http://www.wikidata.org/entity/Q35500'), ('deityName', 'Aphrodite')]
[('subDeity', 'http://www.wikidata.org/entity/Q101609'), ('subDeityName', 'Twelve Olympians'), ('deity', 'http://www.wikidata.org/entity/Q37340'), ('deityName', 'Apollo')]
[('subDeity', 'http://www.wikidata.org/entity/Q101609'), ('subDeityName', 'Twelve Olympians'), ('deity', 'http://www.wikidata.org/entity/Q40901'), ('deityName', 'Ares')]
[('subDeity', 'http://www.wikidata.org/entity/Q101609'), ('subDeityName', 'Twelve Olympians'), ('deity', 'http://www.wikidata.org/entity/Q39503'), ('deityName', 'Artemis')]
[('subDeity', 'http://www.wikidata.org/entity/Q101609'), ('subDeityName', 'Twelve Olympians'), ('deity', 'http://www.wikidata.org/entity/Q37122'), ('deityName', 'Athena')]
[('subDeity', 'http://www.wikidata.org/entity/Q101609'), ('subDeityName', 'Twelve Olympians'), ('deity', 'http://www.wikidata.org/

In [13]:
# query 7

queryString = """
SELECT DISTINCT ?deity ?deityName
WHERE {
   # bind something
   ?deity wdt:P361 wd:Q101609. #?deity --'part of'--> 'Twelve Olympians'
   # get the label
   OPTIONAL {
       ?deity sc:name ?deityName.
    }.
}
ORDER BY asc(?deityName)
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('deity', 'http://www.wikidata.org/entity/Q35500'), ('deityName', 'Aphrodite')]
[('deity', 'http://www.wikidata.org/entity/Q37340'), ('deityName', 'Apollo')]
[('deity', 'http://www.wikidata.org/entity/Q40901'), ('deityName', 'Ares')]
[('deity', 'http://www.wikidata.org/entity/Q39503'), ('deityName', 'Artemis')]
[('deity', 'http://www.wikidata.org/entity/Q37122'), ('deityName', 'Athena')]
[('deity', 'http://www.wikidata.org/entity/Q40730'), ('deityName', 'Demeter')]
[('deity', 'http://www.wikidata.org/entity/Q44384'), ('deityName', 'Hephaestus')]
[('deity', 'http://www.wikidata.org/entity/Q38012'), ('deityName', 'Hera')]
[('deity', 'http://www.wikidata.org/entity/Q41484'), ('deityName', 'Hermes')]
[('deity', 'http://www.wikidata.org/entity/Q41419'), ('deityName', 'Hestia')]
[('deity', 'http://www.wikidata.org/entity/Q41127'), ('deityName', 'Poseidon')]
[('deity', 'http://www.wikidata.org/entity/Q34201'), ('deityName', 'Zeus')]
12


In [14]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the SUBCLASSES of a specialDeity
# GETTING deities --> subDeities --> specialDeity

queryString = """
SELECT DISTINCT ?deity ?deityName ?subDeity ?subDeityName #?specialDeity ?specialDeityName # ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885.                 
    ?subDeity wdt:P279 ?specialDeity. #subclass of
    ?deity wdt:P31 ?subDeity. # instance of
    
    
   
    # get the label
    ?subDeity sc:name ?subDeityName.
    ?specialDeity sc:name ?specialDeityName.
    ?deity sc:name ?deityName.
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('deity', 'http://www.wikidata.org/entity/Q8140979'), ('deityName', 'Menoetius'), ('subDeity', 'http://www.wikidata.org/entity/Q45713'), ('subDeityName', 'titan')]
[('deity', 'http://www.wikidata.org/entity/Q1075258'), ('deityName', 'Chloris'), ('subDeity', 'http://www.wikidata.org/entity/Q3346693'), ('subDeityName', 'Greek nymph')]
[('deity', 'http://www.wikidata.org/entity/Q176283'), ('deityName', 'Aoide'), ('subDeity', 'http://www.wikidata.org/entity/Q66016'), ('subDeityName', 'Muses')]
[('deity', 'http://www.wikidata.org/entity/Q505207'), ('deityName', 'Melete'), ('subDeity', 'http://www.wikidata.org/entity/Q66016'), ('subDeityName', 'Muses')]
[('deity', 'http://www.wikidata.org/entity/Q690388'), ('deityName', 'Mneme'), ('subDeity', 'http://www.wikidata.org/entity/Q66016'), ('subDeityName', 'Muses')]
[('deity', 'http://www.wikidata.org/entity/Q203673'), ('deityName', 'Kallisto'), ('subDeity', 'http://www.wikidata.org/entity/Q3346693'), ('subDeityName', 'Greek nymph')]
[('d

In [15]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the SUBCLASSES of a specialDeity
# GETTING deities --> subDeities --> specialDeity
# TRYING TO USE path patterns

queryString = """
SELECT DISTINCT ?deity ?deityName ?specialDeity ?specialDeityName #?subDeity ?subDeityName # ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885. 
                
    ?deity wdt:P31/wdt:P279* ?specialDeity. # all instance of specialDeity or its subClass
       
    # get the label
    ?specialDeity sc:name ?specialDeityName.
    ?deity sc:name ?deityName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('deity', 'http://www.wikidata.org/entity/Q8140979'), ('deityName', 'Menoetius'), ('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity')]
[('deity', 'http://www.wikidata.org/entity/Q5081127'), ('deityName', 'Acmon'), ('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity')]
[('deity', 'http://www.wikidata.org/entity/Q1075258'), ('deityName', 'Chloris'), ('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity')]
[('deity', 'http://www.wikidata.org/entity/Q145915'), ('deityName', 'Nekhbet'), ('specialDeity', 'http://www.wikidata.org/entity/Q146083'), ('specialDeityName', 'Ancient Egyptian deity')]
[('deity', 'http://www.wikidata.org/entity/Q29166150'), ('deityName', 'Thrace'), ('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity')]
[('deity', 'http://www.wikidata.org/entity/Q58373'), ('deityName', 'Amun'), ('specia

In [17]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the SUBCLASSES of a specialDeity
# GETTING deities --> subDeities --> specialDeity
# TRYING TO USE path patterns

    # wd:Q176283('Aoide')
    # wd:Q505207('Melete')
    # wd:Q690388('Mneme')
    # wd:Q103975('Calliope')
    # wd:Q103992('Melpomene')
    # wd:Q104002('Urania')
    # wd:Q104012('Euterpe')
    # wd:Q104042('Terpsichore')
    # wd:Q104413('Erato')
    # wd:Q104425('Polyhymnia')
    # wd:Q108019('Thalia')
    # wd:Q5957283('Hypate')
    # wd:Q6821184('Mese')
    # wd:Q3523555('Thelxinoë')
    # wd:Q9049602('Nete')
    # wd:Q12899635('Dia')
    # wd:Q12899764('Enope')
    # wd:Q12899831('Eunice')
    # wd:Q12900215('Helice')

# CHECKING IF MUSES NOW ARE PRESENT
queryString = """
SELECT DISTINCT ?deity ?deityName ?specialDeity ?specialDeityName #?subDeity ?subDeityName # ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885. 
                
    ?deity wdt:P31/wdt:P279* ?specialDeity. # all instance of specialDeity or its subClass
       
    # get the label
    ?specialDeity sc:name ?specialDeityName.
    ?deity sc:name ?deityName.

    FILTER (?deity in (wd:Q104413,wd:Q176283,wd:Q505207))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('deity', 'http://www.wikidata.org/entity/Q176283'), ('deityName', 'Aoide'), ('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity')]
[('deity', 'http://www.wikidata.org/entity/Q104413'), ('deityName', 'Erato'), ('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity')]
[('deity', 'http://www.wikidata.org/entity/Q505207'), ('deityName', 'Melete'), ('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity')]
3


In [18]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the SUBCLASSES of a specialDeity
# GETTING deities --> subDeities --> specialDeity
# TRYING TO USE path patterns
# checking if the twelve olympians are preset

# wd:Q35500 ('Aphrodite')
# wd:Q37340 ('Apollo')
# wd:Q40901 ('Ares')
# wd:Q39503 ('Artemis')
# wd:Q37122 ('Athena')
# wd:Q40730 ('Demeter')
# wd:Q44384 ('Hephaestus')
# wd:Q38012 ('Hera')
# wd:Q41484 ('Hermes')
# wd:Q41419 ('Hestia')
# wd:Q41127 ('Poseidon')
# wd:Q34201 ('Zeus')

# CHECKING IF MUSES NOW ARE PRESENT
queryString = """
SELECT DISTINCT ?deity ?deityName ?specialDeity ?specialDeityName #?subDeity ?subDeityName # ?p ?pName 
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885. 
                
    ?deity wdt:P31/wdt:P279* ?specialDeity. # all instance of specialDeity or its subClass
       
    # get the label
    ?specialDeity sc:name ?specialDeityName.
    ?deity sc:name ?deityName.

    FILTER (?deity in (wd:Q35500,wd:Q37340,wd:Q40901,wd:Q39503,wd:Q37122,wd:Q40730,wd:Q44384,wd:Q38012,wd:Q41484,wd:Q41419,wd:Q41127,wd:Q34201))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('deity', 'http://www.wikidata.org/entity/Q34201'), ('deityName', 'Zeus'), ('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity')]
[('deity', 'http://www.wikidata.org/entity/Q44384'), ('deityName', 'Hephaestus'), ('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity')]
[('deity', 'http://www.wikidata.org/entity/Q38012'), ('deityName', 'Hera'), ('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity')]
[('deity', 'http://www.wikidata.org/entity/Q41484'), ('deityName', 'Hermes'), ('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity')]
[('deity', 'http://www.wikidata.org/entity/Q37122'), ('deityName', 'Athena'), ('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity')]
[('deity', 'http://www.wikidata.org/entity/Q40730'), ('deityName', 'Demeter'), ('specialDeity', 'http://www

#### GOOD, working fine with patterns, + No need to use wdt:P361 ('part of') for 'Twelve Olympians'

In [19]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the SUBCLASSES of a specialDeity
# GETTING deities --> subDeities --> specialDeity
# TRYING TO USE path patterns

# GROUPING AND COUNTIN

queryString = """
SELECT DISTINCT ?specialDeity ?specialDeityName (COUNT(*) AS ?totalDeities)
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885. 
                
    ?deity wdt:P31/wdt:P279* ?specialDeity. # all instance of specialDeity or its subClass
       
    # get the label
    ?specialDeity sc:name ?specialDeityName.
    ?deity sc:name ?deityName.
}
GROUP BY ?specialDeity ?specialDeityName 
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('specialDeity', 'http://www.wikidata.org/entity/Q146083'), ('specialDeityName', 'Ancient Egyptian deity'), ('totalDeities', '156')]
[('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity'), ('totalDeities', '1197')]
2


In [20]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the SUBCLASSES of a specialDeity
# GETTING deities --> subDeities --> specialDeity
# TRYING TO USE path patterns

# GROUPING AND COUNTIN

queryString = """
SELECT DISTINCT ?culture ?cultureName (COUNT(*) AS ?totalDeities)
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885. 
                
    ?deity wdt:P31/wdt:P279* ?specialDeity. # all instance of specialDeity or its subClass
       
    # get the label
    ?specialDeity sc:name ?specialDeityName.
    ?culture sc:name ?cultureName.
}
GROUP BY ?culture ?cultureName 
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('culture', 'http://www.wikidata.org/entity/Q11772'), ('cultureName', 'Ancient Greece'), ('totalDeities', '1246')]
[('culture', 'http://www.wikidata.org/entity/Q11768'), ('cultureName', 'Ancient Egypt'), ('totalDeities', '281')]
2


In [21]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the SUBCLASSES of a specialDeity
# GETTING deities --> subDeities --> specialDeity
# TRYING TO USE path patterns

# GROUPING AND COUNTIN

queryString = """
SELECT DISTINCT ?culture ?cultureName (COUNT(?deity) AS ?totalDeities)
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885. 
                
    ?deity wdt:P31/wdt:P279* ?specialDeity. # all instance of specialDeity or its subClass
       
    # get the label
    ?specialDeity sc:name ?specialDeityName.
    ?culture sc:name ?cultureName.
}
GROUP BY ?culture ?cultureName 
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('culture', 'http://www.wikidata.org/entity/Q11772'), ('cultureName', 'Ancient Greece'), ('totalDeities', '1246')]
[('culture', 'http://www.wikidata.org/entity/Q11768'), ('cultureName', 'Ancient Egypt'), ('totalDeities', '281')]
2


In [24]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

# Getting all deities from Ancient Egypt & Ancient Greece
# need to go in depth with subclasses to get all the deities, like the muses
# Getting all the SUBCLASSES of a specialDeity
# GETTING deities --> subDeities --> specialDeity
# TRYING TO USE path patterns

# GROUPING AND COUNTING

queryString = """
SELECT DISTINCT ?specialDeity ?specialDeityName
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)
        (wd:Q11768)
    }
    ?specialDeity wdt:P2596 ?culture;
                wdt:P279 wd:Q178885. 
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('specialDeity', 'http://www.wikidata.org/entity/Q146083')]
[('specialDeity', 'http://www.wikidata.org/entity/Q22989102')]
2


#### PROBLEMs, 

A) GOOGLE SAYS THAT THERE ARE: (Possible reason: It's a snapshot of the wikidata.)
1) 3142 greek gods/godesses 
2) The ancient Egyptians worshipped over 1,400 different gods and goddesses

B) If I group by ?specialDeity it loses some pairs meanwhile if I group by ?culture it doesn't:

1) with specialDeity:
    -   [('specialDeity', 'http://www.wikidata.org/entity/Q146083'), ('specialDeityName', 'Ancient Egyptian deity'), ('totalDeities', '156')]
    -   [('specialDeity', 'http://www.wikidata.org/entity/Q22989102'), ('specialDeityName', 'Greek deity'), ('totalDeities', '1197')]
2) with culture:
    - [('culture', 'http://www.wikidata.org/entity/Q11772'), ('cultureName', 'Ancient Greece'), ('totalDeities', '1246')]
    - [('culture', 'http://www.wikidata.org/entity/Q11768'), ('cultureName', 'Ancient Egypt'), ('totalDeities', '281')]

### Final query for this task

In [25]:
# Compare ancient Greece with ancient Egypt and find how many deities each one has. 

# --- old findings ---
# TASK 1: Q11768 ('Ancient Egypt')
# wd:Q41127 ('Poseidon')
# wd:Q22989102 ('Greek deity')
# wd:Q178885 ('deity')
# wd:Q101609 (Twelve Olympians')
# --- end old findings ---

# wd:Q22989102 ('Greek deity') --wdt:P2596 ('culture')-->    wd:Q11772 ('Ancient Greece')
#                              --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q146083 (Ancient Egyptian deity') --wdt:P2596 ('culture')-----> wd:Q11768 ('Ancient Egypt')
#                                      --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')

# wd:Q104413('Erato') --wdt:P31 ('instance of')--> wd:Q66016 ('Muses')--wdt:P279 ('subclass of')--> wd:Q22989102 ('Greek deity')

queryString = """
SELECT DISTINCT ?culture ?cultureName (COUNT(?deity) AS ?totalDeities)
WHERE {
    # bind something
    VALUES (?culture){
        (wd:Q11772)   # ('Ancient Greece')
        (wd:Q11768)   # ('Ancient Egypt')
    }
    ?specialDeity wdt:P2596 ?culture;       # --wdt:P2596 ('culture')-->
                wdt:P279 wd:Q178885.        # --wdt:P279 ('subclass of')--> wd:Q178885 ('deity')
                
    # all direct instance or instances of subClasses of specialDeity 
    ?deity wdt:P31/wdt:P279* ?specialDeity. # --wdt:P31 ('instance of')-->
                                            # --wdt:P279 ('subclass of')--> 
       
    # get the label
    ?specialDeity sc:name ?specialDeityName.
    ?culture sc:name ?cultureName.
}
GROUP BY ?culture ?cultureName 
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('culture', 'http://www.wikidata.org/entity/Q11772'), ('cultureName', 'Ancient Greece'), ('totalDeities', '1246')]
[('culture', 'http://www.wikidata.org/entity/Q11768'), ('cultureName', 'Ancient Egypt'), ('totalDeities', '281')]
2
